# EDI Data Extraction

## Project Initiation

In [ ]:
import re
import pandas as pd
from random import randint,choice,choices
from english_words import english_words_lower_alpha_set as words
from edi_codes import *
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
from IPython.display import clear_output
import pyodbc
import numpy as np
import pyodbc

In [ ]:
words.add('effective')

In [ ]:
table_names_2023 = ['ATG_SYS_EDI880_1_20230922',
'ATG_SYS_EDI880_1_20230927',
'ATG_SYS_EDI880_1_20231004',
'ATG_SYS_EDI880_1_20231005',
'ATG_SYS_EDI880_1_20231006',
'ATG_SYS_EDI880_1_20231009',
'ATG_SYS_EDI880_1_20231010',
'ATG_SYS_EDI880_1_20231011',
'ATG_SYS_EDI880_1_20231013',
'ATG_SYS_EDI880_1_20231020',
'ATG_SYS_EDI880_1_20231026',
'ATG_SYS_EDI880_1_20231027',
'ATG_SYS_EDI880_1_20231030',
'ATG_SYS_EDI880_2_20230922',
'ATG_SYS_EDI880_20230830',
'ATG_SYS_EDI880_20230831',
'ATG_SYS_EDI880_20230921',
'ATG_SYS_EDI880_20230922',
'ATG_SYS_EDI880_20230925',
'ATG_SYS_EDI880_20230926',
'ATG_SYS_EDI880_20230927',
'ATG_SYS_EDI880_20230928',
'ATG_SYS_EDI880_20231003',
'ATG_SYS_EDI880_20231004',
'ATG_SYS_EDI880_20231005',
'ATG_SYS_EDI880_20231006',
'ATG_SYS_EDI880_20231009',
'ATG_SYS_EDI880_20231010',
'ATG_SYS_EDI880_20231011',
'ATG_SYS_EDI880_20231012',
'ATG_SYS_EDI880_20231013',
'ATG_SYS_EDI880_20231016',
'ATG_SYS_EDI880_20231017',
'ATG_SYS_EDI880_20231018',
'ATG_SYS_EDI880_20231019',
'ATG_SYS_EDI880_20231020',
'ATG_SYS_EDI880_20231023',
'ATG_SYS_EDI880_20231024',
'ATG_SYS_EDI880_20231026',
'ATG_SYS_EDI880_20231027',
'ATG_SYS_EDI880_20231030',
'ATG_SYS_EDI880_20231031',
'ATG_SYS_EDI880_20231104',
'ATG_SYS_EDI880_20231111',
'ATG_SYS_EDI880_20231118',
'ATG_SYS_EDI880_20231125',
'ATG_SYS_EDI880_20231202',
'ATG_SYS_EDI880_20231209',
'ATG_SYS_EDI880_20231216',
'ATG_SYS_EDI880_20231223',
'ATG_SYS_EDI880_20231230'
]
table_names_2024 = [
    'ATG_SYS_EDI880_20240106',
    'ATG_SYS_EDI880_20240113',
    'ATG_SYS_EDI880_20240120',
    'ATG_SYS_EDI880_20240127',
    'ATG_SYS_EDI880_20240203',
    'ATG_SYS_EDI880_20240210',
    'ATG_SYS_EDI880_20240217',
    'ATG_SYS_EDI880_20240224',
    'ATG_SYS_EDI880_20240302',
    'ATG_SYS_EDI880_20240309',
    'ATG_SYS_EDI880_20240316',
    'ATG_SYS_EDI880_20240323',
    'ATG_SYS_EDI880_20240330',
    'ATG_SYS_EDI880_20240406'
]

## SQL Preparation

In [ ]:
server = 'apu'
database = 'source_MitchellGrocery_2023'
connectionString = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Integrated Security={True};Autocommit={True};Trusted_Connection=yes;'
conn = pyodbc.connect(connectionString)
cursor_1 = conn.cursor()

In [ ]:
all_edi_lines = []
for table in table_names_2023:
    print(table)
    cursor_1.execute(f"SELECT Edi880a\n"
                     f"FROM [source_MitchellGrocery_2023].[dbo].[{table}]")
    rows = cursor_1.fetchall()
    rows = list(rows)
    all_edi_lines.extend(rows)

In [ ]:
full_2023_data = []
open("All_EDI_Files_2023_Original_EDIs.txt",'w').close()
for table in table_names_2023[:]:
    clear_output(wait=False)
    print(table)
    cursor_1.execute(f"SELECT Edi880a\n"
                     f"FROM {table}")
    data = cursor_1.fetchall()
    with open("All_EDI_Files_2023_Original_EDIs.txt",'a') as file:
        for row in data:
            file.write(str(row).replace("'","").replace("(","").replace(")","").replace(',','').replace('-',' ')+'\n')

In [ ]:
full_data_hdr_allowance_df = pd.read_sql_query(f'select * from [source_MitchellGrocery_2024].[dbo].[ATG_EDI_Invoice_Hdr_Allowances]', conn)

In [ ]:
full_data_hdr_df = pd.read_sql_query(f'select * from [source_MitchellGrocery_2024].[dbo].[ATG_EDI_Invoice_Hdr]', conn)

In [ ]:
full_data_dtl_allowance_df = pd.read_sql_query(f'select * from [source_MitchellGrocery_2024].[dbo].[ATG_EDI_Invoice_Dtl_Allowances]', conn)

## Reading SQL Queries

In [ ]:
with open('All_EDI_Files_2023_Original_EDIs.txt','r') as file:
    lines = file.read()
    full_data = lines.split('\n')
print(len(full_data))

## Editing Invoices

In [ ]:
new_lines = []
for line in full_data:
    if('880 ' in line):
        new_line = line.replace('880 ','').replace('\n','').replace('  ',' ').replace('  ',' ')\
                         .replace('  ',' ').replace('  ',' ').replace('  ',' ').replace('  ',' ')\
                            .replace('  ',' ').replace('  ',' ').replace('  ',' ').replace('  ',' ')\
                                .replace('  ',' ').replace('  ',' ').replace('  ',' ').replace('  ',' ')\
                                    .replace('  ',' ').replace('  ',' ').replace('  ',' ').replace('  ',' ')\
                                        .replace('  ',' ').replace('  ',' ').replace('  ',' ').replace('  ',' ')\
                                            .replace('  ',' ').replace('  ',' ').replace('  ',' ').replace('  ',' ')\
                                                .replace('N1 ','N1').replace('N2 ','N2').replace('N3 ','N3')\
                                                    .replace('N4 ','N4').replace('N9 ','N9')
        new_line = new_line[1:]
        new_lines.append(new_line)

## Splitting Invoices

In [ ]:
invoices = []
invoice_lines = []

### Edited Invoices

In [ ]:
for line in new_lines:
    if('DATA' in line or ("G01" in line[:4] and len(invoice_lines)>4)):
        invoices.append('\n'.join(invoice_lines))
        invoice_lines.clear()
        invoice_lines.append(line)
    else:
        invoice_lines.append(line)
print(invoices[0])

In [ ]:
_ = invoices.pop(0)
del _

In [ ]:
print(invoices[0])

### Original Invoices

In [ ]:
for line in full_data:
    if('#880' in line[:4] or ("G01" in line[:10] and len(invoice_lines)>4)):
        invoices.append('\n'.join(invoice_lines))
        invoice_lines.clear()
        invoice_lines.append(line)
    else:
        invoice_lines.append(line)
print(invoices[0])

In [ ]:
_ = invoices.pop(0)
del _

In [ ]:
for _ in range(10):
    print(invoices[randint(0,len(invoices)-1)])

## Creating Tables

### Edited Invoices

#### Invoice Header

In [ ]:
all_header_columns = []

In [ ]:
st01_transaction_set_header_columns = ['ST01_143_Transaction_Set_Identifier_Numeric_Code','ST01_143_Transaction_Set_Identifier_Code',
                                       'ST01_329_Transaction_Set_Control_Number','ST01_1705_Implementation_Convention_Preference']
all_header_columns.extend(st01_transaction_set_header_columns)

In [ ]:
g01_invoice_identification_columns = ['G01_245_Invoice_Date','G01_076_Invoice_Number',
                                    'G01_323_Purchase_Order_Date','G01_324_Purchase_Order_Number']
all_header_columns.extend(g01_invoice_identification_columns)

In [ ]:
n9_reference_identification_initial_columns = []
for iteration in range(10):
    for col in ['N9_128_Reference_Number_Qualifier_Code','N9_127_Reference_Number','N9_369_Free_Form_Description']:
        n9_reference_identification_initial_columns.append(f"{col}_{iteration+1}")
all_header_columns.extend(n9_reference_identification_initial_columns)

In [ ]:
g62_date_time_columns = []
for iteration in range(5):
    for col in ['G62_432_Date_Qualifier','G62_373_Date',]:
        g62_date_time_columns.append(f"{col}_{iteration+1}")
all_header_columns.extend(g62_date_time_columns)

In [ ]:
nte_note_special_instruction_columns = []
for iteration in range(20):
    for col in ['NTE_363_Note_Reference_Code','NTE_003_Free_Form_Message']:
        nte_note_special_instruction_columns.append(f"{col}_{iteration+1}")
all_header_columns.extend(nte_note_special_instruction_columns)

In [ ]:
g27_carrier_detail_columns = []
for iteration in range(5):
    for col in ['G27_091_Transportation_Method_Type_Code','G27_207_Equipment_Number','G27_387_Routing']:
        g27_carrier_detail_columns.append(f"{col}_{iteration+1}")
all_header_columns.extend(g27_carrier_detail_columns)

In [ ]:
g23_terms_of_sales_columns = []
for iteration in range(20):
    for col in ['G23_336_Terms_Type_Code','G23_333_Terms_Basis_Date_Code','G23_282_Terms_Start_Date','G23_283_Terms_Due_Date_Qualifier',
                'G23_338_Terms_Discount_Percent','G23_370_Terms_Discount_Due_Date','G23_351_Terms_Discount_Days_Due',
                'G23_446_Terms_Net_Due_Date','G23_386_Terms_Net_Days','G23_362_Terms_Discount_Amount',
                'G23_391_Discounted_Amount','G23_390_Amount_Subject_To_Terms_Discount',
                'G23_003_Free_Form_Message']:
        g23_terms_of_sales_columns.append(f"{col}_{iteration+1}")
all_header_columns.extend(g23_terms_of_sales_columns)

In [ ]:
for col in g23_terms_of_sales_columns:
    print(f'({col})')

In [ ]:
g25_fob_information_columns = ['G25_146_Shipment_Method_Of_Payment','G25_433_FOB_Point_Code','G25_434_FOB_Point']
all_header_columns.extend(g25_fob_information_columns)

In [ ]:
n1_n2_n3_n4_name_loop_columns = []
for iteration in range(7):
    for col in ['N1_098_Entity_Identification_Code','N1_093_Name','N1_066_Identification_Code_Qualifier','N1_067_Identification_Code','N3_166_Address_Info',
                'N4_019_City_Name','N4_156_State_Province_Code','N4_116_Postal_Code']:
        n1_n2_n3_n4_name_loop_columns.append(f"{col}_{iteration+1}")
all_header_columns.extend(n1_n2_n3_n4_name_loop_columns)

In [ ]:
all_invoice_summarizing_columns = ['G31_382_Number_Of_Units_Shipped','G31_355_Unit_Or_Basis_For_Measurement_Code_1','G31_081_Weight',
                                   'G31_355_Unit_Or_Basis_For_Measurement_Code_2','G31_183_Volume','G31_355_Unit_Or_Basis_For_Measurement_Code_3',
                                   'G33_361_Amount']
final_summary_list = []
for iterations in range(3):
    for col in all_invoice_summarizing_columns:
        final_summary_list.append(f"{col}_{iterations+1}")
all_header_columns.extend(final_summary_list)
all_invoice_summarizing_columns = final_summary_list

In [ ]:
code_columns = ["N9_128_Reference_Number_Qualifier_Numeric_Code_1",
"N9_128_Reference_Number_Qualifier_Numeric_Code_2",
"N9_128_Reference_Number_Qualifier_Numeric_Code_3",
"N9_128_Reference_Number_Qualifier_Numeric_Code_4",
"N9_128_Reference_Number_Qualifier_Numeric_Code_5",
"N9_128_Reference_Number_Qualifier_Numeric_Code_6",
"N9_128_Reference_Number_Qualifier_Numeric_Code_7",
"N9_128_Reference_Number_Qualifier_Numeric_Code_8",
"N9_128_Reference_Number_Qualifier_Numeric_Code_9",
"N9_128_Reference_Number_Qualifier_Numeric_Code_10",
"NTE_363_Note_Reference_Numeric_Code_1",
"NTE_363_Note_Reference_Numeric_Code_2",
"NTE_363_Note_Reference_Numeric_Code_3",
"NTE_363_Note_Reference_Numeric_Code_4",
"NTE_363_Note_Reference_Numeric_Code_5",
"NTE_363_Note_Reference_Numeric_Code_6",
"NTE_363_Note_Reference_Numeric_Code_7",
"NTE_363_Note_Reference_Numeric_Code_8",
"NTE_363_Note_Reference_Numeric_Code_9",
"NTE_363_Note_Reference_Numeric_Code_10",
"NTE_363_Note_Reference_Numeric_Code_11",
"NTE_363_Note_Reference_Numeric_Code_12",
"NTE_363_Note_Reference_Numeric_Code_13",
"NTE_363_Note_Reference_Numeric_Code_14",
"NTE_363_Note_Reference_Numeric_Code_15",
"NTE_363_Note_Reference_Numeric_Code_16",
"NTE_363_Note_Reference_Numeric_Code_17",
"NTE_363_Note_Reference_Numeric_Code_18",
"NTE_363_Note_Reference_Numeric_Code_19",
"NTE_363_Note_Reference_Numeric_Code_20",
"G27_091_Transportation_Method_Type_Numeric_Code_1",
"G27_091_Transportation_Method_Type_Numeric_Code_2",
"G27_091_Transportation_Method_Type_Numeric_Code_3",
"G27_091_Transportation_Method_Type_Numeric_Code_4",
"G27_091_Transportation_Method_Type_Numeric_Code_5",
"G23_336_Terms_Type_Numeric_Code_1",
"G23_333_Terms_Basis_Date_Numeric_Code_1",
"G23_336_Terms_Type_Numeric_Code_2",
"G23_333_Terms_Basis_Date_Numeric_Code_2",
"G23_336_Terms_Type_Numeric_Code_3",
"G23_333_Terms_Basis_Date_Numeric_Code_3",
"G23_336_Terms_Type_Numeric_Code_4",
"G23_333_Terms_Basis_Date_Numeric_Code_4",
"G23_336_Terms_Type_Numeric_Code_5",
"G23_333_Terms_Basis_Date_Numeric_Code_5",
"G23_336_Terms_Type_Numeric_Code_6",
"G23_333_Terms_Basis_Date_Numeric_Code_6",
"G23_336_Terms_Type_Numeric_Code_7",
"G23_333_Terms_Basis_Date_Numeric_Code_7",
"G23_336_Terms_Type_Numeric_Code_8",
"G23_333_Terms_Basis_Date_Numeric_Code_8",
"G23_336_Terms_Type_Numeric_Code_9",
"G23_333_Terms_Basis_Date_Numeric_Code_9",
"G23_336_Terms_Type_Numeric_Code_10",
"G23_333_Terms_Basis_Date_Numeric_Code_10",
"G23_336_Terms_Type_Numeric_Code_11",
"G23_333_Terms_Basis_Date_Numeric_Code_11",
"G23_336_Terms_Type_Numeric_Code_12",
"G23_333_Terms_Basis_Date_Numeric_Code_12",
"G23_336_Terms_Type_Numeric_Code_13",
"G23_333_Terms_Basis_Date_Numeric_Code_13",
"G23_336_Terms_Type_Numeric_Code_14",
"G23_333_Terms_Basis_Date_Numeric_Code_14",
"G23_336_Terms_Type_Numeric_Code_15",
"G23_333_Terms_Basis_Date_Numeric_Code_15",
"G23_336_Terms_Type_Numeric_Code_16",
"G23_333_Terms_Basis_Date_Numeric_Code_16",
"G23_336_Terms_Type_Numeric_Code_17",
"G23_333_Terms_Basis_Date_Numeric_Code_17",
"G23_336_Terms_Type_Numeric_Code_18",
"G23_333_Terms_Basis_Date_Numeric_Code_18",
"G23_336_Terms_Type_Numeric_Code_19",
"G23_333_Terms_Basis_Date_Numeric_Code_19",
"G23_336_Terms_Type_Numeric_Code_20",
"G23_333_Terms_Basis_Date_Numeric_Code_20",
"G25_433_FOB_Point_Numeric_Code",
"N1_098_Entity_Identification_Numeric_Code_1",
"N1_066_Identification_Numeric_Code_Qualifier_1",
"N1_098_Entity_Identification_Numeric_Code_2",
"N1_066_Identification_Numeric_Code_Qualifier_2",
"N1_098_Entity_Identification_Numeric_Code_3",
"N1_066_Identification_Numeric_Code_Qualifier_3",
"N1_098_Entity_Identification_Numeric_Code_4",
"N1_066_Identification_Numeric_Code_Qualifier_4",
"N1_098_Entity_Identification_Numeric_Code_5",
"N1_066_Identification_Numeric_Code_Qualifier_5",
"N1_098_Entity_Identification_Numeric_Code_6",
"N1_066_Identification_Numeric_Code_Qualifier_6",
"N1_067_Identification_Numeric_Code_6",
"N1_098_Entity_Identification_Numeric_Code_7",
"N1_066_Identification_Numeric_Code_Qualifier_7",
"N1_067_Identification_Numeric_Code_7",
"G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_1_1",
"G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_2_1",
"G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_3_1",
"G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_1_2",
"G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_2_2",
"G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_3_2",
"G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_1_3",
"G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_2_3",
"G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_3_3"]
all_header_columns.extend(code_columns)

In [ ]:
for col in all_header_columns:
    print(col)

In [ ]:
print(f"{len(all_header_columns):,}")

#### Invoice Header Allowance

In [ ]:
all_header_allowance_columns = []

In [ ]:
g72_g73_allowance_or_charge_columns = ['ATG_Hdr_Ref','G72_340_Allowance_Or_Charge_Code','G72_331_Allowance_Or_Charge_Method_Of_Handling_Code',
                                       'G72_359_Allowance_Or_Charge_Rate','G72_339_Allowance_Or_Charge_Quantity','G72_355_Unit_Or_Basis_For_Measurement_Code',
                                       'G72_360_Allowance_Or_Charge_Total_Amount','G73_369_Free_Form_Description']
all_header_allowance_columns.extend(g72_g73_allowance_or_charge_columns)

In [ ]:
for col in all_header_allowance_columns:
    print(f"{col}")

In [ ]:
print(f"{len(all_header_allowance_columns):,}")

#### Invoice Detail

In [ ]:
all_item_detail_invoice_columns = []

In [ ]:
g17_item_detail_invoice = ['ATG_Ref','ATG_Hdr_Ref','G17_358_Quantity_Invoiced','G17_355_Unit_Or_Basis_For_Measurement_Code','G17_237_Item_List_Cost',
                        'G17_438_UPC_Case_Code','G17_235_Product_Service_ID_Qualifier_1','G17_234_Product_Service_ID_1',
                        'G17_235_Product_Service_ID_Qualifier_2','G17_234_Product_Service_ID_2',
                        'G17_382_Number_Of_Units_Shipped','G17_355_Unit_Or_Basis_For_Measurement',
                        'G17_782_Monetary_Amount','Miscellaneous_Five_Digit_Number']
all_item_detail_invoice_columns.extend(g17_item_detail_invoice)

In [ ]:
g69_line_item_detail_description_columns = ['G69_369_Free_Form_Description']
all_item_detail_invoice_columns.extend(g69_line_item_detail_description_columns)

In [ ]:
g20_item_packing_detail = ['G20_356_Pack_Number_Inner_Pack_Units_Per_Outer_Pack_Unit']
all_item_detail_invoice_columns.extend(g20_item_packing_detail)

In [ ]:
for col in all_item_detail_invoice_columns:
    print(col)

In [ ]:
print(f"{len(all_item_detail_invoice_columns):,}")

#### Invoice Detail Allowances Or Charges

In [ ]:
all_invoice_detail_allow_or_charge_columns = []

In [ ]:
g72_g73_allowance_or_charge_columns = ['ATG_Hdr_Ref','ATG_Dtl_Ref','G72_340_Allowance_Or_Charge_Code','G72_331_Allowance_Or_Charge_Method_Of_Handling_Code',
                'G72_359_Allowance_Or_Charge_Rate','G72_339_Allowance_Charge_Quantity','G72_355_Unit_Basis_For_Measurement',
                'G72_360_Allowance_Or_Charge_Total_Amount','G73_369_Free_Form_Description']
all_invoice_detail_allow_or_charge_columns.extend(g72_g73_allowance_or_charge_columns)

In [ ]:
for col in ['ATG_Hdr_Ref','ATG_Dtl_Ref','G72_340_Allowance_Or_Charge_Code','G72_331_Allowance_Or_Charge_Method_Of_Handling_Code',
                'G72_359_Allowance_Or_Charge_Rate','G72_339_Allowance_Charge_Quantity','G72_355_Unit_Basis_For_Measurement',
                'G72_360_Allowance_Or_Charge_Total_Amount','G73_369_Free_Form_Description']:
    print(col)

In [ ]:
print(f"{len(all_invoice_detail_allow_or_charge_columns):,}")

### Original Invoices

#### Invoice Header

In [ ]:
all_header_columns = []

In [ ]:
st01_transaction_set_header_columns = ['ST01_143_Transaction_Set_Identifier_Numeric_Code','ST01_143_Transaction_Set_Identifier_Code',
                                       'ST01_329_Transaction_Set_Control_Number','ST01_1705_Implementation_Convention_Preference']
all_header_columns.extend(st01_transaction_set_header_columns)

In [ ]:
g01_invoice_identification_columns = ['G01_245_Invoice_Date','G01_076_Invoice_Number',
                                    'G01_323_Purchase_Order_Date','G01_324_Purchase_Order_Number','G01_424_Vendor_Order_Number',
                                    'G01_474_Master_Reference_Number','G01_472_Link_Sequence_Number',
                                    'G01_640_Transaction_Type_Numeric_Code','G01_640_Transaction_Type_Numeric_Code']
all_header_columns.extend(g01_invoice_identification_columns)

In [ ]:
n9_reference_identification_initial_columns = ['N9_128_Reference_Qualifier_Numeric_Code','N9_128_Reference_Qualifier_Code','N9_127_Reference_Number',
                                               'N9_369_Free_Form_Description',
                                               'N9_373_Date','N9_337_Time','N9_623_Time_Numeric_Code','N9_623_Time_Code']
all_header_columns.extend(n9_reference_identification_initial_columns)

In [ ]:
g61_contact_columns = ['G61_366_Contact_Function_Numeric_Code','G61_366_Contact_Function_Code','G61_93_Name',
                       'G61_365_Communication_Number_Qualifier_Numeric_Code','G61_365_Communication_Number_Qualifier','G61_364_Communication_Number',
                       'G61_443_Contact_Inquiry_Reference']
all_header_columns.extend(g61_contact_columns)

In [ ]:
g62_date_time_columns = ['G62_432_Date_Qualifier_Numeric_Code','G62_432_Date_Qualifier_Code','G62_373_Date',
                         'G62_176_Time_Qualifier_Numeric_Code','G62_176_Time_Qualifier_Code',
                         'G62_337_Time','G62_623_Time_Code_Numeric_Code','G62_623_Time_Code']
all_header_columns.extend(g62_date_time_columns)

In [ ]:
nte_note_special_instruction_columns = ['NTE_363_Note_Reference_Numeric_Code','NTE_363_Note_Reference_Code','NTE_352_Free_Form_Message']
all_header_columns.extend(nte_note_special_instruction_columns)

In [ ]:
g27_carrier_detail_columns = ['G27_91_Transportation_Method_Type_Numeric_Code','G27_091_Transportation_Method_Type_Code',
                              'G27_206_Equipment_Initial','G27_207_Equipment_Number','G27_140_Standard_Carrier_Alpha_Code',
                              'G27_387_Routing','G27_368_Shipment_Order_Status_Numeric_Code','G27_368_Shipment_Order_Status_Code']
all_header_columns.extend(g27_carrier_detail_columns)

In [ ]:
g23_terms_of_sales_columns = ['G23_336_Terms_Type_Numeric_Code','G23_336_Terms_Type_Code','G23_333_Terms_Basis_Date_Numeric_Code',
                              'G23_333_Terms_Basis_Date_Numeric_Code','G23_282_Terms_Start_Date','G23_283_Terms_Due_Date_Qualifier_Numeric_Code',
                              'G23_283_Terms_Due_Date_Qualifier',
                            'G23_338_Terms_Discount_Percent','G23_370_Terms_Discount_Due_Date','G23_351_Terms_Discount_Days_Due',
                            'G23_446_Terms_Net_Due_Date','G23_386_Terms_Net_Days','G23_362_Terms_Discount_Amount',
                            'G23_391_Discounted_Amount','G23_390_Amount_Subject_To_Terms_Discount',
                            'G23_343_Installment_Invoice_Amount_Due','G23_342_Percent_of_Invoice_Payable'
                            'G23_003_Free_Form_Message','G23_713_Installment_Group_Indicator']
all_header_columns.extend(g23_terms_of_sales_columns)

In [ ]:
g25_fob_information_columns = ['G25_146_Shipment_Method_Of_Payment','G25_433_FOB_Point_Code','G25_434_FOB_Point']
all_header_columns.extend(g25_fob_information_columns)

In [ ]:
n1_n2_n3_n4_name_loop_columns = ['N1_098_Entity_Identification_Numeric_Code','N1_098_Entity_Identification_Code','N1_093_Name',
                                 'N1_066_Identification_Numeric_Code_Qualifier','N1_066_Identification_Code_Qualifier','N1_067_Identification_Code',
                                 'N1_067_Identification_Code','N1_706_Entity_Relationship_Numeric_Code','N1_706_Entity_Relationship_Code',
                                 'N1_098_Entity_Identifier_Numeric_Code','N1_098_Entity_Identifier_Code','N2_093_Name_1','N2_093_Name_2',
                                 'N3_166_Address_Info_1','N3_166_Address_Info_2','N4_019_City_Name','N4_156_State_Province_Code','N4_116_Postal_Code',
                                 'N4_026_Country_Code','N4_309_Location_Qualifier_Numeric_Code','N4_309_Location_Qualifier_Code',
                                 'N4_310_Location_Identifier']
all_header_columns.extend(n1_n2_n3_n4_name_loop_columns)

In [ ]:
all_invoice_summarizing_columns = ['G31_382_Number_Of_Units_Shipped','G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_1'
                                   'G31_355_Unit_Or_Basis_For_Measurement_Code_1','G31_081_Weight_1',
                                   'G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_2','G31_355_Unit_Or_Basis_For_Measurement_Code_2',
                                   'G31_183_Volume','G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_3','G31_355_Unit_Or_Basis_For_Measurement_Code_3',
                                   'G31_398_Order_Sizing_Factor','G31_417_Price_Bracket_Identifier','G31_107_Payment_Method_Numeric_Code',
                                   'G31_107_Payment_Method_Code','G31_380_Quantity','G31_081_Weight_2'
                                   'G33_361_Amount']
all_header_columns.extend(all_invoice_summarizing_columns)

#### Invoice Header Allowance

In [ ]:
all_header_allowance_columns = []

In [ ]:
g72_g73_allowance_or_charge_columns = ['ATG_Hdr_Ref','G72_340_Allowance_Or_Charge_Numeric_Code','G72_340_Allowance_Or_Charge_Code',
                                       'G72_331_Allowance_Or_Charge_Method_Of_Handling_Numeric_Code','G72_331_Allowance_Or_Charge_Method_Of_Handling_Code',
                                       'G72_341_Allowance_Or_Charge_Number','G72_769_Exception_Number',
                                       'G72_359_Allowance_Or_Charge_Rate','G72_339_Allowance_Or_Charge_Quantity','G72_355_Unit_Or_Basis_For_Measurement_Numeric_Code'
                                       'G72_355_Unit_Or_Basis_For_Measurement_Code','G72_360_Allowance_Or_Charge_Total_Amount',
                                       'G72_332_Percent','G72_828_Dollar_Basis_For_Percent','G72_770_Option_Number','G73_369_Free_Form_Description']
all_header_allowance_columns.extend(g72_g73_allowance_or_charge_columns)

#### Invoice Detail

In [ ]:
all_item_detail_invoice_columns = []

In [ ]:
g17_g69_item_detail_invoice = ['ATG_Hdr_Ref','G17_358_Quantity_Invoiced','G17_355_Unit_Or_Basis_For_Measurement_Numeric_Code_1',
                           'G17_355_Unit_Or_Basis_For_Measurement_Code_1','G17_237_Item_List_Cost',
                           'G17_438_UPC_Case_Code','G17_235_Product_Service_ID_Qualifier_Numeric_Code_1',
                           'G17_235_Product_Service_ID_Qualifier_Code_1','G17_234_Product_Service_ID_1',
                           'G17_235_Product_Service_ID_Qualifier_Numeric_Code_2','G17_235_Product_Service_ID_Qualifier_Code_1',
                           'G17_234_Product_Service_ID_2','G17_417_Price_Bracket_Identifier',
                           'G17_382_Number_Of_Units_Shipped','G17_355_Unit_Or_Basis_For_Measurement_Numeric_Code_2',
                           'G17_355_Unit_Or_Basis_For_Measurement_Code_2','G17_439_Price_List_Number',
                           'G17_440_Price_List_Issue_Number','G17_782_Monetary_Amount','G69_369_Free_Form_Description']
all_item_detail_invoice_columns.extend(g17_g69_item_detail_invoice)

In [ ]:
g19_line_item_detail_columns = ['G19_382_Number_Of_Units_Shipped','G19_355_Unit_Or_Basis_For_Measurement_Numeric_Code','G19_355_Unit_Or_Basis_For_Measurement_Code',
                                'G19_383_Quantity_Difference','G19_368_Shipment_Order_Status_Numeric_Code','G19_368_Shipment_Order_Status_Code',
                                'G19_381_Price_Reason_Numeric_Code','G19_381_Price_Reason_Code','G19_445_Terms_Exception_Numeric_Code','G19_445_Terms_Exception_Code',
                                'G19_438_UPC_Case_Code','G19_235_Product_Service_ID_Qualifier_Numeric_Code','G19_235_Product_Service_ID_Qualifier',
                                'G19_234_Product_Service_ID']
all_item_detail_invoice_columns.extend(g19_line_item_detail_columns)

In [ ]:
g20_item_packing_detail_columns = ['G20_356_Pack','G20_357_Size','G20_355_Unit_Or_Basis_For_Measurement_Numeric_Code_1',
                                   'G20_355_Unit_Or_Basis_For_Measurement_Code_1','G20_081_Weight','G20_355_Unit_Or_Basis_For_Measurement_Numeric_Code_2',
                                   'G20_355_Unit_Or_Basis_For_Measurement_Code_2','G20_183_Volume','G20_355_Unit_Or_Basis_For_Measurement_Numeric_Code_3',
                                   'G20_355_Unit_Or_Basis_For_Measurement_Code_3','G20_397_Color','G20_810_Inner_Pack']
all_item_detail_invoice_columns.extend(g20_item_packing_detail_columns)

In [ ]:
n9_reference_identification_initial_columns = ['N9_128_Reference_Number_Qualifier_Numeric_Code','N9_128_Reference_Number_Qualifier_Code',
                                               'N9_127_Reference_Number','N9_369_Free_Form_Description',
                                               'N9_373_Date','N9_337_Time','N9_623_Time_Numeric_Code','N9_623_Time_Code']
all_item_detail_invoice_columns.extend(n9_reference_identification_initial_columns)

## Functions To Extract Data

### Invoice Header

#### ST01 --> Transaction Set Header

In [ ]:
def transaction_set_header(text):
    text = str(text)
    transaction_set_header_list = ["NULL"]*4
    text = text.replace('ST01','').replace('ST880','').replace('ST','')
    transaction_set_identifier_code_found = False
    for group in range(len(text)-3):
        transaction_set_identifier_code = text[group:group+3]
        for key,item in st01_transaction_set_identifier_codes.items():
            if(key==transaction_set_identifier_code):
                transaction_set_identifier_code_found = True
                transaction_set_header_list[0] = key
                transaction_set_header_list[1] = item
        if(transaction_set_identifier_code_found):
            break
    if(len(text.split(' '))>=2):
        if(len(text.split(' ')[1])>=4 and len(text.split(' ')[1])<=9):
            transaction_set_header_list[2] = text.split(' ')[1]
    if(len(text.split(' '))==3):
        transaction_set_header_list[3] = text.split(' ')[-1]

#### G01 --> Invoice Identification

In [ ]:
def invoice_identification(text):
    text = str(text).split(' ')
    invoice_identification_list = ["NULL","NULL","NULL","NULL"]
    first_word = text[0].replace('G0100200000','').replace('INV','').replace("CI",'')
    invoice_identification_list[0] = '20'+first_word[:2]+'/'+first_word[2:4]+'/'+first_word[4:6]
    invoice_identification_list[1] = first_word[6:]
    if(len(text)>1):
        if(len(text[1])==6):
            invoice_identification_list[2] = text[1]
        elif(len(text[1])<6):
            invoice_identification_list[3] = text[1]
        else:
            invoice_identification_list[2] = '20'+text[1][:2]+'/'+text[1][2:4]+'/'+text[1][4:6]
            invoice_identification_list[3] = text[1][6:]
    return invoice_identification_list

#### N9 --> Reference Identification

In [ ]:
def reference_identification(text):
    text = str(text).upper()
    text = text.replace("N900300000","")
    text = text.split(" ")
    reference_identification_numbers = ['NULL','NULL','NULL']
    reference_identification_qualifier_code = text[0][:3]
    for code,item in n9_reference_identification_qualifier_codes.items():
        if(code==reference_identification_qualifier_code):
            reference_identification_numbers[0] = item
    if(reference_identification_numbers[0]=="NULL"):
        reference_identification_qualifier_code = text[0][:2]
        for code,item in n9_reference_identification_qualifier_codes.items():
            if(code==reference_identification_qualifier_code):
                reference_identification_numbers[0] = item  
    if(len(reference_identification_qualifier_code)==2):
        reference_identification_numbers[1] = text[0][2:]
    else:
        reference_identification_numbers[1] = text[0][3:]
    if(len(text)>1):
        reference_identification_numbers[2] = ' '.join(text[1:])
    return reference_identification_numbers

#### G62 --> Date/Time

In [ ]:
def date_time(text):
    text = str(text)
    text = text.replace('G6200500000','')
    final_date_time = ["NULL",'NULL']
    date_qualifier_code = text[:2]
    for code,item in g62_date_qualifier_codes.items():
        if code==date_qualifier_code:
            final_date_time[0] = item
    date = text[2:]
    date = '20'+date[:2]+'/'+date[2:4]+'/'+date[4:6]
    final_date_time[1] = date
    return final_date_time

#### NTE --> Note/Special Instruction

In [ ]:
def note_special_instruction(text):
    special_instruction_list = ["NULL","NULL"]
    text = str(text)
    text = text.split(' ')
    note_reference_code = text[0][-3:]
    for code,item in nte_note_reference_codes.items():
        if note_reference_code == code:
            special_instruction_list[0] = item
    if(len(text)>1):
        special_instruction_list[1] = ' '.join(text[1:])
    return special_instruction_list

#### G27 --> Carrier Detail

In [ ]:
def carrier_detail(text):
    carrier_detail_list = ["NULL","NULL","NULL"]
    text = str(text).split(' ')
    pattern = re.compile(r"[A-Z]{0,1}[0-9]{3,6}[\-\/]{0,1}[0-9]{0,8}")
    transportation_type_code = text[0][-2:]
    for code,item in g27_carrier_detail_transportation_method_type_codes.items():
        if transportation_type_code == code:
            carrier_detail_list[0] = item
    if(carrier_detail_list[0] == "NULL"):
        transportation_type_code = text[0][-1]
        for code,item in g27_carrier_detail_transportation_method_type_codes.items():
            if transportation_type_code == code:
                carrier_detail_list[0] = item
    routing_words = []
    for word in range(1,len(text)):
        if(
            (re.search(pattern,text[word])==None)or
            (text[word] in words)or
            (text[word-1] in routing_words)or
            (
                (word<len(text)-1)and('%'==text[word+1])
            )
          ):
            routing_words.append(text[word])
    if(len(routing_words)>0):
        carrier_detail_list[2] = ' '.join(routing_words)
    if(len(text)>1):
        for word in range(1,len(text)):
            if(
                (not(text[word] in routing_words))and
                (re.match(pattern,text[word]))
              ):
                carrier_detail_list[1] = text[word]
    return carrier_detail_list

#### G23 --> Terms of Sale

In [ ]:
def terms_of_sale(text):
    text = str(text)
    original = text
    text = text.replace('-','')
    years = ['20','21','22','23','24']
    months = ['01','02','03','04','05','06','07','08','09','10','11','12']
    days = ['01','02','03','04','05','06','07','08','09']
    days.extend([str(x) for x in range(10,32)])
    text = text.split(' ')
    terms_of_sale_list = ["NULL"]*13
    date_one_word_index = (-1,-1)
    date_two_word_index = (-1,-1)
    if(text[0][-1]=='0' and len(text[1])==2):
        terms_type_code = text[1]
        for code,item in g23_terms_type_codes.items():
            if(code==terms_type_code):
                terms_of_sale_list[0] = item
        text.pop(0)
        text.pop(0)
    else:
        terms_type_code = text[0][-2:]
        for code,item in g23_terms_type_codes.items():
            if(code==terms_type_code):
                terms_of_sale_list[0] = item
        text.pop(0)
    for index in range(len(text)):
        text[index] = text[index].replace('0400000022','2').replace('020000000','').replace('000000022','2').replace('00000022','2').replace('00000000','').replace('0000000','').replace('000000','').replace('00000','')
    if(len(text[0])<6):
        for index in range(len(text[0])-1):
            for code,item in g23_terms_basis_date_codes.items():
                if(text[0][index:index+2]==code):
                    terms_of_sale_list[1] = item
    else:
        for word in text:
            if(len(word)>0):
                if(len(word)==6):
                    if((word[:2] in years) and (word[2:4] in months) and (word[4:] in days) and (terms_of_sale_list[2]=="NULL")):
                        terms_of_sale_list[2] = '20'+word[:2]+'/'+word[2:4]+'/'+word[4:]
                        date_one_word_index = (text.index(word),0)
                    elif(word[:2] in years and word[2:4] in months and word[4:] in days and terms_of_sale_list[5]=="NULL"):
                        terms_of_sale_list[5] = '20'+word[:2]+'/'+word[2:4]+'/'+word[4:]
                        date_two_word_index = (text.index(word),0)
                elif(len(word)<4 and '%' in word and len(word)>1):
                    pass
                elif(len(word)>6):
                    if(terms_of_sale_list[2]=="NULL" or terms_of_sale_list[5]=="NULL"):
                        for index,letter in enumerate(word[:-5]):
                            if((word[index:index+2] in years) and (word[index+2:index+4] in months) and (word[index+4:index+6] in days) and (terms_of_sale_list[2]=="NULL")):
                                terms_of_sale_list[2] = '20'+word[index:index+2]+'/'+word[index+2:index+4]+'/'+word[index+4:index+6]
                                date_one_word_index = (text.index(word),index+6)
                            elif((word[index:index+2] in years) and (word[index+2:index+4] in months) and (word[index+4:index+6] in days) and (terms_of_sale_list[5]=="NULL")):
                                terms_of_sale_list[5] = '20'+word[index:index+2]+'/'+word[index+2:index+4]+'/'+word[index+4:index+6]
                                date_two_word_index = (text.index(word),index+6)
                    if(text.index(word)==date_one_word_index[0] and text.index(word)==date_two_word_index[0]):
                        terms_basis_date_code = word[date_one_word_index[1]:date_two_word_index[1]-6]
                        for index,number in enumerate(terms_basis_date_code[:-1]):
                            for code,item in g23_terms_basis_date_codes.items():
                                if(code==terms_basis_date_code[index:index+2]):
                                    terms_of_sale_list[3] = item
                    elif(text.index(word)==date_two_word_index[0] and text.index(word)>date_one_word_index[0]):
                        terms_basis_date_code = word[:date_two_word_index[1]-6]
                        for index,number in enumerate(terms_basis_date_code[:-1]):
                            for code,item in g23_terms_basis_date_codes.items():
                                if(code==terms_basis_date_code[index:index+2]):
                                    terms_of_sale_list[3] = item
            if(not(word.isnumeric())):
                for description_word in text[text.index(word)-1:-1]:
                    if(len(description_word)<3 and description_word.isnumeric() and text[text.index(description_word)+1]=='%'):
                        terms_of_sale_list[12] = ' '.join(text[text.index(word):])
                        return terms_of_sale_list
                    elif('%' in description_word and description_word[0].isnumeric()):
                        terms_of_sale_list[12] = ' '.join(text[text.index(word):])
                        return terms_of_sale_list
                    else:
                        terms_of_sale_list[12] = ' '.join(text[text.index(word):])
                        return terms_of_sale_list
    return terms_of_sale_list

In [ ]:
def terms_of_sale_from_original(text):
    text = str(text)
    print(text)
    print(f"{'-'*18}{text[18:20]} TERMS TYPE CODE\n"
          f"{'-'*20}{text[20:22]} TERMS BASIS DATE CODE\n"
          f"{'-'*22}{text[22:28]} TERMS START DATE\n"
          f"{'-'*28}{text[28:30]} TERMS DUE DATE QUALIFIER\n"
          f"{'-'*30}{text[30:31]} DECIMAL POSITIONS 1\n"
          f"{'-'*31}{text[31:37]} TERMS DISCOUNT PERCENT\n"
          f"{'-'*37}{text[37:43]} TERMS DISCOUNT DUE DATE\n"
          f"{'-'*43}{text[43:44]} DECIMAL POSITIONS 2\n"
          f"{'-'*44}{text[44:47]} TERMS DISCOUNT DAYS DUE\n"
          f"{'-'*47}{text[47:53]} TERMS NET DUE DATE\n"
          f"{'-'*53}{text[53:54]} DECIMAL POSITIONS 3\n"
          f"{'-'*54}{text[54:57]} TERMS NET DAYS\n"
          f"{'-'*57}{text[57:58]} DECIMAL POSITIONS 4\n"
          f"{'-'*58}{text[58:68]} TERMS DISCOUNT AMOUNT\n"
          f"{'-'*68}{text[68:69]} DECIMAL POSITIONS 5\n"
          f"{'-'*69}{text[69:79]} DISCOUNTED AMOUNT\n"
          f"{'-'*79}{text[79:80]} DECIMAL POSITIONS 6\n"
          f"{'-'*80}{text[80:90]} AMOUNT SUBJECT TO TERMS DISCOUNT\n"
          f"{'-'*90}{text[90:]} FREE FORM MESSAGE")
    terms_of_sale_list = ["NULL"]*16
    return None
    #TERMS TYPE CODE
    terms_of_sale_list[0] = text[18]+text[19]
    for key,item in g23_terms_type_codes.items():
        if(key==terms_of_sale_list[0]):
            terms_of_sale_list[1] = item
    if(terms_of_sale_list[1]=="NULL"):
        terms_of_sale_list[0] = "NULL"

    #TERMS BASIS CODE
    terms_of_sale_list[2] = text[20]+text[21]
    for key,item in g23_terms_basis_date_codes.items():
        if(key==terms_of_sale_list[2]):
            terms_of_sale_list[3] = item
    if(terms_of_sale_list[3]=="NULL"):
        terms_of_sale_list[2] = "NULL"
    
    #TERMS START DATE
    terms_of_sale_list[4] = '20'+text[22:24]+'/'+text[24:26]+'/'+text[26:28]
    if(' ' in text[22:28]):
        terms_of_sale_list[4] = "NULL"
    
    #TERMS DUE DATE QUALIFIER
    terms_of_sale_list[5] = text[28]+text[29]
    for key,item in g23_terms_due_date_qualifier_codes.items():
        if(key==terms_of_sale_list[5]):
            terms_of_sale_list[6] = item
    if(terms_of_sale_list[6]=="NULL"):
        terms_of_sale_list[5] = "NULL"
    
    #NO. DECIMAL POSITIONS 1
    decimal_positions = int(text[30])
    #TERMS DISCOUNT DAYS DUE
    terms_of_sale_list[7] = int(text[31:37])/(10**decimal_positions)
    #TERMS NET DUE DATE
    terms_of_sale_list[8] = '20'+text[37:39]+'/'+text[39:41]+'/'+text[41:43]
    if(' ' in text[37:43]):
        terms_of_sale_list[8] = "NULL"
    decimal_positions = int(text[43])
    terms_of_sale_list[9] = int(text[44:47])/(10**decimal_positions)
    terms_of_sale_list[10] = '20'+text[47:49]+'/'+text[49:51]+'/'+text[51:53]
    decimal_positions = int(text[53])
    terms_of_sale_list[11] = int(text[54:57])/(10**decimal_positions)
    decimal_positions = int(text[57])
    print(terms_of_sale_list)
    terms_of_sale_list[12] = int(text[58:68])/(10**decimal_positions)
    decimal_positions = int(text[68])
    terms_of_sale_list[13] = int(text[69:79])/(10**decimal_positions)
    decimal_positions = int(text[79])
    terms_of_sale_list[14] = int(text[80:89])/(10**decimal_positions)
    terms_of_sale_list[15] = text[107:]
    return terms_of_sale_list

In [196]:
def terms_of_sale_terms_type_code(text):
    text = str(text).replace('G23008000000','').replace('G2300800000','').replace('G230080000','').replace('G23021000000','')
    while(text[0]==' '):
        text = text[1:]
    if(len(text)>=2):
        for key,item in g23_terms_type_codes.items():
            if(key==text[:2]):
                return item
        return "None"

In [198]:
def terms_of_sale_terms_basis_date_code(text):
    text = str(text).replace('G23008000000','').replace('G2300800000','').replace('G230080000','').replace('G23021000000','')
    years = ['20','21','22','23','24']
    months = ['01','02','03','04','05','06','07','08','09','10','11','12']
    days = ['01','02','03','04','05','06','07','08','09']
    days.extend([str(x) for x in range(10,32)])
    first_word = text.split(' ')[0]
    if(len(first_word)>=3 and not(first_word[:2] in years and first_word[2:4] in months and first_word[4:6] in days)):
        code = first_word[2:]
        for key,item in g23_terms_basis_date_codes.items():
            if(key==code):
                return item
        return "None"

In [ ]:
for invoice in invoices[:]:
    for line in invoice.split('\n'):
        if('G23' in line[:3]):
            print(line)
            #print(terms_of_sale_terms_basis_date_code(line))
            print()

#### G25 --> F.O.B. Information

In [ ]:
def fob_information(text):
    fob_information_list = ["NULL","NULL","NULL"]
    text = str(text).split(' ')
    first_word = text[0][text[0].index("00000")+5:]
    shipment_method_of_payment_code = first_word[:2]
    fob_point_code = first_word[2:4]
    for code,item in g25_shipment_method_of_payment.items():
        if(shipment_method_of_payment_code==code):
            fob_information_list[0] = item
    for code,item in g25_fob_point_codes.items():
        if(fob_point_code==code):
            fob_information_list[1] = item
    fob_point_description = []
    if(len(first_word)>4):
        fob_point_description.append(first_word[4:])
    if(len(text)>1):
        for word in text[1:]:
            fob_point_description.append(word)
    if(len(fob_point_description)>0):
        fob_information_list[2] = ' '.join(fob_point_description)
    return fob_information_list

#### N1 --> Name

In [ ]:
def name(text):
    name_list = ["NULL","NULL","NULL","NULL"]
    text = str(text).split(' ')
    first_word = text[0].replace('N101000000','')
    pattern = re.compile(r"[0-9]{5,20}")
    if(len(first_word)<3):
        entity_identifier_code = first_word
        for code,item in n1_entity_identifier_codes.items():
            if(entity_identifier_code==code):
                name_list[0] = item
        for index,word in enumerate(text[1:-1]):
            if(word.isnumeric() and re.search(pattern,text[index+1])):
                name_list[1] = ' '.join(text[1:index])
    else:
        entity_identifier_code = first_word[:2]
        for code,item in n1_entity_identifier_codes.items():
            if(entity_identifier_code==code):
                name_list[0] = item
        text[0] = first_word[2:]
        for index,word in enumerate(text[:-1]):
            if(word.isnumeric() and re.search(pattern,text[index+1])):
                name_list[1] = ' '.join(text[:index])
                for code,item in n1_identifier_code_qualifier_codes.items():
                    if(word==code):
                        name_list[2] = item
    
    if(re.search(pattern,text[-1])):
        name_list[3] = text[-1]
    return name_list

#### N3 --> Address Information

In [ ]:
def address_information(text):
    text = str(text)
    return [text.replace('N301200000','')]

#### N4 --> Geographic Location

In [ ]:
def geographic_location(text):
    text = str(text)
    geographic_location_list =  ["NULL","NULL","NULL"]
    text = text.replace('N401300000','')
    text = text.split(' ')
    geographic_location_list[0] = ' '.join(text[:-1])
    geographic_location_list[1] = text[-1][:2]
    if(len(text[-1])<8):
        geographic_location_list[2] = text[-1][2:]
    else:
        geographic_location_list[2] = text[-1][2:7]+'-'+text[-1][7:].replace('--','-').replace('--','-').replace('--','-').replace('--','-')
    return geographic_location_list

#### G31 --> Total Invoice Quantity

In [ ]:
def total_invoice_quantity(text):
    original = text
    print(original)
    while(text[-1]==' '):
        text = text[:-1]
    text = str(text).replace('-','').replace(' ','0')
    if('G31025000001' in text):
        decimal_places = 1
    elif('G31025000002' in text):
        decimal_places = 2
    else:
        decimal_places = 0
    total_invoice_quantity_list = ["NULL"]*6
    text = text.replace('G3102500000000000000','').replace('G310250000020000000','').replace('G31025000002000000','').replace('G3102500000200000','')\
    .replace('G310250000020000','').replace('G31025000002000','').replace('G3102500000200','')\
    .replace('G31025000001000000','').replace('G3102500000100000','').replace('G310250000000000000','').replace('G310250000000000000000','')\
    .replace('G31025000000000000000','').replace('G3102500000000000000','')\
    .replace('G3102500000000000000','').replace('G310250000000000000','').replace('G31025000000000000','').replace('G3102500000000000','')\
    .replace('G310250000000000','').replace('G31025000000000','').replace('G3102500000000','').replace('G310250000000','').replace('G31025000000','')\
    .replace('G3102500000','').replace('G310250000','').replace('G31025000','').replace('G3102500','').replace('G310250','').replace('G31025','')
    pattern1 = re.compile(r"^[0-9]{1,10}[A-Z0-9]{2}")
    if(re.search(pattern1,text)):
        total_invoice_quantity_list[0] = int(re.search(pattern1,text[:7]).group()[:-2])/(10**decimal_places)
        g31_unit_or_basis_for_measurement_code = re.search(pattern1,text).group()
        final_code = g31_unit_or_basis_for_measurement_code[-2]+g31_unit_or_basis_for_measurement_code[-1]
        for code,item in g31_unit_or_basis_for_measurement_codes.items():
            if(final_code==code):
                total_invoice_quantity_list[1] = item
    else:
        g31_unit_or_basis_for_measurement_code = text[:2]
        for code,item in g31_unit_or_basis_for_measurement_codes.items():
            if(g31_unit_or_basis_for_measurement_code==code):
                total_invoice_quantity_list[1] = item
    if(text.replace(g31_unit_or_basis_for_measurement_code,'').isnumeric()):
        decimal_places = int(text.replace(g31_unit_or_basis_for_measurement_code,'')[0])
    weight_section = text.replace(f'{g31_unit_or_basis_for_measurement_code}00000000000','').replace(f'{g31_unit_or_basis_for_measurement_code}0000000000','')\
                         .replace(f'{g31_unit_or_basis_for_measurement_code}000000000','').replace(f'{g31_unit_or_basis_for_measurement_code}00000000','')\
                         .replace(f'{g31_unit_or_basis_for_measurement_code}0000000','').replace(f'{g31_unit_or_basis_for_measurement_code}200000000','')\
                         .replace(f'{g31_unit_or_basis_for_measurement_code}300000000','').replace(f'{g31_unit_or_basis_for_measurement_code}20000000','')\
                         .replace(f'{g31_unit_or_basis_for_measurement_code}30000000','').replace(f'{g31_unit_or_basis_for_measurement_code}00000000000','')\
                         .replace(f'{g31_unit_or_basis_for_measurement_code}0000000000','').replace(f'{g31_unit_or_basis_for_measurement_code}000000000','')\
                         .replace(f'{g31_unit_or_basis_for_measurement_code}00000000','').replace(f'{g31_unit_or_basis_for_measurement_code}0000000','')\
                         .replace(f'{g31_unit_or_basis_for_measurement_code}2000000','').replace(f'{g31_unit_or_basis_for_measurement_code}3000000','')\
                         .replace(f'{g31_unit_or_basis_for_measurement_code}200000','').replace(f'{g31_unit_or_basis_for_measurement_code}300000','')\
                         .replace(f'{g31_unit_or_basis_for_measurement_code}20000','').replace(f'{g31_unit_or_basis_for_measurement_code}30000','')\
                         .replace(f'{g31_unit_or_basis_for_measurement_code}2000','').replace(f'{g31_unit_or_basis_for_measurement_code}3000','')\
                         .replace(f'{g31_unit_or_basis_for_measurement_code}200','').replace(f'{g31_unit_or_basis_for_measurement_code}300','')\
                         .replace(f'{g31_unit_or_basis_for_measurement_code}20','').replace(f'{g31_unit_or_basis_for_measurement_code}30','')\
                         .replace(f'{g31_unit_or_basis_for_measurement_code}1000000','').replace(f'{g31_unit_or_basis_for_measurement_code}4000000','')\
                         .replace(f'{g31_unit_or_basis_for_measurement_code}100000','').replace(f'{g31_unit_or_basis_for_measurement_code}400000','')\
                         .replace(f'{g31_unit_or_basis_for_measurement_code}10000','').replace(f'{g31_unit_or_basis_for_measurement_code}40000','')\
                         .replace(f'{g31_unit_or_basis_for_measurement_code}1000','').replace(f'{g31_unit_or_basis_for_measurement_code}4000','')\
                         .replace(f'{g31_unit_or_basis_for_measurement_code}100','').replace(f'{g31_unit_or_basis_for_measurement_code}400','')\
                         .replace(f'{g31_unit_or_basis_for_measurement_code}10','').replace(f'{g31_unit_or_basis_for_measurement_code}40','')\
                         .replace(f'{g31_unit_or_basis_for_measurement_code}0000000','').replace(f'{g31_unit_or_basis_for_measurement_code}5000000','')\
                         .replace(f'{g31_unit_or_basis_for_measurement_code}000000','').replace(f'{g31_unit_or_basis_for_measurement_code}500000','')\
                         .replace(f'{g31_unit_or_basis_for_measurement_code}00000','').replace(f'{g31_unit_or_basis_for_measurement_code}50000','')\
                         .replace(f'{g31_unit_or_basis_for_measurement_code}0000','').replace(f'{g31_unit_or_basis_for_measurement_code}5000','')\
                         .replace(f'{g31_unit_or_basis_for_measurement_code}000','').replace(f'{g31_unit_or_basis_for_measurement_code}500','')\
                         .replace(f'{g31_unit_or_basis_for_measurement_code}00','').replace(f'{g31_unit_or_basis_for_measurement_code}50','')\
                         .replace(f'{g31_unit_or_basis_for_measurement_code}','')
    while(len(weight_section)>2 and weight_section[0]=='0'):
        weight_section = weight_section[1:]
    if(len(weight_section)<2):
        return total_invoice_quantity_list
    if(weight_section[:2]=='LB'):
        total_invoice_quantity_list[3] = "Pound"
        return total_invoice_quantity_list
    else:
        try:
            total_invoice_quantity_list[2] = int(weight_section[:weight_section.index('LB')])/(10**decimal_places)
            total_invoice_quantity_list[3] = "Pound"
        except:
            try:
                total_invoice_quantity_list[2] = int(re.search(r"[0-9]{1,10}[A-Z]{2}",weight_section).group()[:-2])/(10**decimal_places)
                total_invoice_quantity_list_code = re.search(r"[0-9]{1,10}[A-Z]{2}",weight_section).group()[-2:]
            except:
                total_invoice_quantity_list[2] = 0
                total_invoice_quantity_list_code = re.search(r"[A-Za-z]{2}",weight_section).group()[-2:]
            for code,item in g31_unit_or_basis_for_measurement_codes.items():
                if(total_invoice_quantity_list_code==code):
                    total_invoice_quantity_list[3] = item
    volume_section = ""
    for key,item in g31_unit_or_basis_for_measurement_codes.items():
        if(item==str(total_invoice_quantity_list[3])):
            volume_section = weight_section.replace(str(total_invoice_quantity_list[2]).replace('.00','').replace('.0','').replace('.',''),'').replace(key,'')
            print(volume_section)
    if(len(volume_section)>0):
        while(len(volume_section)>0 and volume_section[-1]==' '):
            volume_section = volume_section[:-1]
        while(len(volume_section)>0 and volume_section[0]==' '):
            volume_section = volume_section[1:]
        if(len(volume_section)>0):
            decimal_places = int(volume_section[0])
            if(len(volume_section)>3):
                volume_section = volume_section[1:]
                while(volume_section[0]=='0'):
                    volume_section = volume_section[1:]
                if(len(volume_section)>0):
                    if(volume_section[:-2].isnumeric()):
                        total_invoice_quantity_list[4] = int(volume_section[:-2])/(10**decimal_places)
                        for code,item in g31_unit_or_basis_for_measurement_codes.items():
                            if(volume_section[-2:]==code):
                                total_invoice_quantity_list[5] = item
                    else:
                        total_invoice_quantity_list[4] = 0
                        for code,item in g31_unit_or_basis_for_measurement_codes.items():
                            if(volume_section[-2:]==code):
                                total_invoice_quantity_list[5] = item
    return total_invoice_quantity_list

#### G33 --> Total Dollars Summary

In [ ]:
def total_dollars_summary(text):
    text = str(text)
    print(text)
    if('G33026000002' in text):
        decimal_places = 2
    elif('G33026000001' in text):
        decimal_places = 1
    elif('G330260000000' in text):
        decimal_places = 0
    text = text.replace('G33026000002000000','').replace('G33026000001000000','').replace('G3302600000000000000','').replace('G330260000000000000','').replace('G33026000000000000','')\
        .replace('G3302600000200000','').replace('G3302600000100000','').replace('G3302600000000000','')\
        .replace('G330260000020000','').replace('G330260000010000','').replace('G330260000000000','')\
        .replace('G33026000002000','').replace('G33026000001000','').replace('G33026000000000','')\
        .replace('G3302600000200','').replace('G3302600000100','').replace('G3302600000000','')\
        .replace('G330260000020','').replace('G330260000010','').replace('G330260000000','')
    #while(text[0]=='0'):
    #    text = text[1:]
    return [round(int(text)/(10**decimal_places),2)]

### Header Allowances or Charges

#### G72 --> Allowance or Charge

In [ ]:
def allowance_or_charge(text):
    text = str(text)
    pattern = re.compile(r"[A-Z]{2}")
    allowance_rate = re.compile(r"[0]{3,100}[1-9]{1}[0-9]{0,3}[0]{3,100}")
    allowance_or_charge_list = ["NULL","NULL","0.00","0","NULL","NULL"]
    total_amount_pattern = re.compile(r"^[0-9]{1}[-]{0,1}[0]{1,25}[1-9]{1}[0-9]{0,6}")
    not_rate_pattern = re.compile(r"[0]{5,100}[A-Z]{2}")
    text = text.replace('G7201400000','').replace('G7202300000','').split(' ')
    decimal_places = 0
        #print(original_text)
    # Dealing with components of the first word
    if(len(text[0])<4):
        allowance_or_charge_code = text[0]
        for code,item in g72_allowance_or_charge_codes.items():
            if(code==allowance_or_charge_code):
                allowance_or_charge_list[0] = item
    else:
        allowance_or_charge_code = text[0][:-2]
        allowance_or_charge_method_of_handling_code = text[0][-2:]
        for code,item in g72_allowance_or_charge_codes.items():
            if(code==allowance_or_charge_code):
                allowance_or_charge_list[0] = item
        for code,item in g72_allowance_or_charge_method_of_handling_codes.items():
            if(code==allowance_or_charge_method_of_handling_code):
                allowance_or_charge_list[1] = item
    
    #Cleaning up the last word
    last_word = text[-1]
    #print(f"{original_text}\n{original_last_word}\n{last_word}\n")
    last_word = last_word.split('-')
    if(len(text)>1):
# -------------------------------------------------------------------------------------------------------------------
# This condition checks if the word has a length of exactly 2 indicating that it only contains the allowance or charge
# method of handling code
        if(len(text[1])==2):
            allowance_or_charge_method_of_handling_code = text[1] 
            for code,item in g72_allowance_or_charge_method_of_handling_codes.items():
                if(code==allowance_or_charge_method_of_handling_code):
                    allowance_or_charge_list[1] = item
# -------------------------------------------------------------------------------------------------------------------
# This condition checks if the length of the word is exactly 10 meaning it could either only include the total amount
# or only the allowance or charge rate
        elif(len(text[1])==10):
            decimal_places = int(text[1][0])
            if(re.search(total_amount_pattern,text[1])):
                total_amount = re.search(total_amount_pattern,text[1]).group()[2:]
                total_amount = int(total_amount)
                total_amount = round((total_amount)/(10**(decimal_places)),2)
                if(total_amount>10):
                    allowance_or_charge_list[5] = str(total_amount)
                else:
                    allowance_or_charge_list[2] = str(total_amount)
# -------------------------------------------------------------------------------------------------------------------
# This condition will check if there are letters indicating unit of measurement followed by the total amount
        elif(re.search(pattern,text[1]) and text[1][-1].isnumeric() and len(text[1])<26):
            unit_basis_for_measurement_code = re.search(pattern,text[1]).group()
            num_units = text[1][:text[1].index(unit_basis_for_measurement_code)]
            if(not(num_units[-5]=='0')):
                allowance_or_charge_list[3] = num_units[-5:]
            elif(not(num_units[-4]=='0')):
                allowance_or_charge_list[3] = num_units[-4:]
            elif(not(num_units[-3]=='0')):
                allowance_or_charge_list[3] = num_units[-3:]
            elif(not(num_units[-2]=='0')):
                allowance_or_charge_list[3] = num_units[-2:]
            else:
                allowance_or_charge_list[3] = num_units[-1]
            for code,item in g72_unit_or_basis_for_measurement_codes.items():
                if(code==unit_basis_for_measurement_code):
                    allowance_or_charge_list[4] = item
            total_amount = text[1][text[1].index(unit_basis_for_measurement_code)+len(unit_basis_for_measurement_code):]
            decimal_places = total_amount[1][0]
            while(total_amount[0]=='0' and len(total_amount)>0):
                total_amount = total_amount[1:]
            if(not(len(total_amount)==0)):
                total_amount = int(total_amount)
                allowance_or_charge_list[5] = str(round((total_amount)/(10**int(decimal_places)),2))
# -------------------------------------------------------------------------------------------------------------------
# This condition will check if there are letters indicating unit of measurement followed by the total amount and the
# allowance or charge rate at the beginning of the word
        elif(re.search(pattern,text[1]) and text[1][-1].isnumeric() and len(text[1])>26):
            unit_basis_for_measurement_code = re.search(pattern,text[1]).group()
            num_units = text[1][:text[2].index(unit_basis_for_measurement_code)]
            if(not(num_units[-5]=='0')):
                allowance_or_charge_list[3] = num_units[-5:]
            elif(not(num_units[-4]=='0')):
                allowance_or_charge_list[3] = num_units[-4:]
            elif(not(num_units[-3]=='0')):
                allowance_or_charge_list[3] = num_units[-3:]
            elif(not(num_units[-2]=='0')):
                allowance_or_charge_list[3] = num_units[-2:]
            else:
                allowance_or_charge_list[3] = num_units[-1]
            for code,item in g72_unit_or_basis_for_measurement_codes.items():
                if(code==unit_basis_for_measurement_code):
                    allowance_or_charge_list[4] = item
            total_amount = text[1][text[1].index(unit_basis_for_measurement_code)+len(unit_basis_for_measurement_code):]
            decimal_places = total_amount[1][0]
            total_amount = total_amount[1:]
            while(total_amount[0]=='0' and len(total_amount)>0):
                total_amount = total_amount[1:]
            if(not(len(total_amount)==0)):
                allowance_or_charge_list[5] = str(round((int(total_amount))/(10**int(decimal_places)),2))
            decimal_places = text[1][0]
            rate = re.findall(allowance_rate,text[1][1:])
            if(len(rate)==0):pass
            else:
                allowance_charge_rate = rate[0]
                if(text[1].index(allowance_charge_rate)<text[1].index(unit_basis_for_measurement_code)):
                    final_rate = re.search(r"[1-9]{1}[0-9]{0,4}",allowance_charge_rate).group()
                    if(final_rate[-2:]=='00'):
                        final_rate = final_rate[:-2]
                        allowance_or_charge_list[2] = str(round((int(final_rate))/(10**int(decimal_places)),2))
# -------------------------------------------------------------------------------------------------------------------
# This condition checks if the word ends with letters indicating the total amount is not here yet and if the string 
# is longer than 15 characters, that means the charge rate is at the beginning of the word followed by number of units                
        elif(re.search(pattern,text[1]) and not(text[1][-1].isnumeric()) and len(text[1])>15):
            decimal_places = text[1][0]
            unit_basis_for_measurement_code = re.search(pattern,text[1]).group()
            num_units = text[1][:text[1].index(unit_basis_for_measurement_code)]
            if(not(num_units[-5]=='0')):
                allowance_or_charge_list[3] = num_units[-5:]
            elif(not(num_units[-4]=='0')):
                allowance_or_charge_list[3] = num_units[-4:]
            elif(not(num_units[-3]=='0')):
                allowance_or_charge_list[3] = num_units[-3:]
            elif(not(num_units[-2]=='0')):
                allowance_or_charge_list[3] = num_units[-2:]
            else:
                allowance_or_charge_list[3] = num_units[-1]
            total_amount = text[1][text[1].index(unit_basis_for_measurement_code)+len(unit_basis_for_measurement_code):]
            while((len(total_amount)>0)and(total_amount[0]=='0')):
                total_amount = total_amount[1:]
            if(not(len(total_amount)==0)):
                allowance_or_charge_list[5] = str(round((int(total_amount))/((10)**int(decimal_places)),2))
    if(len(text)>2):
        if(re.search(pattern,text[2])):
            decimal_places = text[2][0]
            unit_basis_for_measurement_code = re.search(pattern,text[2]).group()
            num_units = text[2][:text[2].index(unit_basis_for_measurement_code)]
            if(not(num_units[-5]=='0')):
                allowance_or_charge_list[3] = num_units[-5:]
            elif(not(num_units[-4]=='0')):
                allowance_or_charge_list[3] = num_units[-4:]
            elif(not(num_units[-3]=='0')):
                allowance_or_charge_list[3] = num_units[-3:]
            elif(not(num_units[-2]=='0')):
                allowance_or_charge_list[3] = num_units[-2:]
            else:
                allowance_or_charge_list[3] = num_units[-1]
            for code,item in g72_unit_or_basis_for_measurement_codes.items():
                if(code==unit_basis_for_measurement_code):
                    allowance_or_charge_list[4] = item
            if(
                (abs(text[2].index(unit_basis_for_measurement_code)-len(text[2]))>2)or
                (len(text)>3)
              ):
                if(len(text)>3):
                    decimal_places = int(text[3][0])
                    total_amount = re.search(total_amount_pattern,text[3]).group()
                    total_amount = int(total_amount)
                    total_amount = str(round((total_amount)/(10**(decimal_places)),2))
                    allowance_or_charge_list[5] = total_amount
                else:
                    final_section = text[2][text[2].index(unit_basis_for_measurement_code)+len(unit_basis_for_measurement_code):]
                    decimal_places = int(final_section[0])
                    if(re.search(total_amount_pattern,final_section)):
                        total_amount = re.search(total_amount_pattern,final_section).group()[2:]
                        total_amount = int(total_amount)
                        total_amount = str(round((total_amount)/(10**(decimal_places)),2))
                        allowance_or_charge_list[5] = total_amount
        elif(len(text[2])==10):
            decimal_places = float(text[2][0])
            if(re.search(total_amount_pattern,text[2])):
                total_amount = re.search(total_amount_pattern,text[2]).group()[2:]
                total_amount = float(total_amount)
                total_amount = round((total_amount)/(10**(decimal_places)),2)
                if(total_amount>10):
                    allowance_or_charge_list[5] = str(total_amount)
                else:
                    allowance_or_charge_list[2] = str(total_amount)
    for word in text:
        if(re.search(not_rate_pattern,word)):
            allowance_or_charge_list[2] = '0'
    for item in range(len(allowance_or_charge_list)):
        allowance_or_charge_list[item] = str(allowance_or_charge_list[item])
    return allowance_or_charge_list

#### G73 --> Allowance/Charge Description

In [ ]:
def allowance_or_charge_description(text):
    text = str(text)
    return [text.replace('G7301500000','').replace('G7302400000','')]

### Invoice Detail

#### G17 --> Item Detail Invoice

In [ ]:
def item_detail_invoice(text):
    item_detail_invoice_list = ["NULL"]*12
    original = text
    text = str(text)
    text = text.replace('-','')
    text = text.replace('G17016000000000000000','').replace('G1701600000000000000','').replace('G170160000000000000','').replace('G17016000000000000','')\
                .replace('G1701600000000000','').replace('G170160000000000','').replace('G17016000000000','').replace('G1701600000000','')
    prechecks = text
    text = str(text)
    if(not(text[0]=="G")):
        prechecks = text
        decimal_places = 0
        if(text[:7].isnumeric()):
            if((not(text[7].isnumeric()))and(text[8].isnumeric())):
                if(re.search(r"[1-58]{1}[A-RX]{1}",text[6:8])):
                    quantity_invoiced = text[:6]
                    quantity_invoiced = int(quantity_invoiced)
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
                    unit_basis_for_measurement_code = text[6:8]
                    for code,item in g17_unit_or_basis_for_measurement_codes.items():
                        if(code==unit_basis_for_measurement_code):
                            item_detail_invoice_list[1] = item
                            text = text[8:]
            else:
                quantity_invoiced = text[:7]
                quantity_invoiced = int(quantity_invoiced)
                quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                item_detail_invoice_list[0] = quantity_invoiced
                unit_basis_for_measurement_code = text[7:9]
                for code,item in g17_unit_or_basis_for_measurement_codes.items():
                    if(code==unit_basis_for_measurement_code):
                        item_detail_invoice_list[1] = item
                        text = text[9:]
        elif(text[:6].isnumeric()):
            if((not(text[6].isnumeric()))and(text[7].isnumeric())):
                if(re.search(r"[1-58]{1}[A-RX]{1}",text[5:7])):
                    quantity_invoiced = text[:5]
                    quantity_invoiced = int(quantity_invoiced)
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
                    unit_basis_for_measurement_code = text[5:7]
                    for code,item in g17_unit_or_basis_for_measurement_codes.items():
                        if(code==unit_basis_for_measurement_code):
                            item_detail_invoice_list[1] = item
                            text = text[7:]
            else:
                quantity_invoiced = text[:6]
                quantity_invoiced = int(quantity_invoiced)
                quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                item_detail_invoice_list[0] = quantity_invoiced
                unit_basis_for_measurement_code = text[6:8]
                for code,item in g17_unit_or_basis_for_measurement_codes.items():
                    if(code==unit_basis_for_measurement_code):
                        item_detail_invoice_list[1] = item
                        text = text[8:]
        elif(text[:5].isnumeric()):
            if((not(text[5].isnumeric()))and(text[6].isnumeric())):
                if(re.search(r"[1-58]{1}[A-RX]{1}",text[4:6])):
                    quantity_invoiced = text[:4]
                    quantity_invoiced = int(quantity_invoiced)
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
                    unit_basis_for_measurement_code = text[4:6]
                    for code,item in g17_unit_or_basis_for_measurement_codes.items():
                        if(code==unit_basis_for_measurement_code):
                            item_detail_invoice_list[1] = item
                            text = text[6:]
            else:
                quantity_invoiced = text[:5]
                quantity_invoiced = int(quantity_invoiced)
                quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                item_detail_invoice_list[0] = quantity_invoiced
                text = text[5:]
                unit_basis_for_measurement_code = text[:2]
                for code,item in g17_unit_or_basis_for_measurement_codes.items():
                    if(code==unit_basis_for_measurement_code):
                        item_detail_invoice_list[1] = item
                        text = text[2:]
        elif(text[:4].isnumeric()):
            if((not(text[4].isnumeric()))and(text[5].isnumeric())):
                if(re.search(r"[1-58]{1}[A-RX]{1}",text[3:5])):
                    quantity_invoiced = text[:3]
                    quantity_invoiced = int(quantity_invoiced)
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
                    unit_basis_for_measurement_code = text[3:5]
                    for code,item in g17_unit_or_basis_for_measurement_codes.items():
                        if(code==unit_basis_for_measurement_code):
                            item_detail_invoice_list[1] = item
                            text = text[5:]
            else:
                quantity_invoiced = text[:4]
                quantity_invoiced = int(quantity_invoiced)
                quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                item_detail_invoice_list[0] = quantity_invoiced
                unit_basis_for_measurement_code = text[4:6]
                for code,item in g17_unit_or_basis_for_measurement_codes.items():
                    if(code==unit_basis_for_measurement_code):
                        item_detail_invoice_list[1] = item
                        text = text[6:]
        elif(text[:3].isnumeric()):
            if((not(text[3].isnumeric()))and(text[4].isnumeric())):
                if(re.search(r"[1-58]{1}[A-RX]{1}",text[2:4])):
                    quantity_invoiced = text[:2]
                    quantity_invoiced = int(quantity_invoiced)
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
                    unit_basis_for_measurement_code = text[2:4]
                    for code,item in g17_unit_or_basis_for_measurement_codes.items():
                        if(code==unit_basis_for_measurement_code):
                            item_detail_invoice_list[1] = item
                            text = text[4:]
            else:
                quantity_invoiced = text[:3]
                quantity_invoiced = int(quantity_invoiced)
                quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                item_detail_invoice_list[0] = quantity_invoiced
                unit_basis_for_measurement_code = text[3:5]
                for code,item in g17_unit_or_basis_for_measurement_codes.items():
                    if(code==unit_basis_for_measurement_code):
                        item_detail_invoice_list[1] = item
                        text = text[5:]
        elif(text[:2].isnumeric()):
            if((not(text[2].isnumeric()))and(text[3].isnumeric())):
                if(re.search(r"[1-58]{1}[A-RX]{1}",text[1:3])):
                    quantity_invoiced = text[0]
                    quantity_invoiced = int(quantity_invoiced)
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
                    unit_basis_for_measurement_code = text[1:3]
                    for code,item in g17_unit_or_basis_for_measurement_codes.items():
                        if(code==unit_basis_for_measurement_code):
                            item_detail_invoice_list[1] = item
                            text = text[3:]
            else:
                quantity_invoiced = text[:2]
                quantity_invoiced = int(quantity_invoiced)
                quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                item_detail_invoice_list[0] = quantity_invoiced
                unit_basis_for_measurement_code = text[2:4]
                for code,item in g17_unit_or_basis_for_measurement_codes.items():
                    if(code==unit_basis_for_measurement_code):
                        item_detail_invoice_list[1] = item
                        text = text[4:]
        else:
            if((not(text[1].isnumeric()))and(text[2].isnumeric())):
                if(re.search(r"[1-58]{1}[A-RX]{1}",text[:2])):
                    quantity_invoiced = text[0]
                    quantity_invoiced = int(quantity_invoiced)
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    unit_basis_for_measurement_code = text[:2]
                    for code,item in g17_unit_or_basis_for_measurement_codes.items():
                        if(code==unit_basis_for_measurement_code):
                            item_detail_invoice_list[1] = item
                            text = text[2:]
            else:
                quantity_invoiced = text[0]
                quantity_invoiced = int(quantity_invoiced)
                quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                item_detail_invoice_list[0] = quantity_invoiced
                unit_basis_for_measurement_code = text[1:3]
                for code,item in g17_unit_or_basis_for_measurement_codes.items():
                    if(code==unit_basis_for_measurement_code):
                        item_detail_invoice_list[1] = item
                        text = text[3:]
    else:
        if('G17016000003' in text):
            text = str(text)
            decimal_places = 3
            text = text.replace('G17016000003000000000','').replace('G1701600000300000000','').replace('G170160000030000000','').replace('G17016000003000000','')\
            .replace('G1701600000300000','').replace('G170160000030000','').replace('G17016000003000','').replace('G1701600000300','').replace('G170160000030','')
            prechecks = text
            if(text[:7].isnumeric()):
                if((not(text[7].isnumeric()))and(text[8].isnumeric())):
                    if(re.search(r"[1-58]{1}[A-RX]{1}",text[6:8])):
                        quantity_invoiced = text[:6]
                        quantity_invoiced = int(quantity_invoiced)
                        quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                        item_detail_invoice_list[0] = quantity_invoiced
                        unit_basis_for_measurement_code = text[6:8]
                        for code,item in g17_unit_or_basis_for_measurement_codes.items():
                            if(code==unit_basis_for_measurement_code):
                                item_detail_invoice_list[1] = item
                                text = text[8:]
                else:
                    quantity_invoiced = text[:7]
                    quantity_invoiced = int(quantity_invoiced)
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
                    unit_basis_for_measurement_code = text[7:9]
                    for code,item in g17_unit_or_basis_for_measurement_codes.items():
                        if(code==unit_basis_for_measurement_code):
                            item_detail_invoice_list[1] = item
                            text = text[9:]
            elif(text[:6].isnumeric()):
                if((not(text[6].isnumeric()))and(text[7].isnumeric())):
                    if(re.search(r"[1-58]{1}[A-RX]{1}",text[5:7])):
                        quantity_invoiced = text[:5]
                        quantity_invoiced = int(quantity_invoiced)
                        quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                        item_detail_invoice_list[0] = quantity_invoiced
                        unit_basis_for_measurement_code = text[5:7]
                        for code,item in g17_unit_or_basis_for_measurement_codes.items():
                            if(code==unit_basis_for_measurement_code):
                                item_detail_invoice_list[1] = item
                                text = text[7:]
                else:
                    quantity_invoiced = text[:6]
                    quantity_invoiced = int(quantity_invoiced)
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
                    unit_basis_for_measurement_code = text[6:8]
                    for code,item in g17_unit_or_basis_for_measurement_codes.items():
                        if(code==unit_basis_for_measurement_code):
                            item_detail_invoice_list[1] = item
                            text = text[8:]
            elif(text[:5].isnumeric()):
                if((not(text[5].isnumeric()))and(text[6].isnumeric())):
                    if(re.search(r"[1-58]{1}[A-RX]{1}",text[4:6])):
                        quantity_invoiced = text[:4]
                        quantity_invoiced = int(quantity_invoiced)
                        quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                        item_detail_invoice_list[0] = quantity_invoiced
                        unit_basis_for_measurement_code = text[4:6]
                        for code,item in g17_unit_or_basis_for_measurement_codes.items():
                            if(code==unit_basis_for_measurement_code):
                                item_detail_invoice_list[1] = item
                                text = text[6:]
                else:
                    quantity_invoiced = text[:5]
                    quantity_invoiced = int(quantity_invoiced)
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
                    unit_basis_for_measurement_code = text[5:7]
                    for code,item in g17_unit_or_basis_for_measurement_codes.items():
                        if(code==unit_basis_for_measurement_code):
                            item_detail_invoice_list[1] = item
                            text = text[7:]
            elif(text[:4].isnumeric()):
                if((not(text[4].isnumeric()))and(text[5].isnumeric())):
                    if(re.search(r"[1-58]{1}[A-RX]{1}",text[3:5])):
                        quantity_invoiced = text[:3]
                        quantity_invoiced = int(quantity_invoiced)
                        quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                        item_detail_invoice_list[0] = quantity_invoiced
                        unit_basis_for_measurement_code = text[3:5]
                        for code,item in g17_unit_or_basis_for_measurement_codes.items():
                            if(code==unit_basis_for_measurement_code):
                                item_detail_invoice_list[1] = item
                                text = text[5:]
                else:
                    quantity_invoiced = text[:4]
                    quantity_invoiced = int(quantity_invoiced)
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
                    unit_basis_for_measurement_code = text[4:6]
                    for code,item in g17_unit_or_basis_for_measurement_codes.items():
                        if(code==unit_basis_for_measurement_code):
                            item_detail_invoice_list[1] = item
                            text = text[6:]
            elif(text[:3].isnumeric()):
                if((not(text[3].isnumeric()))and(text[4].isnumeric())):
                    if(re.search(r"[1-58]{1}[A-RX]{1}",text[2:4])):
                        quantity_invoiced = text[:2]
                        quantity_invoiced = int(quantity_invoiced)
                        quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                        item_detail_invoice_list[0] = quantity_invoiced
                        unit_basis_for_measurement_code = text[2:4]
                        for code,item in g17_unit_or_basis_for_measurement_codes.items():
                            if(code==unit_basis_for_measurement_code):
                                item_detail_invoice_list[1] = item
                                text = text[4:]
                else:
                    quantity_invoiced = text[:3]
                    quantity_invoiced = int(quantity_invoiced)
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
                    unit_basis_for_measurement_code = text[3:5]
                    for code,item in g17_unit_or_basis_for_measurement_codes.items():
                        if(code==unit_basis_for_measurement_code):
                            item_detail_invoice_list[1] = item
                            text = text[5:]
            elif(text[:2].isnumeric()):
                if((not(text[2].isnumeric()))and(text[3].isnumeric())):
                    if(re.search(r"[1-58]{1}[A-RX]{1}",text[1:3])):
                        quantity_invoiced = text[0]
                        quantity_invoiced = int(quantity_invoiced)
                        quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                        item_detail_invoice_list[0] = quantity_invoiced
                        unit_basis_for_measurement_code = text[1:3]
                        for code,item in g17_unit_or_basis_for_measurement_codes.items():
                            if(code==unit_basis_for_measurement_code):
                                item_detail_invoice_list[1] = item
                                text = text[3:]
                else:
                    quantity_invoiced = text[:2]
                    quantity_invoiced = int(quantity_invoiced)
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
                    unit_basis_for_measurement_code = text[2:4]
                    for code,item in g17_unit_or_basis_for_measurement_codes.items():
                        if(code==unit_basis_for_measurement_code):
                            item_detail_invoice_list[1] = item
                            text = text[4:]
            else:
                if((not(text[1].isnumeric()))and(text[2].isnumeric())):
                    if(re.search(r"[1-58]{1}[A-RX]{1}",text[:2])):
                        unit_basis_for_measurement_code = text[:2]
                        for code,item in g17_unit_or_basis_for_measurement_codes.items():
                            if(code==unit_basis_for_measurement_code):
                                item_detail_invoice_list[1] = item
                                text = text[2:]
                else:
                    quantity_invoiced = text[0]
                    quantity_invoiced = int(quantity_invoiced)
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
                    unit_basis_for_measurement_code = text[1:3]
                    for code,item in g17_unit_or_basis_for_measurement_codes.items():
                        if(code==unit_basis_for_measurement_code):
                            item_detail_invoice_list[1] = item
                            text = text[3:]
    
        if('G17016000002' in text):
            decimal_places = 2
            text = text.replace('G17016000002000000000','').replace('G1701600000200000000','').replace('G170160000020000000','').replace('G17016000002000000','')\
            .replace('G1701600000200000','').replace('G170160000020000','').replace('G17016000002000','').replace('G1701600000200','').replace('G170160000020','')
            prechecks = text
            if(text[:7].isnumeric()):
                if((not(text[7].isnumeric()))and(text[8].isnumeric())):
                    if(re.search(r"[1-58]{1}[A-RX]{1}",text[6:8])):
                        quantity_invoiced = text[:6]
                        quantity_invoiced = int(quantity_invoiced)
                        quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                        item_detail_invoice_list[0] = quantity_invoiced
                        unit_basis_for_measurement_code = text[6:8]
                        for code,item in g17_unit_or_basis_for_measurement_codes.items():
                            if(code==unit_basis_for_measurement_code):
                                item_detail_invoice_list[1] = item
                                text = text[8:]
                else:
                    quantity_invoiced = text[:7]
                    quantity_invoiced = int(quantity_invoiced)
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
                    unit_basis_for_measurement_code = text[7:9]
                    for code,item in g17_unit_or_basis_for_measurement_codes.items():
                        if(code==unit_basis_for_measurement_code):
                            item_detail_invoice_list[1] = item
                            text = text[9:]
            elif(text[:6].isnumeric()):
                if((not(text[6].isnumeric()))and(text[7].isnumeric())):
                    if(re.search(r"[1-58]{1}[A-RX]{1}",text[5:7])):
                        quantity_invoiced = text[:5]
                        quantity_invoiced = int(quantity_invoiced)
                        quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                        item_detail_invoice_list[0] = quantity_invoiced
                        unit_basis_for_measurement_code = text[5:7]
                        for code,item in g17_unit_or_basis_for_measurement_codes.items():
                            if(code==unit_basis_for_measurement_code):
                                item_detail_invoice_list[1] = item
                                text = text[7:]
                else:
                    quantity_invoiced = text[:6]
                    quantity_invoiced = int(quantity_invoiced)
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
                    unit_basis_for_measurement_code = text[6:8]
                    for code,item in g17_unit_or_basis_for_measurement_codes.items():
                        if(code==unit_basis_for_measurement_code):
                            item_detail_invoice_list[1] = item
                            text = text[8:]
            elif(text[:5].isnumeric()):
                if((not(text[5].isnumeric()))and(text[6].isnumeric())):
                    if(re.search(r"[1-58]{1}[A-RX]{1}",text[4:6])):
                        quantity_invoiced = text[:4]
                        quantity_invoiced = int(quantity_invoiced)
                        quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                        item_detail_invoice_list[0] = quantity_invoiced
                        unit_basis_for_measurement_code = text[4:6]
                        for code,item in g17_unit_or_basis_for_measurement_codes.items():
                            if(code==unit_basis_for_measurement_code):
                                item_detail_invoice_list[1] = item
                                text = text[6:]
                else:
                    quantity_invoiced = text[:5]
                    quantity_invoiced = int(quantity_invoiced)
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
                    unit_basis_for_measurement_code = text[5:7]
                    for code,item in g17_unit_or_basis_for_measurement_codes.items():
                        if(code==unit_basis_for_measurement_code):
                            item_detail_invoice_list[1] = item
                            text = text[7:]
            elif(text[:4].isnumeric()):
                if((not(text[4].isnumeric()))and(text[5].isnumeric())):
                    if(re.search(r"[1-58]{1}[A-RX]{1}",text[3:5])):
                        quantity_invoiced = text[:3]
                        quantity_invoiced = int(quantity_invoiced)
                        quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                        item_detail_invoice_list[0] = quantity_invoiced
                        unit_basis_for_measurement_code = text[3:5]
                        for code,item in g17_unit_or_basis_for_measurement_codes.items():
                            if(code==unit_basis_for_measurement_code):
                                item_detail_invoice_list[1] = item
                                text = text[5:]
                else:
                    quantity_invoiced = text[:4]
                    quantity_invoiced = int(quantity_invoiced)
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
                    unit_basis_for_measurement_code = text[4:6]
                    for code,item in g17_unit_or_basis_for_measurement_codes.items():
                        if(code==unit_basis_for_measurement_code):
                            item_detail_invoice_list[1] = item
                            text = text[6:]
            elif(text[:3].isnumeric()):
                if((not(text[3].isnumeric()))and(text[4].isnumeric())):
                    if(re.search(r"[1-58]{1}[A-RX]{1}",text[2:4])):
                        quantity_invoiced = text[:2]
                        quantity_invoiced = int(quantity_invoiced)
                        quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                        item_detail_invoice_list[0] = quantity_invoiced
                        unit_basis_for_measurement_code = text[2:4]
                        for code,item in g17_unit_or_basis_for_measurement_codes.items():
                            if(code==unit_basis_for_measurement_code):
                                item_detail_invoice_list[1] = item
                                text = text[4:]
                else:
                    quantity_invoiced = text[:3]
                    quantity_invoiced = int(quantity_invoiced)
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
                    unit_basis_for_measurement_code = text[3:5]
                    for code,item in g17_unit_or_basis_for_measurement_codes.items():
                        if(code==unit_basis_for_measurement_code):
                            item_detail_invoice_list[1] = item
                            text = text[5:]
            elif(text[:2].isnumeric()):
                if((not(text[2].isnumeric()))and(text[3].isnumeric())):
                    if(re.search(r"[1-58]{1}[A-RX]{1}",text[1:3])):
                        quantity_invoiced = text[0]
                        quantity_invoiced = int(quantity_invoiced)
                        quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                        item_detail_invoice_list[0] = quantity_invoiced
                        unit_basis_for_measurement_code = text[1:3]
                        for code,item in g17_unit_or_basis_for_measurement_codes.items():
                            if(code==unit_basis_for_measurement_code):
                                item_detail_invoice_list[1] = item
                                text = text[3:]
                else:
                    quantity_invoiced = text[:2]
                    quantity_invoiced = int(quantity_invoiced)
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
                    unit_basis_for_measurement_code = text[2:4]
                    for code,item in g17_unit_or_basis_for_measurement_codes.items():
                        if(code==unit_basis_for_measurement_code):
                            item_detail_invoice_list[1] = item
                            text = text[4:]
            else:
                if((not(text[1].isnumeric()))and(text[2].isnumeric())):
                    if(re.search(r"[1-58]{1}[A-RX]{1}",text[:2])):
                        unit_basis_for_measurement_code = text[:2]
                        for code,item in g17_unit_or_basis_for_measurement_codes.items():
                            if(code==unit_basis_for_measurement_code):
                                item_detail_invoice_list[1] = item
                                text = text[2:]
                else:
                    quantity_invoiced = text[0]
                    quantity_invoiced = int(quantity_invoiced)
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
                    unit_basis_for_measurement_code = text[1:3]
                    for code,item in g17_unit_or_basis_for_measurement_codes.items():
                        if(code==unit_basis_for_measurement_code):
                            item_detail_invoice_list[1] = item
                            text = text[3:]
    
        if('G17016000001' in text):
            decimal_places = 1
            text = text.replace('G17016000001000000000','').replace('G1701600000100000000','').replace('G170160000010000000','').replace('G17016000001000000','')\
            .replace('G1701600000100000','').replace('G170160000010000','').replace('G17016000001000','').replace('G1701600000100','').replace('G170160000010','')
            prechecks = text
            if(text[:7].isnumeric()):
                if((not(text[7].isnumeric()))and(text[8].isnumeric())):
                    if(re.search(r"[1-58]{1}[A-RX]{1}",text[6:8])):
                        quantity_invoiced = text[:6]
                        quantity_invoiced = int(quantity_invoiced)
                        quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                        item_detail_invoice_list[0] = quantity_invoiced
                        unit_basis_for_measurement_code = text[6:8]
                        for code,item in g17_unit_or_basis_for_measurement_codes.items():
                            if(code==unit_basis_for_measurement_code):
                                item_detail_invoice_list[1] = item
                                text = text[8:]
                else:
                    quantity_invoiced = text[:7]
                    quantity_invoiced = int(quantity_invoiced)
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
                    unit_basis_for_measurement_code = text[7:9]
                    for code,item in g17_unit_or_basis_for_measurement_codes.items():
                        if(code==unit_basis_for_measurement_code):
                            item_detail_invoice_list[1] = item
                            text = text[9:]
            elif(text[:6].isnumeric()):
                if((not(text[6].isnumeric()))and(text[7].isnumeric())):
                    if(re.search(r"[1-58]{1}[A-RX]{1}",text[5:7])):
                        quantity_invoiced = text[:5]
                        quantity_invoiced = int(quantity_invoiced)
                        quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                        item_detail_invoice_list[0] = quantity_invoiced
                        unit_basis_for_measurement_code = text[5:7]
                        for code,item in g17_unit_or_basis_for_measurement_codes.items():
                            if(code==unit_basis_for_measurement_code):
                                item_detail_invoice_list[1] = item
                                text = text[7:]
                else:
                    quantity_invoiced = text[:6]
                    quantity_invoiced = int(quantity_invoiced)
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
                    unit_basis_for_measurement_code = text[6:8]
                    for code,item in g17_unit_or_basis_for_measurement_codes.items():
                        if(code==unit_basis_for_measurement_code):
                            item_detail_invoice_list[1] = item
                            text = text[8:]
            elif(text[:5].isnumeric()):
                if((not(text[5].isnumeric()))and(text[6].isnumeric())):
                    if(re.search(r"[1-58]{1}[A-RX]{1}",text[4:6])):
                        quantity_invoiced = text[:4]
                        quantity_invoiced = int(quantity_invoiced)
                        quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                        item_detail_invoice_list[0] = quantity_invoiced
                        unit_basis_for_measurement_code = text[4:6]
                        for code,item in g17_unit_or_basis_for_measurement_codes.items():
                            if(code==unit_basis_for_measurement_code):
                                item_detail_invoice_list[1] = item
                                text = text[6:]
                else:
                    quantity_invoiced = text[:5]
                    quantity_invoiced = int(quantity_invoiced)
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
                    unit_basis_for_measurement_code = text[5:7]
                    for code,item in g17_unit_or_basis_for_measurement_codes.items():
                        if(code==unit_basis_for_measurement_code):
                            item_detail_invoice_list[1] = item
                            text = text[7:]
            elif(text[:4].isnumeric()):
                if((not(text[4].isnumeric()))and(text[5].isnumeric())):
                    if(re.search(r"[1-58]{1}[A-RX]{1}",text[3:5])):
                        quantity_invoiced = text[:3]
                        quantity_invoiced = int(quantity_invoiced)
                        quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                        item_detail_invoice_list[0] = quantity_invoiced
                        unit_basis_for_measurement_code = text[3:5]
                        for code,item in g17_unit_or_basis_for_measurement_codes.items():
                            if(code==unit_basis_for_measurement_code):
                                item_detail_invoice_list[1] = item
                                text = text[5:]
                else:
                    quantity_invoiced = text[:4]
                    quantity_invoiced = int(quantity_invoiced)
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
                    unit_basis_for_measurement_code = text[4:6]
                    for code,item in g17_unit_or_basis_for_measurement_codes.items():
                        if(code==unit_basis_for_measurement_code):
                            item_detail_invoice_list[1] = item
                            text = text[6:]
            elif(text[:3].isnumeric()):
                if((not(text[3].isnumeric()))and(text[4].isnumeric())):
                    if(re.search(r"[1-58]{1}[A-RX]{1}",text[2:4])):
                        quantity_invoiced = text[:2]
                        quantity_invoiced = int(quantity_invoiced)
                        quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                        item_detail_invoice_list[0] = quantity_invoiced
                        unit_basis_for_measurement_code = text[2:4]
                        for code,item in g17_unit_or_basis_for_measurement_codes.items():
                            if(code==unit_basis_for_measurement_code):
                                item_detail_invoice_list[1] = item
                                text = text[4:]
                else:
                    quantity_invoiced = text[:3]
                    quantity_invoiced = int(quantity_invoiced)
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
                    unit_basis_for_measurement_code = text[3:5]
                    for code,item in g17_unit_or_basis_for_measurement_codes.items():
                        if(code==unit_basis_for_measurement_code):
                            item_detail_invoice_list[1] = item
                            text = text[5:]
            elif(text[:2].isnumeric()):
                if((not(text[2].isnumeric()))and(text[3].isnumeric())):
                    if(re.search(r"[1-58]{1}[A-RX]{1}",text[1:3])):
                        quantity_invoiced = text[0]
                        quantity_invoiced = int(quantity_invoiced)
                        quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                        item_detail_invoice_list[0] = quantity_invoiced
                        unit_basis_for_measurement_code = text[1:3]
                        for code,item in g17_unit_or_basis_for_measurement_codes.items():
                            if(code==unit_basis_for_measurement_code):
                                item_detail_invoice_list[1] = item
                                text = text[3:]
                else:
                    quantity_invoiced = text[:2]
                    quantity_invoiced = int(quantity_invoiced)
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
                    unit_basis_for_measurement_code = text[2:4]
                    for code,item in g17_unit_or_basis_for_measurement_codes.items():
                        if(code==unit_basis_for_measurement_code):
                            item_detail_invoice_list[1] = item
                            text = text[4:]
            else:
                if((not(text[1].isnumeric()))and(text[2].isnumeric())):
                    if(re.search(r"[1-58]{1}[A-RX]{1}",text[:2])):
                        unit_basis_for_measurement_code = text[:2]
                        for code,item in g17_unit_or_basis_for_measurement_codes.items():
                            if(code==unit_basis_for_measurement_code):
                                item_detail_invoice_list[1] = item
                                text = text[2:]
                else:
                    quantity_invoiced = text[0]
                    quantity_invoiced = int(quantity_invoiced)
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
                    unit_basis_for_measurement_code = text[1:3]
                    for code,item in g17_unit_or_basis_for_measurement_codes.items():
                        if(code==unit_basis_for_measurement_code):
                            item_detail_invoice_list[1] = item
                            text = text[3:]
    
    if(len(text)==len(prechecks)):
        first_word = text.split(' ')[0]
        if(re.search(r"[A-Z]{2}",first_word)):
            unit_basis_for_measurement_code = re.search(r"[A-Z]{2}",first_word).group()
            index = str(text).index(unit_basis_for_measurement_code)
            if(not(text[0]=="G")):
                if("G17016000003" in original):
                    decimal_places = 3
                    quantity_invoiced = int(text[:index])
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
                elif("G17016000002" in original):
                    decimal_places = 2
                    quantity_invoiced = int(text[:index])
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
                elif("G17016000001" in original):
                    decimal_places = 1
                    quantity_invoiced = int(text[:index])
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
                else:
                    decimal_places = 0
                    quantity_invoiced = int(text[:index])
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
            for code,item in g17_unit_or_basis_for_measurement_codes.items():
                if(code==unit_basis_for_measurement_code):
                    item_detail_invoice_list[1] = item
                    text = text[index+len(unit_basis_for_measurement_code):]
        elif(re.search(r"[A-Z]{1}",first_word)):
            unit_basis_for_measurement_code = re.search(r"[A-Z]{1}",first_word).group()
            index = str(text).index(unit_basis_for_measurement_code)
            if(not(text[0]=="G")):
                if("G17016000003" in original):
                    decimal_places = 3
                    quantity_invoiced = int(text[:index])
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
                elif("G17016000002" in original):
                    decimal_places = 2
                    quantity_invoiced = int(text[:index])
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
                elif("G17016000001" in original):
                    decimal_places = 1
                    quantity_invoiced = int(text[:index])
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
                else:
                    decimal_places = 0
                    quantity_invoiced = int(text[:index])
                    quantity_invoiced = ((quantity_invoiced)/(10**decimal_places))
                    item_detail_invoice_list[0] = quantity_invoiced
            for code,item in g17_unit_or_basis_for_measurement_codes.items():
                if(code==unit_basis_for_measurement_code):
                    item_detail_invoice_list[1] = item
                    text = text[index+len(unit_basis_for_measurement_code):]
    
    text = prechecks
    text = str(text).split(' ') 
    if(re.search(r"[A-Z]",text[0][-1]) and len(re.findall(r"[A-Z]",text[0]))==1):
        unit_basis_for_measurement_code = text[0][-2:]
        for code,item in g17_unit_or_basis_for_measurement_codes.items():
            if(code==unit_basis_for_measurement_code):
                item_detail_invoice_list[1] = item
        item_detail_invoice_list[0] = text[0][:-2]
        other_codes = text[2]
        for code,item in g17_product_service_id_qualifier_codes.items():
            if(code==other_codes[:2]):
                item_detail_invoice_list[4] = item
        for code,item in g17_product_service_id_qualifier_codes.items():
            if(code==other_codes[2:4]):
                item_detail_invoice_list[6] = item
        for code,item in g17_unit_or_basis_for_measurement_codes.items():
            if(code==other_codes[4:]):
                item_detail_invoice_list[9] = item
        final_word = text[-1]
        decimal_places = final_word[0]
        decimal_places = int(decimal_places)
        monetary_amount = final_word[1:]
        monetary_amount = int(monetary_amount)
        monetary_amount = monetary_amount/(10**decimal_places)
        item_detail_invoice_list[-2] = str(monetary_amount)
        return item_detail_invoice_list
    elif(re.search(r"[A-Z]",text[0][-1]) and len(re.findall(r"[A-Z]{1,2}",text[0]))==2):
        first_id_qualifier = text[0][-2:]
        for code,item in g17_product_service_id_qualifier_codes.items():
            if(code==first_id_qualifier):
                item_detail_invoice_list[4] = item
        item_detail_invoice_list[3] = text[0][-14:-2]
        if(text[0][:4].isnumeric()):
            item_list_cost = text[0][6:]
            decimal_places = item_list_cost[0]
            decimal_places = int(decimal_places)
            item_list_cost = item_list_cost[1:]
            while(item_list_cost[0]=='0'):
                item_list_cost = item_list_cost[1:]
            item_list_cost = item_list_cost[:-14]
            item_list_cost = int(item_list_cost)
            item_list_cost = item_list_cost/(10**decimal_places)
            item_list_cost = str(item_list_cost)
            item_detail_invoice_list[2] = item_list_cost
        elif(text[0][:3].isnumeric()):
            item_list_cost = text[0][5:]
            decimal_places = item_list_cost[0]
            decimal_places = int(decimal_places)
            item_list_cost = item_list_cost[1:]
            while(item_list_cost[0]=='0'):
                item_list_cost = item_list_cost[1:]
            item_list_cost = item_list_cost[:-14]
            item_list_cost = int(item_list_cost)
            item_list_cost = item_list_cost/(10**decimal_places)
            item_list_cost = str(item_list_cost)
            item_detail_invoice_list[2] = item_list_cost
        elif(text[0][:2].isnumeric()):
            item_list_cost = text[0][4:]
            decimal_places = item_list_cost[0]
            decimal_places = int(decimal_places)
            item_list_cost = item_list_cost[1:]
            while(item_list_cost[0]=='0'):
                item_list_cost = item_list_cost[1:]
            item_list_cost = item_list_cost[:-14]
            item_list_cost = int(item_list_cost)
            item_list_cost = item_list_cost/(10**decimal_places)
            item_list_cost = str(item_list_cost)
            item_detail_invoice_list[2] = item_list_cost
        else:
            item_list_cost = text[0][3:]
            decimal_places = item_list_cost[0]
            decimal_places = int(decimal_places)
            item_list_cost = item_list_cost[1:]
            while(item_list_cost[0]=='0'):
                item_list_cost = item_list_cost[1:]
            item_list_cost = item_list_cost[:-14]
            item_list_cost = int(item_list_cost)
            item_list_cost = item_list_cost/(10**decimal_places)
            item_list_cost = str(item_list_cost)
            item_detail_invoice_list[2] = item_list_cost

        if(len(text)==4):
            first_id_qualifier = text[1][:2]
            for code,item in g17_product_service_id_qualifier_codes.items():
                if(code==first_id_qualifier):
                    item_detail_invoice_list[6] = item
            item_detail_invoice_list[7] = text[1][2:]
            first_id_qualifier = text[2][-2:]
            for code,item in g17_unit_or_basis_for_measurement_codes.items():
                if(code==first_id_qualifier):
                    item_detail_invoice_list[9] = item
            num_units_shipped = text[2][:-2]
            while(len(num_units_shipped)>0 and num_units_shipped[0]=='0'):
                num_units_shipped = num_units_shipped[1:]
            if(len(num_units_shipped)>0):
                item_detail_invoice_list[8] = num_units_shipped
            if(text[-1].isnumeric()):
                monetary_amount = text[-1]
                decimal_places = monetary_amount[0]
                decimal_places = int(decimal_places)
                monetary_amount = monetary_amount[1:]
                monetary_amount = int(monetary_amount)
                monetary_amount = monetary_amount/(10**decimal_places)
                item_detail_invoice_list[10] = monetary_amount
            elif(len(text[-1])==2):
                first_id_qualifier = text[-1]
                for code,item in g17_unit_or_basis_for_measurement_codes.items():
                    if(code==first_id_qualifier):
                        item_detail_invoice_list[9] = item
            return item_detail_invoice_list


    if(re.search(r"^[0-9]{1,30}[A-Z]{1,2}",text[0])):
        first_word = str(text[0])
        first_word = first_word.replace(re.search(r"^[0-9]{1,10}[A-Z]{1,2}",first_word).group(),"")
        if(len(first_word)>0):
            decimal_places = first_word[0]
            decimal_places = int(decimal_places)
            if(re.search(r"^[0-9]{1}[0]{1,25}",first_word)):
                item_list_cost = first_word.replace(re.search(r"^[0-9]{1}[0]{1,25}",first_word).group(),"")
            else:
                item_list_cost = first_word
            if(len(item_list_cost)<10 and len(item_list_cost)>0 and not(re.search(r"[A-Z]{1,2}",item_list_cost))):
                item_list_cost = int(item_list_cost)
                item_list_cost = ((item_list_cost)/(10**decimal_places))
                item_detail_invoice_list[2] = str(item_list_cost)
            elif(len(item_list_cost)>12 and re.search(r"[A-Z]{1,2}",item_list_cost)):
                if(re.search(r"^[0-9]{1,100}[A-Z]{2}",item_list_cost)):
                    end_index = item_list_cost.index(re.search(r"[A-Z]{2}",item_list_cost).group())
                    if(end_index>12):
                        start_index = end_index-12
                        item_detail_invoice_list[3] = item_list_cost[start_index:end_index]
                        item_cost = item_list_cost[:start_index]
                        item_cost = int(item_cost)
                        item_cost = ((item_cost)/(10**decimal_places))
                        item_detail_invoice_list[2] = str(item_cost)
                        first_id_qualifier = re.search(r"[A-Z]{2}",item_list_cost).group()
                        for code,item in g17_product_service_id_qualifier_codes.items():
                            if(code==first_id_qualifier):
                                item_detail_invoice_list[4] = item
                        if(end_index<len(item_list_cost)-1):
                            item_detail_invoice_list[5] = item_list_cost[end_index+2:]
                    else:
                        upc_code = item_list_cost[:end_index]
                        while(len(upc_code)<12):
                            upc_code = '0' + upc_code
                        item_detail_invoice_list[3] = upc_code
                        item_detail_invoice_list[2] = '0.00'
            elif(len(item_list_cost)>12 and len(item_list_cost)<22 and not(re.search(r"[A-Z]{1,2}",item_list_cost))):
                item_detail_invoice_list[3] = item_list_cost[-12:]
                item_list_cost = item_list_cost[:-12]
                item_list_cost = int(item_list_cost)
                item_list_cost = ((item_list_cost)/(10**decimal_places))
                item_detail_invoice_list[2] = str(item_list_cost)
            
            #print(item_detail_invoice_list)
            #return item_detail_invoice_list
            
    if(len(text)>1):
        if(re.search(r"[A-Z]",text[1][0]) and item_detail_invoice_list[4]=="NULL"):
            first_id_qualifier = text[1][:2]
            for code,item in g17_product_service_id_qualifier_codes.items():
                if(code==first_id_qualifier):
                    item_detail_invoice_list[4] = item
            if(len(text[1])>2 and len(text[1])<10):
                item_detail_invoice_list[5] = text[1][2:]
            elif(len(text[1])==2 and len(text)>2):
                if(re.search(r"[A-Z]",text[2][0]) and item_detail_invoice_list[6]=="NULL"):
                    first_id_qualifier = text[2][:2]
                    for code,item in g17_product_service_id_qualifier_codes.items():
                        if(code==first_id_qualifier):
                            item_detail_invoice_list[6] = item
                    if(len(text[2])>2 and len(text[2])<10):
                        item_detail_invoice_list[7] = text[1][2:]
                elif(re.search(r"^[0-9]{1,20}[A-Z]{1,2}[0-9]{1,20}",text[2])):
                    if(re.search(r"[A-Z]{2}",text[2])):
                        second_qualifier_code = re.search(r"[A-Z]{2}",text[2]).group()
                        item_detail_invoice_list[5] = text[2][:text[2].index(second_qualifier_code)]
                        for code,item in g17_product_service_id_qualifier_codes.items():
                            if(code==second_qualifier_code):
                                item_detail_invoice_list[6] = item 
                        if(re.search(r"[0]{4,20}",text[2][text[2].index(second_qualifier_code):])):
                            item_detail_invoice_list[7] = text[2][text[2].index(second_qualifier_code)+len(second_qualifier_code):text[2].index(re.search(r"[0]{4,20}",text[2][text[2].index(second_qualifier_code):]))]
                        else:
                            item_detail_invoice_list[7] = text[2][text[2].index(second_qualifier_code)+len(second_qualifier_code):]
        elif(re.search(r"[A-Z]",text[1][0]) and item_detail_invoice_list[6]=="NULL"):
            first_id_qualifier = text[1][:2]
            for code,item in g17_product_service_id_qualifier_codes.items():
                if(code==first_id_qualifier):
                    item_detail_invoice_list[6] = item
            if(len(text[1])>2 and len(text[1])<10):
                item_detail_invoice_list[7] = text[1][2:]

    text = prechecks
    text = str(text).split(' ') 
    if(len(text)>1):
        if(len(text[-1])==5 and text[-1][0]=='8'):
            item_detail_invoice_list[-1] = text[-1]
        if(len(text)>2):
            final_word = text[-1]
            second_to_last_word = text[-2]
            if(len(final_word)>=3 and final_word.isnumeric() and not(final_word[0]=='8')):
                decimal_places = final_word[0]
                decimal_places = int(decimal_places)
                monetary_amount = final_word[1:]
                monetary_amount = int(monetary_amount)
                monetary_amount = ((monetary_amount)/(10**decimal_places))
                item_detail_invoice_list[-2] = str(monetary_amount)
            if(len(second_to_last_word)>3):
                if(re.search(r"[A-Z]",second_to_last_word[-1]) and not(re.search(r"[A-Z]",second_to_last_word[:-3])) and second_to_last_word[0] in '012' and '000' in second_to_last_word[1:]):
                    unit_basis_for_measurement_code = second_to_last_word[-2:]
                    for code,item in g17_unit_or_basis_for_measurement_codes.items():
                        if(code==unit_basis_for_measurement_code):
                            item_detail_invoice_list[-3] = item
                    decimal_places = second_to_last_word[0]
                    decimal_places = int(decimal_places)
                    num_cases = second_to_last_word[1:-2]
                    while(len(num_cases)>0 and num_cases[0]=='0'):
                        num_cases = num_cases[1:]
                    if(len(num_cases)>0):
                        num_cases = int(num_cases)
                        num_cases = ((num_cases)/(10**decimal_places))
                        num_cases = str(num_cases)
                        item_detail_invoice_list[-4] = num_cases
    return item_detail_invoice_list

In [ ]:
count = 0
detail_lists = []
triples = 0
for invoice in invoices[:1]:
    count += 1
    if(count%1000==0):
        clear_output(wait=False)
        print(count)
    invoice = str(invoice)
    for line in invoice.split('\n'):
        if("G17" in line[:5]):
            detail_lists.append(item_detail_invoice(line))
#print(f"{triples:,}")

In [ ]:
df = pd.DataFrame(detail_lists,columns=['G17_358_Quantity_Invoiced','G17_355_Unit_Or_Basis_For_Measurement_Code','G17_237_Item_List_Cost',
                        'G17_438_UPC_Case_Code','G17_235_Product_Service_ID_Qualifier_1','G17_234_Product_Service_ID_1',
                        'G17_235_Product_Service_ID_Qualifier_2','G17_234_Product_Service_ID_2',
                        'G17_382_Number_Of_Units_Shipped','G17_355_Unit_Or_Basis_For_Measurement',
                        'G17_782_Monetary_Amount','Miscellaneous_Five_Digit_Number'])

In [ ]:
df.info()

In [ ]:
for col in df.columns:
    print(col+": "+str(len(df[~(df[col]=="NULL")].index)))

In [ ]:
df.head()

#### G69 --> Free Form Description

In [ ]:
def item_detail_description(text):
    text = str(text)
    description = text.replace('G6901700000','')
    return description

#### G20 --> Item Packaging Detail

In [187]:
def item_packaging_detail(text):
    text = str(text)
    while(text[-1]==' '):
        text = text[:-1]
    text = text.replace(' ','0').replace('G2001900000000000','').replace('G200190000000000','').replace('G20019000000000','').replace('G2001900000000','')
    return text

### Item Detail Allowances

## Extracting EDI Data

### Extracting EDI Header

In [ ]:
invoice_header_table = pd.DataFrame(data=[["NULL"]*len(all_header_columns)]*len(invoices),columns=all_header_columns)
g01_columns = []
n9_columns = []
g62_columns = []
nte_columns = []
g27_columns = []
g23_columns = []
g25_columns = []
n1_columns = []
n3_columns = []
n4_columns = []
g31_columns = []
g33_columns = []
for col in invoice_header_table.columns:
    if("G01" in col[:3]):
        g01_columns.append(col)
    elif("N9" in col[:3]):
        n9_columns.append(col)
    elif("G62" in col[:3]):
        g62_columns.append(col)
    elif("NTE" in col[:3]):
        nte_columns.append(col)
    elif("G27" in col[:3]):
        g27_columns.append(col)
    elif("G23" in col[:3]):
        g23_columns.append(col)
    elif("G25" in col[:3]):
        g25_columns.append(col)
    elif("N1" in col[:3]):
        n1_columns.append(col)
    elif("N3" in col[:3]):
        n3_columns.append(col)
    elif("N4" in col[:3]):
        n4_columns.append(col)
    elif("G31" in col[:3]):
        g31_columns.append(col)
    elif("G33" in col[:3]):
        g33_columns.append(col)
g01_data = []
n9_data = []
g62_data = []
nte_data = []
g27_data = []
g23_data = []
g25_data = []
n1_data = []
n3_data = []
n4_data = []
g31_data = []
g33_data = []
table_index = 0

In [ ]:
data = []
count = 0
for invoice in invoices[:1]:
    if(count>0 and count%1000==0):
        clear_output(wait=True)
        print(f"{count:,}")
    count += 1
    if(True):
        g01_data = []
        g01_finds = 0
        n9_data = []
        n9_finds = 0
        g62_data = []
        g62_finds = 0
        nte_data = []
        nte_finds = 0
        g27_data = []
        g27_finds = 0
        g23_data = []
        g23_finds = 0
        g25_data = []
        g25_finds = 0
        n1_data = []
        n1_finds = 0
        n3_data = []
        n3_finds = 0
        n4_data = []
        n4_finds = 0
        g31_data = []
        g31_finds = 0
        g33_data = []
        g33_finds = 0
    invoice = str(invoice)
    for line in invoice.split('\n'):
        if("G01" in line[:5]):
            g01_data = invoice_identification(line)
            for item in range(len(g01_data)):
                invoice_header_table.loc[table_index,g01_columns[item+(4*g01_finds)]] = g01_data[item]
            g01_finds += 1
        if("N9" in line[:5]):
            n9_data = reference_identification(line)
            for item in range(len(n9_data)):
                invoice_header_table.loc[table_index,n9_columns[item+(3*n9_finds)]] = n9_data[item]
            n9_finds += 1
        if("G62" in line[:5]):
            g62_data = date_time(line)
            for item in range(len(g62_data)):
                invoice_header_table.loc[table_index,g62_columns[item+(2*g62_finds)]] = g62_data[item]
            g62_finds += 1
        if("NTE" in line[:5]):
            nte_data = note_special_instruction(line)
            for item in range(len(nte_data)):
                invoice_header_table.loc[table_index,nte_columns[item+(2*nte_finds)]] = nte_data[item]
            nte_finds += 1
        if("G27" in line[:5]):
            g27_data = carrier_detail(line)
            for item in range(len(g27_data)):
                invoice_header_table.loc[table_index,g27_columns[item+3*(g27_finds)]] = g27_data[item]
            g27_finds += 1
        if("G23" in line[:5]):
            g23_data = terms_of_sale(line)
            for item in range(len(g23_data)):
                invoice_header_table.loc[table_index,g23_columns[item+(12*g23_finds)]] = g23_data[item]
            g23_finds += 1
        if("G25" in line[:5]):
            g25_data = fob_information(line)
            for item in range(len(g25_data)):
                invoice_header_table.loc[table_index,g25_columns[item+(2*g25_finds)]] = g25_data[item]
            g25_finds += 1
        if("N1" in line[:5]):
            n1_data = name(line)
            for item in range(len(n1_data)):
                invoice_header_table.loc[table_index,n1_columns[item+(4*n1_finds)]] = n1_data[item]
            n1_finds += 1
        if("N3" in line[:5]):
            n3_data = address_information(line)
            for item in range(len(n3_data)):
                invoice_header_table.loc[table_index,n3_columns[item+n3_finds]] = n3_data[item]
            n3_finds += 1 
        if("N4" in line[:5]):
            n4_data = geographic_location(line)
            for item in range(len(n4_data)):
                invoice_header_table.loc[table_index,n4_columns[item+(3*n4_finds)]] = n4_data[item]
            n4_finds += 1
        if("G31" in line[:5]):
            g31_data = total_invoice_quantity(line)
            for item in range(len(g31_data)):
                invoice_header_table.loc[table_index,g31_columns[item+(6*g31_finds)]] = g31_data[item]
            g31_finds += 1
        if("G33" in line[:5]):
            g33_data = total_dollars_summary(line)
            for item in range(len(g33_data)):
                invoice_header_table.loc[table_index,g33_columns[item+g33_finds]] = g33_data[item]
            g33_finds += 1
    table_index += 1

### Extracting EDI Header Allowance

In [ ]:
server = 'barney'
database = 'sandbox_mp'
connectionString = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Integrated Security={False};Autocommit={False};Trusted_Connection=yes;'
connection = pyodbc.connect(connectionString)
cursor_1 = connection.cursor()

In [ ]:
rows = []
g72_data = []
g73_data = []
count = 0
atg_ref_header = -1
for invoice in invoices:
    if(count>0 and count%1000==0):
        clear_output(wait=True)
        print(f"{count:,}")
    count += 1
    if(True):
        invoice_date = "NULL"
        invoice_number = "NULL"
        atg_ref_header = -1
        g72_data = []
        g73_data = ["NULL"]
        g72_line = -1
    invoice = str(invoice)
    invoice = invoice.split('\n')
    for line in invoice:
        if("G01" in line[:5]):
            invoice_data = invoice_identification(line)
            invoice_date = invoice_data[0]
            invoice_number = invoice_data[1]
            cursor_1.execute(
                            """SELECT [ATG_Ref]
                            FROM [sandbox_mp].[dbo].[ATG_EDI_Invoice_Hdr]
                            WHERE
                            (G01_245_Invoice_Date=?)
                            AND
                            (G01_076_Invoice_Number=?)""",
                            [invoice_date,invoice_number]
                        )
            try:
                atg_ref_header = cursor_1.fetchone()[0]
            except Exception as e:
                atg_ref_header = -1
            break
    for line in range(len(invoice)):
        if("G72" in invoice[line][:5]):
            g72_data = allowance_or_charge(invoice[line])
            if("G73" in invoice[line+1][:5]):
                g73_data = allowance_or_charge_description(invoice[line+1])
            if(atg_ref_header==-1):pass
            else:
                rows.append([atg_ref_header,g72_data[0],g72_data[1],g72_data[2],g72_data[3],g72_data[4],g72_data[5],g73_data[0]])

In [ ]:
invoice_header_allowance_table = pd.DataFrame(data=rows,columns=all_header_allowance_columns)

In [ ]:
for index in invoice_header_allowance_table.index:
    try:
        if(int(len(invoice_header_allowance_table)-index)%1000==0):
            clear_output(wait=False)
            print(f"{len(invoice_header_allowance_table)-index:,}") 
        parameters = {
            "@ATG_Hdr_Ref":invoice_header_allowance_table['ATG_Hdr_Ref'][index],
            "@G72_340_Allowance_Or_Charge_Code":invoice_header_allowance_table['G72_340_Allowance_Or_Charge_Code'][index],
            "@G72_331_Allowance_Or_Charge_Method_Of_Handling_Code":invoice_header_allowance_table['G72_331_Allowance_Or_Charge_Method_Of_Handling_Code'][index],
            "@G72_359_Allowance_Or_Charge_Rate":invoice_header_allowance_table['G72_359_Allowance_Or_Charge_Rate'][index],
            "@G72_339_Allowance_Or_Charge_Quantity":invoice_header_allowance_table['G72_339_Allowance_Or_Charge_Quantity'][index],
            "@G72_355_Unit_Or_Basis_For_Measurement_Code":invoice_header_allowance_table['G72_355_Unit_Or_Basis_For_Measurement_Code'][index],
            "@G72_360_Allowance_Or_Charge_Total_Amount":invoice_header_allowance_table['G72_360_Allowance_Or_Charge_Total_Amount'][index],
            "@G73_369_Free_Form_Description":invoice_header_allowance_table['G73_369_Free_Form_Description'][index]
                }
        
        cursor_1.execute(
                        "EXEC [barney].[sandbox_mp].[dbo].[ATG_EDI_Invoice_Hdr_Allowances_Upsert] ?,?,?,?,?,?,?,?",
                        [str(parameters[p]) for p in parameters]
                    )
        cursor_1.commit()
    except Exception as e:
        fail_count += 1
        print(e)
        print(f"Fails: {fail_count:,}")

### Extracting Invoice Detail

In [ ]:
server = 'barney'
database = 'sandbox_mp'
connectionString = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Integrated Security={False};Autocommit={False};Trusted_Connection=yes;'
connection = pyodbc.connect(connectionString)
cursor_1 = connection.cursor()

In [ ]:
invoice_detail_table = pd.DataFrame(data=None,columns=all_item_detail_invoice_columns)
invoice_detail_table = invoice_detail_table.astype(str)
invoice_detail_allowance_table = pd.DataFrame(data=None,columns=all_invoice_detail_allow_or_charge_columns)
invoice_detail_allowance_table = invoice_detail_allowance_table.astype(str)

In [ ]:
if(True):
    rows = []
    invoice_detail_rows = []
    invoice_detail_allowance_rows = []
    item_rows = []
    detail_invoices = []
    descriptions = []
    packaging_details_list = []
    allowance_list = []
    allowance_description_list = []
    count = 0
    atg_ref_header = -1
    fails = 0
    fail_count = 0
for invoice in invoices:
    if(count>0 and count%10000==0):
        #clear_output(wait=True)
        print(f"{count:,}")
    count += 1
    if(True):
        rows = []
        item_rows = []
        detail_invoices = []
        descriptions = []
        packaging_details_list = []
        allowance_list = []
        allowance_description_list = []
        atg_ref_header = -1
    invoice = str(invoice)
    invoice = invoice.split('\n')
    lines = invoice
    len(invoice_detail_table) = 0
    for line in invoice:
        if("G01" in line[:5]):
            invoice_data = invoice_identification(line)
            invoice_date = invoice_data[0]
            invoice_number = invoice_data[1]
            
            cursor_1.execute(
                            """SELECT [ATG_Ref]
                            FROM [sandbox_mp].[dbo].[ATG_EDI_Invoice_Hdr]
                            WHERE
                            (G01_076_Invoice_Number=?)
                            AND
                            (G01_245_Invoice_Date=?)""",
                            [invoice_number,invoice_date]
                        )
            try:
                atg_ref_header = cursor_1.fetchone()[0]
            except Exception as e:
                fails += 1
                atg_ref_header = -1
                #print(f"{e}\n{invoice_date}\n{invoice_number}\n{fails}\n")
    
    while(len(invoice_detail_table)<len(lines)):
        item_rows.clear()
        detail_invoices.clear()
        descriptions.clear()
        packaging_details_list.clear()
        if("G17" in lines[len(invoice_detail_table)][:5]):
            item_rows.append(lines[len(invoice_detail_table)])
            len(invoice_detail_table) += 1
            while((len(invoice_detail_table)<len(lines))and(not("G17" in lines[len(invoice_detail_table)][:5]))and(not("G31" in lines[len(invoice_detail_table)][:5]))and(not("G33" in lines[len(invoice_detail_table)][:5]))):
                item_rows.append(lines[len(invoice_detail_table)])
                len(invoice_detail_table) += 1
            final_row = [len(invoice_detail_rows),atg_ref_header]
            final_allowance_row = [atg_ref_header]
            detail_invoices.clear()
            descriptions.clear()
            packaging_details_list.clear()
            for item in item_rows:
                if("G17" in item[:5]):
                    detail_invoices = item_detail_invoice(item)
                    for item in detail_invoices:
                        final_row.append(item)
                elif("G69" in item[:5]):
                    descriptions.append(item_detail_description(item))
                elif("G20" in item[:5]):
                    packaging_details_list.append(item_packaging_detail(item))
                elif("G72" in item[:5]):
                    allowance_list = allowance_or_charge(item)
                    for item in allowance_list:
                        final_allowance_row.append(item)
                    
                elif("G73" in item[:5]):
                    allowance_description_list.append(allowance_or_charge_description(item))
            if(len(descriptions)>0):
                final_row.append(' '.join(descriptions))
            else:
                final_row.append("NULL")
            if(len(packaging_details_list)>0):
                final_row.append(' '.join(packaging_details_list))
            else:
                final_row.append("NULL")
            if(len(allowance_description_list)>0):
                final_allowance_row.append(' '.join(allowance_description_list))
            else:
                final_allowance_row.append("NULL")
            
            if(not(atg_ref_header==-1)):
                invoice_detail_table.iloc[len(invoice_detail_table)] = final_row
                invoice_detail_allowance_table.loc[len(invoice_detail_allowance_table)] = final_allowance_row
        try:
            parameters = {
                "@ATG_Hdr_Ref":invoice_detail_table['ATG_Hdr_Ref'][len(invoice_detail_table)-1],
                "@G17_358_Quantity_Invoiced":invoice_detail_table['G17_358_Quantity_Invoiced'][len(invoice_detail_table)-1],
                "@G17_355_Unit_Or_Basis_For_Measurement_Code":invoice_detail_table['G17_355_Unit_Or_Basis_For_Measurement_Code'][len(invoice_detail_table)-1],
                "@G17_237_Item_List_Cost":invoice_detail_table['G17_237_Item_List_Cost'][len(invoice_detail_table)-1],
                "@G17_438_UPC_Case_Code":invoice_detail_table['G17_438_UPC_Case_Code'][len(invoice_detail_table)-1],
                "@G17_235_Product_Service_ID_Qualifier_1":invoice_detail_table['G17_235_Product_Service_ID_Qualifier_1'][len(invoice_detail_table)-1],
                "@G17_234_Product_Service_ID_1":invoice_detail_table['G17_234_Product_Service_ID_1'][len(invoice_detail_table)-1],
                "@G17_235_Product_Service_ID_Qualifier_2":invoice_detail_table['G17_235_Product_Service_ID_Qualifier_2'][len(invoice_detail_table)-1],
                "@G17_234_Product_Service_ID_2":invoice_detail_table['G17_234_Product_Service_ID_2'][len(invoice_detail_table)-1],
                "@G17_382_Number_Of_Units_Shipped":invoice_detail_table['G17_382_Number_Of_Units_Shipped'][len(invoice_detail_table)-1],
                "@G17_355_Unit_Or_Basis_For_Measurement":invoice_detail_table['G17_355_Unit_Or_Basis_For_Measurement'][len(invoice_detail_table)-1],
                "@G17_782_Monetary_Amount":invoice_detail_table['G17_782_Monetary_Amount'][len(invoice_detail_table)-1],
                "@Miscellaneous_Five_Digit_Number":invoice_detail_table['Miscellaneous_Five_Digit_Number'][len(invoice_detail_table)-1],
                "@G69_369_Free_Form_Description":invoice_detail_table['G69_369_Free_Form_Description'][len(invoice_detail_table)-1],
                "@G20_356_Pack_Number_Inner_Pack_Units_Per_Outer_Pack_Unit":invoice_detail_table['G20_356_Pack_Number_Inner_Pack_Units_Per_Outer_Pack_Unit'][len(invoice_detail_table)-1]
                    }
            
            cursor_1.execute(
                            "EXEC [barney].[sandbox_mp].[dbo].[ATG_EDI_Invoice_Dtl_Upsert] ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?",
                            [str(parameters[p]) for p in parameters]
                        )
            atg_dtl_ref = cursor_1.fetchone()[0]
            cursor_1.commit()
            parameters = {
                '@ATG_Hdr_Ref':invoice_detail_allowance_table['ATG_Hdr_Ref'],
                '@ATG_Dtl_Ref':atg_dtl_ref,
                '@G72_340_Allowance_Or_Charge_Code':invoice_detail_allowance_table['G72_340_Allowance_Or_Charge_Code'],
                '@G72_331_Allowance_Or_Charge_Method_Of_Handling_Code':invoice_detail_allowance_table['G72_331_Allowance_Or_Charge_Method_Of_Handling_Code'],
                '@G72_359_Allowance_Or_Charge_Rate':invoice_detail_allowance_table['G72_359_Allowance_Or_Charge_Rate'],
                '@G72_339_Allowance_Charge_Quantity':invoice_detail_allowance_table['G72_339_Allowance_Charge_Quantity'],
                '@G72_355_Unit_Basis_For_Measurement':invoice_detail_allowance_table['G72_355_Unit_Basis_For_Measurement'],
                '@G72_360_Allowance_Or_Charge_Total_Amount':invoice_detail_allowance_table['G72_360_Allowance_Or_Charge_Total_Amount'],
                '@G73_369_Free_Form_Description':invoice_detail_allowance_table['G73_369_Free_Form_Description']
            }
            cursor_1.execute(
                            "EXEC [barney].[sandbox_mp].[dbo].[ATG_EDI_Invoice_Dtl_Allowances_Upsert] ?,?,?,?,?,?,?,?,?",
                            [str(parameters[p]) for p in parameters]
                        )
            atg_dtl_ref = cursor_1.fetchone()[0]
            cursor_1.commit()
        except Exception as e:
            fail_count += 1
            print(e)
            print(f"Fails: {fail_count:,}")
    
        else:
            len(invoice_detail_table) += 1

In [ ]:
invoice_detail_table = pd.DataFrame(data=invoice_detail_rows,columns=all_item_detail_invoice_columns)
invoice_detail_table = invoice_detail_table.astype(str)

In [ ]:
invoice_detail_table.info()

In [ ]:
for row in invoice_detail_rows:
    cursor_1.execute(
                    """INSERT INTO [sandbox_mp].[dbo].[ATG_EDI_Invoice_Detail](
                    ATG_Ref,
                    ATG_Hdr_Ref,
                    G17_358_Quantity_Invoiced,
                    G17_355_Unit_Or_Basis_For_Measurement_Code,
                    G17_237_Item_List_Cost,
                    G17_438_UPC_Case_Code,
                    G17_235_Product_Service_ID_Qualifier_1,
                    G17_234_Product_Service_ID_1,
                    G17_235_Product_Service_ID_Qualifier_2,
                    G17_234_Product_Service_ID_2,
                    G17_382_Number_Of_Units_Shipped,
                    G17_355_Unit_Or_Basis_For_Measurement,
                    G17_782_Monetary_Amount,
                    Miscellaneous_Five_Digit_Number,
                    G69_369_Free_Form_Description,
                    G20_356_Pack_Number_Inner_Pack_Units_Per_Outer_Pack_Unit
                    )
                    VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)""",
                    row
                    )
    cursor_1.commit()

In [ ]:
invoice_detail_table.info()

In [ ]:
invoice_detail_table.head()

In [ ]:
cursor_1.close()

In [ ]:
fail_count = 0
for index in invoice_detail_table.index:
    if(index==1000000):
        try:
            if(int(len(invoice_detail_table)-index)%1000==0):
                clear_output(wait=False)
                print(f"{len(invoice_detail_table)-index:,}") 
            parameters = {
                "@ATG_Hdr_Ref":invoice_detail_table['ATG_Hdr_Ref'][index],
                "@G17_358_Quantity_Invoiced":invoice_detail_table['G17_358_Quantity_Invoiced'][index],
                "@G17_355_Unit_Or_Basis_For_Measurement_Code":invoice_detail_table['G17_355_Unit_Or_Basis_For_Measurement_Code'][index],
                "@G17_237_Item_List_Cost":invoice_detail_table['G17_237_Item_List_Cost'][index],
                "@G17_438_UPC_Case_Code":invoice_detail_table['G17_438_UPC_Case_Code'][index],
                "@G17_235_Product_Service_ID_Qualifier_1":invoice_detail_table['G17_235_Product_Service_ID_Qualifier_1'][index],
                "@G17_234_Product_Service_ID_1":invoice_detail_table['G17_234_Product_Service_ID_1'][index],
                "@G17_235_Product_Service_ID_Qualifier_2":invoice_detail_table['G17_235_Product_Service_ID_Qualifier_2'][index],
                "@G17_234_Product_Service_ID_2":invoice_detail_table['G17_234_Product_Service_ID_2'][index],
                "@G17_382_Number_Of_Units_Shipped":invoice_detail_table['G17_382_Number_Of_Units_Shipped'][index],
                "@G17_355_Unit_Or_Basis_For_Measurement":invoice_detail_table['G17_355_Unit_Or_Basis_For_Measurement'][index],
                "@G17_782_Monetary_Amount":invoice_detail_table['G17_782_Monetary_Amount'][index],
                "@Miscellaneous_Five_Digit_Number":invoice_detail_table['Miscellaneous_Five_Digit_Number'][index],
                "@G69_369_Free_Form_Description":invoice_detail_table['G69_369_Free_Form_Description'][index],
                "@G20_356_Pack_Number_Inner_Pack_Units_Per_Outer_Pack_Unit":invoice_detail_table['G20_356_Pack_Number_Inner_Pack_Units_Per_Outer_Pack_Unit'][index]
                    }
            
            cursor_1.execute(
                            "EXEC [barney].[sandbox_mp].[dbo].[ATG_EDI_Invoice_Dtl_Upsert] ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?",
                            [str(parameters[p]) for p in parameters]
                        )
            print(cursor_1.fetchone()[0])
            cursor_1.commit()
            
        except Exception as e:
            fail_count += 1
            print(e)
            print(f"Fails: {fail_count:,}")

## Stored Procedures

In [ ]:
for col in invoice_header_table.columns:
    if("Date" in col):
        print(invoice_header_table[[col]].iloc[617:619])

In [ ]:
for col in invoice_header_table.columns:
    print(type(invoice_header_table[col][0]))

In [ ]:
# HEADER TABLE STORED PROCEDURE
count = 1
fails = []
server = 'barney'
database = 'sandbox_mp'
connectionString = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Integrated Security={False};Autocommit={False};Trusted_Connection=yes;'
connection = pyodbc.connect(connectionString)
cursor_1 = connection.cursor()
for index in invoice_header_table.index:
    try:
        if(int(len(invoice_header_table)-index)%5000==0):
            print(f"{len(invoice_header_table)-index:,}")
            
        parameters = {
            "@G01_245_Invoice_Date":invoice_header_table['G01_245_Invoice_Date'][index],
            "@G01_076_Invoice_Number":invoice_header_table['G01_076_Invoice_Number'][index],
            "@G01_323_Purchase_Order_Date":invoice_header_table['G01_323_Purchase_Order_Date'][index],
            "@G01_324_Purchase_Order_Number":invoice_header_table['G01_324_Purchase_Order_Number'][index],
            "@N9_128_Reference_Number_Qualifier_Code_1":invoice_header_table['N9_128_Reference_Number_Qualifier_Code_1'][index],
            "@N9_127_Reference_Number_1":invoice_header_table['N9_127_Reference_Number_1'][index],
            "@N9_369_Free_Form_Description_1":invoice_header_table['N9_369_Free_Form_Description_1'][index],
            "@N9_128_Reference_Number_Qualifier_Code_2":invoice_header_table['N9_128_Reference_Number_Qualifier_Code_2'][index],
            "@N9_127_Reference_Number_2":invoice_header_table['N9_127_Reference_Number_2'][index],
            "@N9_369_Free_Form_Description_2":invoice_header_table['N9_369_Free_Form_Description_2'][index],
            "@N9_128_Reference_Number_Qualifier_Code_3":invoice_header_table['N9_128_Reference_Number_Qualifier_Code_3'][index],
            "@N9_127_Reference_Number_3":invoice_header_table['N9_127_Reference_Number_3'][index],
            "@N9_369_Free_Form_Description_3":invoice_header_table['N9_369_Free_Form_Description_3'][index],
            "@N9_128_Reference_Number_Qualifier_Code_4":invoice_header_table['N9_128_Reference_Number_Qualifier_Code_4'][index],
            "@N9_127_Reference_Number_4":invoice_header_table['N9_127_Reference_Number_4'][index],
            "@N9_369_Free_Form_Description_4":invoice_header_table['N9_369_Free_Form_Description_4'][index],
            "@N9_128_Reference_Number_Qualifier_Code_5":invoice_header_table['N9_128_Reference_Number_Qualifier_Code_5'][index],
            "@N9_127_Reference_Number_5":invoice_header_table['N9_127_Reference_Number_5'][index],
            "@N9_369_Free_Form_Description_5":invoice_header_table['N9_369_Free_Form_Description_5'][index],
            "@N9_128_Reference_Number_Qualifier_Code_6":invoice_header_table['N9_128_Reference_Number_Qualifier_Code_6'][index],
            "@N9_127_Reference_Number_6":invoice_header_table['N9_127_Reference_Number_6'][index],
            "@N9_369_Free_Form_Description_6":invoice_header_table['N9_369_Free_Form_Description_6'][index],
            "@N9_128_Reference_Number_Qualifier_Code_7":invoice_header_table['N9_128_Reference_Number_Qualifier_Code_7'][index],
            "@N9_127_Reference_Number_7":invoice_header_table['N9_127_Reference_Number_7'][index],
            "@N9_369_Free_Form_Description_7":invoice_header_table['N9_369_Free_Form_Description_7'][index],
            "@N9_128_Reference_Number_Qualifier_Code_8":invoice_header_table['N9_128_Reference_Number_Qualifier_Code_8'][index],
            "@N9_127_Reference_Number_8":invoice_header_table['N9_127_Reference_Number_8'][index],
            "@N9_369_Free_Form_Description_8":invoice_header_table['N9_369_Free_Form_Description_8'][index],
            "@N9_128_Reference_Number_Qualifier_Code_9":invoice_header_table['N9_128_Reference_Number_Qualifier_Code_9'][index],
            "@N9_127_Reference_Number_9":invoice_header_table['N9_127_Reference_Number_9'][index],
            "@N9_369_Free_Form_Description_9":invoice_header_table['N9_369_Free_Form_Description_9'][index],
            "@N9_128_Reference_Number_Qualifier_Code_10":invoice_header_table['N9_128_Reference_Number_Qualifier_Code_10'][index],
            "@N9_127_Reference_Number_10":invoice_header_table['N9_127_Reference_Number_10'][index],
            "@N9_369_Free_Form_Description_10":invoice_header_table['N9_369_Free_Form_Description_10'][index],
            "@G62_432_Date_Qualifier_1":invoice_header_table['G62_432_Date_Qualifier_1'][index],
            "@G62_373_Date_1":invoice_header_table['G62_373_Date_1'][index],
            "@G62_432_Date_Qualifier_2":invoice_header_table['G62_432_Date_Qualifier_2'][index],
            "@G62_373_Date_2":invoice_header_table['G62_373_Date_2'][index],
            "@G62_432_Date_Qualifier_3":invoice_header_table['G62_432_Date_Qualifier_3'][index],
            "@G62_373_Date_3":invoice_header_table['G62_373_Date_3'][index],
            "@G62_432_Date_Qualifier_4":invoice_header_table['G62_432_Date_Qualifier_4'][index],
            "@G62_373_Date_4":invoice_header_table['G62_373_Date_4'][index],
            "@G62_432_Date_Qualifier_5":invoice_header_table['G62_432_Date_Qualifier_5'][index],
            "@G62_373_Date_5":invoice_header_table['G62_373_Date_5'][index],
            "@NTE_363_Note_Reference_Code_1":invoice_header_table['NTE_363_Note_Reference_Code_1'][index],
            "@NTE_003_Free_Form_Message_1":invoice_header_table['NTE_003_Free_Form_Message_1'][index],
            "@NTE_363_Note_Reference_Code_2":invoice_header_table['NTE_363_Note_Reference_Code_2'][index],
            "@NTE_003_Free_Form_Message_2":invoice_header_table['NTE_003_Free_Form_Message_2'][index],
            "@NTE_363_Note_Reference_Code_3":invoice_header_table['NTE_363_Note_Reference_Code_3'][index],
            "@NTE_003_Free_Form_Message_3":invoice_header_table['NTE_003_Free_Form_Message_3'][index],
            "@NTE_363_Note_Reference_Code_4":invoice_header_table['NTE_363_Note_Reference_Code_4'][index],
            "@NTE_003_Free_Form_Message_4":invoice_header_table['NTE_003_Free_Form_Message_4'][index],
            "@NTE_363_Note_Reference_Code_5":invoice_header_table['NTE_363_Note_Reference_Code_5'][index],
            "@NTE_003_Free_Form_Message_5":invoice_header_table['NTE_003_Free_Form_Message_5'][index],
            "@NTE_363_Note_Reference_Code_6":invoice_header_table['NTE_363_Note_Reference_Code_6'][index],
            "@NTE_003_Free_Form_Message_6":invoice_header_table['NTE_003_Free_Form_Message_6'][index],
            "@NTE_363_Note_Reference_Code_7":invoice_header_table['NTE_363_Note_Reference_Code_7'][index],
            "@NTE_003_Free_Form_Message_7":invoice_header_table['NTE_003_Free_Form_Message_7'][index],
            "@NTE_363_Note_Reference_Code_8":invoice_header_table['NTE_363_Note_Reference_Code_8'][index],
            "@NTE_003_Free_Form_Message_8":invoice_header_table['NTE_003_Free_Form_Message_8'][index],
            "@NTE_363_Note_Reference_Code_9":invoice_header_table['NTE_363_Note_Reference_Code_9'][index],
            "@NTE_003_Free_Form_Message_9":invoice_header_table['NTE_003_Free_Form_Message_9'][index],
            "@NTE_363_Note_Reference_Code_10":invoice_header_table['NTE_363_Note_Reference_Code_10'][index],
            "@NTE_003_Free_Form_Message_10":invoice_header_table['NTE_003_Free_Form_Message_10'][index],
            "@NTE_363_Note_Reference_Code_11":invoice_header_table['NTE_363_Note_Reference_Code_11'][index],
            "@NTE_003_Free_Form_Message_11":invoice_header_table['NTE_003_Free_Form_Message_11'][index],
            "@NTE_363_Note_Reference_Code_12":invoice_header_table['NTE_363_Note_Reference_Code_12'][index],
            "@NTE_003_Free_Form_Message_12":invoice_header_table['NTE_003_Free_Form_Message_12'][index],
            "@NTE_363_Note_Reference_Code_13":invoice_header_table['NTE_363_Note_Reference_Code_13'][index],
            "@NTE_003_Free_Form_Message_13":invoice_header_table['NTE_003_Free_Form_Message_13'][index],
            "@NTE_363_Note_Reference_Code_14":invoice_header_table['NTE_363_Note_Reference_Code_14'][index],
            "@NTE_003_Free_Form_Message_14":invoice_header_table['NTE_003_Free_Form_Message_14'][index],
            "@NTE_363_Note_Reference_Code_15":invoice_header_table['NTE_363_Note_Reference_Code_15'][index],
            "@NTE_003_Free_Form_Message_15":invoice_header_table['NTE_003_Free_Form_Message_15'][index],
            "@NTE_363_Note_Reference_Code_16":invoice_header_table['NTE_363_Note_Reference_Code_16'][index],
            "@NTE_003_Free_Form_Message_16":invoice_header_table['NTE_003_Free_Form_Message_16'][index],
            "@NTE_363_Note_Reference_Code_17":invoice_header_table['NTE_363_Note_Reference_Code_17'][index],
            "@NTE_003_Free_Form_Message_17":invoice_header_table['NTE_003_Free_Form_Message_17'][index],
            "@NTE_363_Note_Reference_Code_18":invoice_header_table['NTE_363_Note_Reference_Code_18'][index],
            "@NTE_003_Free_Form_Message_18":invoice_header_table['NTE_003_Free_Form_Message_18'][index],
            "@NTE_363_Note_Reference_Code_19":invoice_header_table['NTE_363_Note_Reference_Code_19'][index],
            "@NTE_003_Free_Form_Message_19":invoice_header_table['NTE_003_Free_Form_Message_19'][index],
            "@NTE_363_Note_Reference_Code_20":invoice_header_table['NTE_363_Note_Reference_Code_20'][index],
            "@NTE_003_Free_Form_Message_20":invoice_header_table['NTE_003_Free_Form_Message_20'][index],
            "@G27_091_Transportation_Method_Type_Code_1":invoice_header_table['G27_091_Transportation_Method_Type_Code_1'][index],
            "@G27_207_Equipment_Number_1":invoice_header_table['G27_207_Equipment_Number_1'][index],
            "@G27_387_Routing_1":invoice_header_table['G27_387_Routing_1'][index],
            "@G27_091_Transportation_Method_Type_Code_2":invoice_header_table['G27_091_Transportation_Method_Type_Code_2'][index],
            "@G27_207_Equipment_Number_2":invoice_header_table['G27_207_Equipment_Number_2'][index],
            "@G27_387_Routing_2":invoice_header_table['G27_387_Routing_2'][index],
            "@G27_091_Transportation_Method_Type_Code_3":invoice_header_table['G27_091_Transportation_Method_Type_Code_3'][index],
            "@G27_207_Equipment_Number_3":invoice_header_table['G27_207_Equipment_Number_3'][index],
            "@G27_387_Routing_3":invoice_header_table['G27_387_Routing_3'][index],
            "@G27_091_Transportation_Method_Type_Code_4":invoice_header_table['G27_091_Transportation_Method_Type_Code_4'][index],
            "@G27_207_Equipment_Number_4":invoice_header_table['G27_207_Equipment_Number_4'][index],
            "@G27_387_Routing_4":invoice_header_table['G27_387_Routing_4'][index],
            "@G27_091_Transportation_Method_Type_Code_5":invoice_header_table['G27_091_Transportation_Method_Type_Code_5'][index],
            "@G27_207_Equipment_Number_5":invoice_header_table['G27_207_Equipment_Number_5'][index],
            "@G27_387_Routing_5":invoice_header_table['G27_387_Routing_5'][index],
            "@G23_336_Terms_Type_Code_1":invoice_header_table['G23_336_Terms_Type_Code_1'][index],
            "@G23_333_Terms_Basis_Date_Code_1":invoice_header_table['G23_333_Terms_Basis_Date_Code_1'][index],
            "@G23_282_Terms_Start_Date_1":invoice_header_table['G23_282_Terms_Start_Date_1'][index],
            "@G23_283_Terms_Due_Date_Qualifier_1":invoice_header_table['G23_283_Terms_Due_Date_Qualifier_1'][index],
            "@G23_338_Terms_Discount_Percent_1":invoice_header_table['G23_338_Terms_Discount_Percent_1'][index],
            "@G23_370_Terms_Discount_Due_Date_1":invoice_header_table['G23_370_Terms_Discount_Due_Date_1'][index],
            "@G23_351_Terms_Discount_Days_Due_1":invoice_header_table['G23_351_Terms_Discount_Days_Due_1'][index],
            "@G23_446_Terms_Net_Due_Date_1":invoice_header_table['G23_446_Terms_Net_Due_Date_1'][index],
            "@G23_386_Terms_Net_Days_1":invoice_header_table['G23_386_Terms_Net_Days_1'][index],
            "@G23_362_Terms_Discount_Amount_1":invoice_header_table['G23_362_Terms_Discount_Amount_1'][index],
            "@G23_391_Discounted_Amount_1":invoice_header_table['G23_391_Discounted_Amount_1'][index],
            "@G23_390_Amount_Subject_To_Terms_Discount_1":invoice_header_table['G23_390_Amount_Subject_To_Terms_Discount_1'][index],
            "@G23_003_Free_Form_Message_1":invoice_header_table['G23_003_Free_Form_Message_1'][index],
            "@G23_336_Terms_Type_Code_2":invoice_header_table['G23_336_Terms_Type_Code_2'][index],
            "@G23_333_Terms_Basis_Date_Code_2":invoice_header_table['G23_333_Terms_Basis_Date_Code_2'][index],
            "@G23_282_Terms_Start_Date_2":invoice_header_table['G23_282_Terms_Start_Date_2'][index],
            "@G23_283_Terms_Due_Date_Qualifier_2":invoice_header_table['G23_283_Terms_Due_Date_Qualifier_2'][index],
            "@G23_338_Terms_Discount_Percent_2":invoice_header_table['G23_338_Terms_Discount_Percent_2'][index],
            "@G23_370_Terms_Discount_Due_Date_2":invoice_header_table['G23_370_Terms_Discount_Due_Date_2'][index],
            "@G23_351_Terms_Discount_Days_Due_2":invoice_header_table['G23_351_Terms_Discount_Days_Due_2'][index],
            "@G23_446_Terms_Net_Due_Date_2":invoice_header_table['G23_446_Terms_Net_Due_Date_2'][index],
            "@G23_386_Terms_Net_Days_2":invoice_header_table['G23_386_Terms_Net_Days_2'][index],
            "@G23_362_Terms_Discount_Amount_2":invoice_header_table['G23_362_Terms_Discount_Amount_2'][index],
            "@G23_391_Discounted_Amount_2":invoice_header_table['G23_391_Discounted_Amount_2'][index],
            "@G23_390_Amount_Subject_To_Terms_Discount_2":invoice_header_table['G23_390_Amount_Subject_To_Terms_Discount_2'][index],
            "@G23_003_Free_Form_Message_2":invoice_header_table['G23_003_Free_Form_Message_2'][index],
            "@G23_336_Terms_Type_Code_3":invoice_header_table['G23_336_Terms_Type_Code_3'][index],
            "@G23_333_Terms_Basis_Date_Code_3":invoice_header_table['G23_333_Terms_Basis_Date_Code_3'][index],
            "@G23_282_Terms_Start_Date_3":invoice_header_table['G23_282_Terms_Start_Date_3'][index],
            "@G23_283_Terms_Due_Date_Qualifier_3":invoice_header_table['G23_283_Terms_Due_Date_Qualifier_3'][index],
            "@G23_338_Terms_Discount_Percent_3":invoice_header_table['G23_338_Terms_Discount_Percent_3'][index],
            "@G23_370_Terms_Discount_Due_Date_3":invoice_header_table['G23_370_Terms_Discount_Due_Date_3'][index],
            "@G23_351_Terms_Discount_Days_Due_3":invoice_header_table['G23_351_Terms_Discount_Days_Due_3'][index],
            "@G23_446_Terms_Net_Due_Date_3":invoice_header_table['G23_446_Terms_Net_Due_Date_3'][index],
            "@G23_386_Terms_Net_Days_3":invoice_header_table['G23_386_Terms_Net_Days_3'][index],
            "@G23_362_Terms_Discount_Amount_3":invoice_header_table['G23_362_Terms_Discount_Amount_3'][index],
            "@G23_391_Discounted_Amount_3":invoice_header_table['G23_391_Discounted_Amount_3'][index],
            "@G23_390_Amount_Subject_To_Terms_Discount_3":invoice_header_table['G23_390_Amount_Subject_To_Terms_Discount_3'][index],
            "@G23_003_Free_Form_Message_3":invoice_header_table['G23_003_Free_Form_Message_3'][index],
            "@G23_336_Terms_Type_Code_4":invoice_header_table['G23_336_Terms_Type_Code_4'][index],
            "@G23_333_Terms_Basis_Date_Code_4":invoice_header_table['G23_333_Terms_Basis_Date_Code_4'][index],
            "@G23_282_Terms_Start_Date_4":invoice_header_table['G23_282_Terms_Start_Date_4'][index],
            "@G23_283_Terms_Due_Date_Qualifier_4":invoice_header_table['G23_283_Terms_Due_Date_Qualifier_4'][index],
            "@G23_338_Terms_Discount_Percent_4":invoice_header_table['G23_338_Terms_Discount_Percent_4'][index],
            "@G23_370_Terms_Discount_Due_Date_4":invoice_header_table['G23_370_Terms_Discount_Due_Date_4'][index],
            "@G23_351_Terms_Discount_Days_Due_4":invoice_header_table['G23_351_Terms_Discount_Days_Due_4'][index],
            "@G23_446_Terms_Net_Due_Date_4":invoice_header_table['G23_446_Terms_Net_Due_Date_4'][index],
            "@G23_386_Terms_Net_Days_4":invoice_header_table['G23_386_Terms_Net_Days_4'][index],
            "@G23_362_Terms_Discount_Amount_4":invoice_header_table['G23_362_Terms_Discount_Amount_4'][index],
            "@G23_391_Discounted_Amount_4":invoice_header_table['G23_391_Discounted_Amount_4'][index],
            "@G23_390_Amount_Subject_To_Terms_Discount_4":invoice_header_table['G23_390_Amount_Subject_To_Terms_Discount_4'][index],
            "@G23_003_Free_Form_Message_4":invoice_header_table['G23_003_Free_Form_Message_4'][index],
            "@G23_336_Terms_Type_Code_5":invoice_header_table['G23_336_Terms_Type_Code_5'][index],
            "@G23_333_Terms_Basis_Date_Code_5":invoice_header_table['G23_333_Terms_Basis_Date_Code_5'][index],
            "@G23_282_Terms_Start_Date_5":invoice_header_table['G23_282_Terms_Start_Date_5'][index],
            "@G23_283_Terms_Due_Date_Qualifier_5":invoice_header_table['G23_283_Terms_Due_Date_Qualifier_5'][index],
            "@G23_338_Terms_Discount_Percent_5":invoice_header_table['G23_338_Terms_Discount_Percent_5'][index],
            "@G23_370_Terms_Discount_Due_Date_5":invoice_header_table['G23_370_Terms_Discount_Due_Date_5'][index],
            "@G23_351_Terms_Discount_Days_Due_5":invoice_header_table['G23_351_Terms_Discount_Days_Due_5'][index],
            "@G23_446_Terms_Net_Due_Date_5":invoice_header_table['G23_446_Terms_Net_Due_Date_5'][index],
            "@G23_386_Terms_Net_Days_5":invoice_header_table['G23_386_Terms_Net_Days_5'][index],
            "@G23_362_Terms_Discount_Amount_5":invoice_header_table['G23_362_Terms_Discount_Amount_5'][index],
            "@G23_391_Discounted_Amount_5":invoice_header_table['G23_391_Discounted_Amount_5'][index],
            "@G23_390_Amount_Subject_To_Terms_Discount_5":invoice_header_table['G23_390_Amount_Subject_To_Terms_Discount_5'][index],
            "@G23_003_Free_Form_Message_5":invoice_header_table['G23_003_Free_Form_Message_5'][index],
            "@G23_336_Terms_Type_Code_6":invoice_header_table['G23_336_Terms_Type_Code_6'][index],
            "@G23_333_Terms_Basis_Date_Code_6":invoice_header_table['G23_333_Terms_Basis_Date_Code_6'][index],
            "@G23_282_Terms_Start_Date_6":invoice_header_table['G23_282_Terms_Start_Date_6'][index],
            "@G23_283_Terms_Due_Date_Qualifier_6":invoice_header_table['G23_283_Terms_Due_Date_Qualifier_6'][index],
            "@G23_338_Terms_Discount_Percent_6":invoice_header_table['G23_338_Terms_Discount_Percent_6'][index],
            "@G23_370_Terms_Discount_Due_Date_6":invoice_header_table['G23_370_Terms_Discount_Due_Date_6'][index],
            "@G23_351_Terms_Discount_Days_Due_6":invoice_header_table['G23_351_Terms_Discount_Days_Due_6'][index],
            "@G23_446_Terms_Net_Due_Date_6":invoice_header_table['G23_446_Terms_Net_Due_Date_6'][index],
            "@G23_386_Terms_Net_Days_6":invoice_header_table['G23_386_Terms_Net_Days_6'][index],
            "@G23_362_Terms_Discount_Amount_6":invoice_header_table['G23_362_Terms_Discount_Amount_6'][index],
            "@G23_391_Discounted_Amount_6":invoice_header_table['G23_391_Discounted_Amount_6'][index],
            "@G23_390_Amount_Subject_To_Terms_Discount_6":invoice_header_table['G23_390_Amount_Subject_To_Terms_Discount_6'][index],
            "@G23_003_Free_Form_Message_6":invoice_header_table['G23_003_Free_Form_Message_6'][index],
            "@G23_336_Terms_Type_Code_7":invoice_header_table['G23_336_Terms_Type_Code_7'][index],
            "@G23_333_Terms_Basis_Date_Code_7":invoice_header_table['G23_333_Terms_Basis_Date_Code_7'][index],
            "@G23_282_Terms_Start_Date_7":invoice_header_table['G23_282_Terms_Start_Date_7'][index],
            "@G23_283_Terms_Due_Date_Qualifier_7":invoice_header_table['G23_283_Terms_Due_Date_Qualifier_7'][index],
            "@G23_338_Terms_Discount_Percent_7":invoice_header_table['G23_338_Terms_Discount_Percent_7'][index],
            "@G23_370_Terms_Discount_Due_Date_7":invoice_header_table['G23_370_Terms_Discount_Due_Date_7'][index],
            "@G23_351_Terms_Discount_Days_Due_7":invoice_header_table['G23_351_Terms_Discount_Days_Due_7'][index],
            "@G23_446_Terms_Net_Due_Date_7":invoice_header_table['G23_446_Terms_Net_Due_Date_7'][index],
            "@G23_386_Terms_Net_Days_7":invoice_header_table['G23_386_Terms_Net_Days_7'][index],
            "@G23_362_Terms_Discount_Amount_7":invoice_header_table['G23_362_Terms_Discount_Amount_7'][index],
            "@G23_391_Discounted_Amount_7":invoice_header_table['G23_391_Discounted_Amount_7'][index],
            "@G23_390_Amount_Subject_To_Terms_Discount_7":invoice_header_table['G23_390_Amount_Subject_To_Terms_Discount_7'][index],
            "@G23_003_Free_Form_Message_7":invoice_header_table['G23_003_Free_Form_Message_7'][index],
            "@G23_336_Terms_Type_Code_8":invoice_header_table['G23_336_Terms_Type_Code_8'][index],
            "@G23_333_Terms_Basis_Date_Code_8":invoice_header_table['G23_333_Terms_Basis_Date_Code_8'][index],
            "@G23_282_Terms_Start_Date_8":invoice_header_table['G23_282_Terms_Start_Date_8'][index],
            "@G23_283_Terms_Due_Date_Qualifier_8":invoice_header_table['G23_283_Terms_Due_Date_Qualifier_8'][index],
            "@G23_338_Terms_Discount_Percent_8":invoice_header_table['G23_338_Terms_Discount_Percent_8'][index],
            "@G23_370_Terms_Discount_Due_Date_8":invoice_header_table['G23_370_Terms_Discount_Due_Date_8'][index],
            "@G23_351_Terms_Discount_Days_Due_8":invoice_header_table['G23_351_Terms_Discount_Days_Due_8'][index],
            "@G23_446_Terms_Net_Due_Date_8":invoice_header_table['G23_446_Terms_Net_Due_Date_8'][index],
            "@G23_386_Terms_Net_Days_8":invoice_header_table['G23_386_Terms_Net_Days_8'][index],
            "@G23_362_Terms_Discount_Amount_8":invoice_header_table['G23_362_Terms_Discount_Amount_8'][index],
            "@G23_391_Discounted_Amount_8":invoice_header_table['G23_391_Discounted_Amount_8'][index],
            "@G23_390_Amount_Subject_To_Terms_Discount_8":invoice_header_table['G23_390_Amount_Subject_To_Terms_Discount_8'][index],
            "@G23_003_Free_Form_Message_8":invoice_header_table['G23_003_Free_Form_Message_8'][index],
            "@G23_336_Terms_Type_Code_9":invoice_header_table['G23_336_Terms_Type_Code_9'][index],
            "@G23_333_Terms_Basis_Date_Code_9":invoice_header_table['G23_333_Terms_Basis_Date_Code_9'][index],
            "@G23_282_Terms_Start_Date_9":invoice_header_table['G23_282_Terms_Start_Date_9'][index],
            "@G23_283_Terms_Due_Date_Qualifier_9":invoice_header_table['G23_283_Terms_Due_Date_Qualifier_9'][index],
            "@G23_338_Terms_Discount_Percent_9":invoice_header_table['G23_338_Terms_Discount_Percent_9'][index],
            "@G23_370_Terms_Discount_Due_Date_9":invoice_header_table['G23_370_Terms_Discount_Due_Date_9'][index],
            "@G23_351_Terms_Discount_Days_Due_9":invoice_header_table['G23_351_Terms_Discount_Days_Due_9'][index],
            "@G23_446_Terms_Net_Due_Date_9":invoice_header_table['G23_446_Terms_Net_Due_Date_9'][index],
            "@G23_386_Terms_Net_Days_9":invoice_header_table['G23_386_Terms_Net_Days_9'][index],
            "@G23_362_Terms_Discount_Amount_9":invoice_header_table['G23_362_Terms_Discount_Amount_9'][index],
            "@G23_391_Discounted_Amount_9":invoice_header_table['G23_391_Discounted_Amount_9'][index],
            "@G23_390_Amount_Subject_To_Terms_Discount_9":invoice_header_table['G23_390_Amount_Subject_To_Terms_Discount_9'][index],
            "@G23_003_Free_Form_Message_9":invoice_header_table['G23_003_Free_Form_Message_9'][index],
            "@G23_336_Terms_Type_Code_10":invoice_header_table['G23_336_Terms_Type_Code_10'][index],
            "@G23_333_Terms_Basis_Date_Code_10":invoice_header_table['G23_333_Terms_Basis_Date_Code_10'][index],
            "@G23_282_Terms_Start_Date_10":invoice_header_table['G23_282_Terms_Start_Date_10'][index],
            "@G23_283_Terms_Due_Date_Qualifier_10":invoice_header_table['G23_283_Terms_Due_Date_Qualifier_10'][index],
            "@G23_338_Terms_Discount_Percent_10":invoice_header_table['G23_338_Terms_Discount_Percent_10'][index],
            "@G23_370_Terms_Discount_Due_Date_10":invoice_header_table['G23_370_Terms_Discount_Due_Date_10'][index],
            "@G23_351_Terms_Discount_Days_Due_10":invoice_header_table['G23_351_Terms_Discount_Days_Due_10'][index],
            "@G23_446_Terms_Net_Due_Date_10":invoice_header_table['G23_446_Terms_Net_Due_Date_10'][index],
            "@G23_386_Terms_Net_Days_10":invoice_header_table['G23_386_Terms_Net_Days_10'][index],
            "@G23_362_Terms_Discount_Amount_10":invoice_header_table['G23_362_Terms_Discount_Amount_10'][index],
            "@G23_391_Discounted_Amount_10":invoice_header_table['G23_391_Discounted_Amount_10'][index],
            "@G23_390_Amount_Subject_To_Terms_Discount_10":invoice_header_table['G23_390_Amount_Subject_To_Terms_Discount_10'][index],
            "@G23_003_Free_Form_Message_10":invoice_header_table['G23_003_Free_Form_Message_10'][index],
            "@G23_336_Terms_Type_Code_11":invoice_header_table['G23_336_Terms_Type_Code_11'][index],
            "@G23_333_Terms_Basis_Date_Code_11":invoice_header_table['G23_333_Terms_Basis_Date_Code_11'][index],
            "@G23_282_Terms_Start_Date_11":invoice_header_table['G23_282_Terms_Start_Date_11'][index],
            "@G23_283_Terms_Due_Date_Qualifier_11":invoice_header_table['G23_283_Terms_Due_Date_Qualifier_11'][index],
            "@G23_338_Terms_Discount_Percent_11":invoice_header_table['G23_338_Terms_Discount_Percent_11'][index],
            "@G23_370_Terms_Discount_Due_Date_11":invoice_header_table['G23_370_Terms_Discount_Due_Date_11'][index],
            "@G23_351_Terms_Discount_Days_Due_11":invoice_header_table['G23_351_Terms_Discount_Days_Due_11'][index],
            "@G23_446_Terms_Net_Due_Date_11":invoice_header_table['G23_446_Terms_Net_Due_Date_11'][index],
            "@G23_386_Terms_Net_Days_11":invoice_header_table['G23_386_Terms_Net_Days_11'][index],
            "@G23_362_Terms_Discount_Amount_11":invoice_header_table['G23_362_Terms_Discount_Amount_11'][index],
            "@G23_391_Discounted_Amount_11":invoice_header_table['G23_391_Discounted_Amount_11'][index],
            "@G23_390_Amount_Subject_To_Terms_Discount_11":invoice_header_table['G23_390_Amount_Subject_To_Terms_Discount_11'][index],
            "@G23_003_Free_Form_Message_11":invoice_header_table['G23_003_Free_Form_Message_11'][index],
            "@G23_336_Terms_Type_Code_12":invoice_header_table['G23_336_Terms_Type_Code_12'][index],
            "@G23_333_Terms_Basis_Date_Code_12":invoice_header_table['G23_333_Terms_Basis_Date_Code_12'][index],
            "@G23_282_Terms_Start_Date_12":invoice_header_table['G23_282_Terms_Start_Date_12'][index],
            "@G23_283_Terms_Due_Date_Qualifier_12":invoice_header_table['G23_283_Terms_Due_Date_Qualifier_12'][index],
            "@G23_338_Terms_Discount_Percent_12":invoice_header_table['G23_338_Terms_Discount_Percent_12'][index],
            "@G23_370_Terms_Discount_Due_Date_12":invoice_header_table['G23_370_Terms_Discount_Due_Date_12'][index],
            "@G23_351_Terms_Discount_Days_Due_12":invoice_header_table['G23_351_Terms_Discount_Days_Due_12'][index],
            "@G23_446_Terms_Net_Due_Date_12":invoice_header_table['G23_446_Terms_Net_Due_Date_12'][index],
            "@G23_386_Terms_Net_Days_12":invoice_header_table['G23_386_Terms_Net_Days_12'][index],
            "@G23_362_Terms_Discount_Amount_12":invoice_header_table['G23_362_Terms_Discount_Amount_12'][index],
            "@G23_391_Discounted_Amount_12":invoice_header_table['G23_391_Discounted_Amount_12'][index],
            "@G23_390_Amount_Subject_To_Terms_Discount_12":invoice_header_table['G23_390_Amount_Subject_To_Terms_Discount_12'][index],
            "@G23_003_Free_Form_Message_12":invoice_header_table['G23_003_Free_Form_Message_12'][index],
            "@G23_336_Terms_Type_Code_13":invoice_header_table['G23_336_Terms_Type_Code_13'][index],
            "@G23_333_Terms_Basis_Date_Code_13":invoice_header_table['G23_333_Terms_Basis_Date_Code_13'][index],
            "@G23_282_Terms_Start_Date_13":invoice_header_table['G23_282_Terms_Start_Date_13'][index],
            "@G23_283_Terms_Due_Date_Qualifier_13":invoice_header_table['G23_283_Terms_Due_Date_Qualifier_13'][index],
            "@G23_338_Terms_Discount_Percent_13":invoice_header_table['G23_338_Terms_Discount_Percent_13'][index],
            "@G23_370_Terms_Discount_Due_Date_13":invoice_header_table['G23_370_Terms_Discount_Due_Date_13'][index],
            "@G23_351_Terms_Discount_Days_Due_13":invoice_header_table['G23_351_Terms_Discount_Days_Due_13'][index],
            "@G23_446_Terms_Net_Due_Date_13":invoice_header_table['G23_446_Terms_Net_Due_Date_13'][index],
            "@G23_386_Terms_Net_Days_13":invoice_header_table['G23_386_Terms_Net_Days_13'][index],
            "@G23_362_Terms_Discount_Amount_13":invoice_header_table['G23_362_Terms_Discount_Amount_13'][index],
            "@G23_391_Discounted_Amount_13":invoice_header_table['G23_391_Discounted_Amount_13'][index],
            "@G23_390_Amount_Subject_To_Terms_Discount_13":invoice_header_table['G23_390_Amount_Subject_To_Terms_Discount_13'][index],
            "@G23_003_Free_Form_Message_13":invoice_header_table['G23_003_Free_Form_Message_13'][index],
            "@G23_336_Terms_Type_Code_14":invoice_header_table['G23_336_Terms_Type_Code_14'][index],
            "@G23_333_Terms_Basis_Date_Code_14":invoice_header_table['G23_333_Terms_Basis_Date_Code_14'][index],
            "@G23_282_Terms_Start_Date_14":invoice_header_table['G23_282_Terms_Start_Date_14'][index],
            "@G23_283_Terms_Due_Date_Qualifier_14":invoice_header_table['G23_283_Terms_Due_Date_Qualifier_14'][index],
            "@G23_338_Terms_Discount_Percent_14":invoice_header_table['G23_338_Terms_Discount_Percent_14'][index],
            "@G23_370_Terms_Discount_Due_Date_14":invoice_header_table['G23_370_Terms_Discount_Due_Date_14'][index],
            "@G23_351_Terms_Discount_Days_Due_14":invoice_header_table['G23_351_Terms_Discount_Days_Due_14'][index],
            "@G23_446_Terms_Net_Due_Date_14":invoice_header_table['G23_446_Terms_Net_Due_Date_14'][index],
            "@G23_386_Terms_Net_Days_14":invoice_header_table['G23_386_Terms_Net_Days_14'][index],
            "@G23_362_Terms_Discount_Amount_14":invoice_header_table['G23_362_Terms_Discount_Amount_14'][index],
            "@G23_391_Discounted_Amount_14":invoice_header_table['G23_391_Discounted_Amount_14'][index],
            "@G23_390_Amount_Subject_To_Terms_Discount_14":invoice_header_table['G23_390_Amount_Subject_To_Terms_Discount_14'][index],
            "@G23_003_Free_Form_Message_14":invoice_header_table['G23_003_Free_Form_Message_14'][index],
            "@G23_336_Terms_Type_Code_15":invoice_header_table['G23_336_Terms_Type_Code_15'][index],
            "@G23_333_Terms_Basis_Date_Code_15":invoice_header_table['G23_333_Terms_Basis_Date_Code_15'][index],
            "@G23_282_Terms_Start_Date_15":invoice_header_table['G23_282_Terms_Start_Date_15'][index],
            "@G23_283_Terms_Due_Date_Qualifier_15":invoice_header_table['G23_283_Terms_Due_Date_Qualifier_15'][index],
            "@G23_338_Terms_Discount_Percent_15":invoice_header_table['G23_338_Terms_Discount_Percent_15'][index],
            "@G23_370_Terms_Discount_Due_Date_15":invoice_header_table['G23_370_Terms_Discount_Due_Date_15'][index],
            "@G23_351_Terms_Discount_Days_Due_15":invoice_header_table['G23_351_Terms_Discount_Days_Due_15'][index],
            "@G23_446_Terms_Net_Due_Date_15":invoice_header_table['G23_446_Terms_Net_Due_Date_15'][index],
            "@G23_386_Terms_Net_Days_15":invoice_header_table['G23_386_Terms_Net_Days_15'][index],
            "@G23_362_Terms_Discount_Amount_15":invoice_header_table['G23_362_Terms_Discount_Amount_15'][index],
            "@G23_391_Discounted_Amount_15":invoice_header_table['G23_391_Discounted_Amount_15'][index],
            "@G23_390_Amount_Subject_To_Terms_Discount_15":invoice_header_table['G23_390_Amount_Subject_To_Terms_Discount_15'][index],
            "@G23_003_Free_Form_Message_15":invoice_header_table['G23_003_Free_Form_Message_15'][index],
            "@G23_336_Terms_Type_Code_16":invoice_header_table['G23_336_Terms_Type_Code_16'][index],
            "@G23_333_Terms_Basis_Date_Code_16":invoice_header_table['G23_333_Terms_Basis_Date_Code_16'][index],
            "@G23_282_Terms_Start_Date_16":invoice_header_table['G23_282_Terms_Start_Date_16'][index],
            "@G23_283_Terms_Due_Date_Qualifier_16":invoice_header_table['G23_283_Terms_Due_Date_Qualifier_16'][index],
            "@G23_338_Terms_Discount_Percent_16":invoice_header_table['G23_338_Terms_Discount_Percent_16'][index],
            "@G23_370_Terms_Discount_Due_Date_16":invoice_header_table['G23_370_Terms_Discount_Due_Date_16'][index],
            "@G23_351_Terms_Discount_Days_Due_16":invoice_header_table['G23_351_Terms_Discount_Days_Due_16'][index],
            "@G23_446_Terms_Net_Due_Date_16":invoice_header_table['G23_446_Terms_Net_Due_Date_16'][index],
            "@G23_386_Terms_Net_Days_16":invoice_header_table['G23_386_Terms_Net_Days_16'][index],
            "@G23_362_Terms_Discount_Amount_16":invoice_header_table['G23_362_Terms_Discount_Amount_16'][index],
            "@G23_391_Discounted_Amount_16":invoice_header_table['G23_391_Discounted_Amount_16'][index],
            "@G23_390_Amount_Subject_To_Terms_Discount_16":invoice_header_table['G23_390_Amount_Subject_To_Terms_Discount_16'][index],
            "@G23_003_Free_Form_Message_16":invoice_header_table['G23_003_Free_Form_Message_16'][index],
            "@G23_336_Terms_Type_Code_17":invoice_header_table['G23_336_Terms_Type_Code_17'][index],
            "@G23_333_Terms_Basis_Date_Code_17":invoice_header_table['G23_333_Terms_Basis_Date_Code_17'][index],
            "@G23_282_Terms_Start_Date_17":invoice_header_table['G23_282_Terms_Start_Date_17'][index],
            "@G23_283_Terms_Due_Date_Qualifier_17":invoice_header_table['G23_283_Terms_Due_Date_Qualifier_17'][index],
            "@G23_338_Terms_Discount_Percent_17":invoice_header_table['G23_338_Terms_Discount_Percent_17'][index],
            "@G23_370_Terms_Discount_Due_Date_17":invoice_header_table['G23_370_Terms_Discount_Due_Date_17'][index],
            "@G23_351_Terms_Discount_Days_Due_17":invoice_header_table['G23_351_Terms_Discount_Days_Due_17'][index],
            "@G23_446_Terms_Net_Due_Date_17":invoice_header_table['G23_446_Terms_Net_Due_Date_17'][index],
            "@G23_386_Terms_Net_Days_17":invoice_header_table['G23_386_Terms_Net_Days_17'][index],
            "@G23_362_Terms_Discount_Amount_17":invoice_header_table['G23_362_Terms_Discount_Amount_17'][index],
            "@G23_391_Discounted_Amount_17":invoice_header_table['G23_391_Discounted_Amount_17'][index],
            "@G23_390_Amount_Subject_To_Terms_Discount_17":invoice_header_table['G23_390_Amount_Subject_To_Terms_Discount_17'][index],
            "@G23_003_Free_Form_Message_17":invoice_header_table['G23_003_Free_Form_Message_17'][index],
            "@G23_336_Terms_Type_Code_18":invoice_header_table['G23_336_Terms_Type_Code_18'][index],
            "@G23_333_Terms_Basis_Date_Code_18":invoice_header_table['G23_333_Terms_Basis_Date_Code_18'][index],
            "@G23_282_Terms_Start_Date_18":invoice_header_table['G23_282_Terms_Start_Date_18'][index],
            "@G23_283_Terms_Due_Date_Qualifier_18":invoice_header_table['G23_283_Terms_Due_Date_Qualifier_18'][index],
            "@G23_338_Terms_Discount_Percent_18":invoice_header_table['G23_338_Terms_Discount_Percent_18'][index],
            "@G23_370_Terms_Discount_Due_Date_18":invoice_header_table['G23_370_Terms_Discount_Due_Date_18'][index],
            "@G23_351_Terms_Discount_Days_Due_18":invoice_header_table['G23_351_Terms_Discount_Days_Due_18'][index],
            "@G23_446_Terms_Net_Due_Date_18":invoice_header_table['G23_446_Terms_Net_Due_Date_18'][index],
            "@G23_386_Terms_Net_Days_18":invoice_header_table['G23_386_Terms_Net_Days_18'][index],
            "@G23_362_Terms_Discount_Amount_18":invoice_header_table['G23_362_Terms_Discount_Amount_18'][index],
            "@G23_391_Discounted_Amount_18":invoice_header_table['G23_391_Discounted_Amount_18'][index],
            "@G23_390_Amount_Subject_To_Terms_Discount_18":invoice_header_table['G23_390_Amount_Subject_To_Terms_Discount_18'][index],
            "@G23_003_Free_Form_Message_18":invoice_header_table['G23_003_Free_Form_Message_18'][index],
            "@G23_336_Terms_Type_Code_19":invoice_header_table['G23_336_Terms_Type_Code_19'][index],
            "@G23_333_Terms_Basis_Date_Code_19":invoice_header_table['G23_333_Terms_Basis_Date_Code_19'][index],
            "@G23_282_Terms_Start_Date_19":invoice_header_table['G23_282_Terms_Start_Date_19'][index],
            "@G23_283_Terms_Due_Date_Qualifier_19":invoice_header_table['G23_283_Terms_Due_Date_Qualifier_19'][index],
            "@G23_338_Terms_Discount_Percent_19":invoice_header_table['G23_338_Terms_Discount_Percent_19'][index],
            "@G23_370_Terms_Discount_Due_Date_19":invoice_header_table['G23_370_Terms_Discount_Due_Date_19'][index],
            "@G23_351_Terms_Discount_Days_Due_19":invoice_header_table['G23_351_Terms_Discount_Days_Due_19'][index],
            "@G23_446_Terms_Net_Due_Date_19":invoice_header_table['G23_446_Terms_Net_Due_Date_19'][index],
            "@G23_386_Terms_Net_Days_19":invoice_header_table['G23_386_Terms_Net_Days_19'][index],
            "@G23_362_Terms_Discount_Amount_19":invoice_header_table['G23_362_Terms_Discount_Amount_19'][index],
            "@G23_391_Discounted_Amount_19":invoice_header_table['G23_391_Discounted_Amount_19'][index],
            "@G23_390_Amount_Subject_To_Terms_Discount_19":invoice_header_table['G23_390_Amount_Subject_To_Terms_Discount_19'][index],
            "@G23_003_Free_Form_Message_19":invoice_header_table['G23_003_Free_Form_Message_19'][index],
            "@G23_336_Terms_Type_Code_20":invoice_header_table['G23_336_Terms_Type_Code_20'][index],
            "@G23_333_Terms_Basis_Date_Code_20":invoice_header_table['G23_333_Terms_Basis_Date_Code_20'][index],
            "@G23_282_Terms_Start_Date_20":invoice_header_table['G23_282_Terms_Start_Date_20'][index],
            "@G23_283_Terms_Due_Date_Qualifier_20":invoice_header_table['G23_283_Terms_Due_Date_Qualifier_20'][index],
            "@G23_338_Terms_Discount_Percent_20":invoice_header_table['G23_338_Terms_Discount_Percent_20'][index],
            "@G23_370_Terms_Discount_Due_Date_20":invoice_header_table['G23_370_Terms_Discount_Due_Date_20'][index],
            "@G23_351_Terms_Discount_Days_Due_20":invoice_header_table['G23_351_Terms_Discount_Days_Due_20'][index],
            "@G23_446_Terms_Net_Due_Date_20":invoice_header_table['G23_446_Terms_Net_Due_Date_20'][index],
            "@G23_386_Terms_Net_Days_20":invoice_header_table['G23_386_Terms_Net_Days_20'][index],
            "@G23_362_Terms_Discount_Amount_20":invoice_header_table['G23_362_Terms_Discount_Amount_20'][index],
            "@G23_391_Discounted_Amount_20":invoice_header_table['G23_391_Discounted_Amount_20'][index],
            "@G23_390_Amount_Subject_To_Terms_Discount_20":invoice_header_table['G23_390_Amount_Subject_To_Terms_Discount_20'][index],
            "@G23_003_Free_Form_Message_20":invoice_header_table['G23_003_Free_Form_Message_20'][index],
            "@G25_146_Shipment_Method_Of_Payment":invoice_header_table['G25_146_Shipment_Method_Of_Payment'][index],
            "@G25_433_FOB_Point_Code":invoice_header_table['G25_433_FOB_Point_Code'][index],
            "@G25_434_FOB_Point":invoice_header_table['G25_434_FOB_Point'][index],
            "@N1_098_Entity_Identification_Code_1":invoice_header_table['N1_098_Entity_Identification_Code_1'][index],
            "@N1_093_Name_1":invoice_header_table['N1_093_Name_1'][index],
            "@N1_066_Identification_Code_Qualifier_1":invoice_header_table['N1_066_Identification_Code_Qualifier_1'][index],
            "@N1_067_Identification_Code_1":invoice_header_table['N1_067_Identification_Code_1'][index],
            "@N3_166_Address_Info_1":invoice_header_table['N3_166_Address_Info_1'][index],
            "@N4_019_City_Name_1":invoice_header_table['N4_019_City_Name_1'][index],
            "@N4_156_State_Province_Code_1":invoice_header_table['N4_156_State_Province_Code_1'][index],
            "@N4_116_Postal_Code_1":invoice_header_table['N4_116_Postal_Code_1'][index],
            "@N1_098_Entity_Identification_Code_2":invoice_header_table['N1_098_Entity_Identification_Code_2'][index],
            "@N1_093_Name_2":invoice_header_table['N1_093_Name_2'][index],
            "@N1_066_Identification_Code_Qualifier_2":invoice_header_table['N1_066_Identification_Code_Qualifier_2'][index],
            "@N1_067_Identification_Code_2":invoice_header_table['N1_067_Identification_Code_2'][index],
            "@N3_166_Address_Info_2":invoice_header_table['N3_166_Address_Info_2'][index],
            "@N4_019_City_Name_2":invoice_header_table['N4_019_City_Name_2'][index],
            "@N4_156_State_Province_Code_2":invoice_header_table['N4_156_State_Province_Code_2'][index],
            "@N4_116_Postal_Code_2":invoice_header_table['N4_116_Postal_Code_2'][index],
            "@N1_098_Entity_Identification_Code_3":invoice_header_table['N1_098_Entity_Identification_Code_3'][index],
            "@N1_093_Name_3":invoice_header_table['N1_093_Name_3'][index],
            "@N1_066_Identification_Code_Qualifier_3":invoice_header_table['N1_066_Identification_Code_Qualifier_3'][index],
            "@N1_067_Identification_Code_3":invoice_header_table['N1_067_Identification_Code_3'][index],
            "@N3_166_Address_Info_3":invoice_header_table['N3_166_Address_Info_3'][index],
            "@N4_019_City_Name_3":invoice_header_table['N4_019_City_Name_3'][index],
            "@N4_156_State_Province_Code_3":invoice_header_table['N4_156_State_Province_Code_3'][index],
            "@N4_116_Postal_Code_3":invoice_header_table['N4_116_Postal_Code_3'][index],
            "@N1_098_Entity_Identification_Code_4":invoice_header_table['N1_098_Entity_Identification_Code_4'][index],
            "@N1_093_Name_4":invoice_header_table['N1_093_Name_4'][index],
            "@N1_066_Identification_Code_Qualifier_4":invoice_header_table['N1_066_Identification_Code_Qualifier_4'][index],
            "@N1_067_Identification_Code_4":invoice_header_table['N1_067_Identification_Code_4'][index],
            "@N3_166_Address_Info_4":invoice_header_table['N3_166_Address_Info_4'][index],
            "@N4_019_City_Name_4":invoice_header_table['N4_019_City_Name_4'][index],
            "@N4_156_State_Province_Code_4":invoice_header_table['N4_156_State_Province_Code_4'][index],
            "@N4_116_Postal_Code_4":invoice_header_table['N4_116_Postal_Code_4'][index],
            "@N1_098_Entity_Identification_Code_5":invoice_header_table['N1_098_Entity_Identification_Code_5'][index],
            "@N1_093_Name_5":invoice_header_table['N1_093_Name_5'][index],
            "@N1_066_Identification_Code_Qualifier_5":invoice_header_table['N1_066_Identification_Code_Qualifier_5'][index],
            "@N1_067_Identification_Code_5":invoice_header_table['N1_067_Identification_Code_5'][index],
            "@N3_166_Address_Info_5":invoice_header_table['N3_166_Address_Info_5'][index],
            "@N4_019_City_Name_5":invoice_header_table['N4_019_City_Name_5'][index],
            "@N4_156_State_Province_Code_5":invoice_header_table['N4_156_State_Province_Code_5'][index],
            "@N4_116_Postal_Code_5":invoice_header_table['N4_116_Postal_Code_5'][index],
            "@N1_098_Entity_Identification_Code_6":invoice_header_table['N1_098_Entity_Identification_Code_6'][index],
            "@N1_093_Name_6":invoice_header_table['N1_093_Name_6'][index],
            "@N1_066_Identification_Code_Qualifier_6":invoice_header_table['N1_066_Identification_Code_Qualifier_6'][index],
            "@N1_067_Identification_Code_6":invoice_header_table['N1_067_Identification_Code_6'][index],
            "@N3_166_Address_Info_6":invoice_header_table['N3_166_Address_Info_6'][index],
            "@N4_019_City_Name_6":invoice_header_table['N4_019_City_Name_6'][index],
            "@N4_156_State_Province_Code_6":invoice_header_table['N4_156_State_Province_Code_6'][index],
            "@N4_116_Postal_Code_6":invoice_header_table['N4_116_Postal_Code_6'][index],
            "@N1_098_Entity_Identification_Code_7":invoice_header_table['N1_098_Entity_Identification_Code_7'][index],
            "@N1_093_Name_7":invoice_header_table['N1_093_Name_7'][index],
            "@N1_066_Identification_Code_Qualifier_7":invoice_header_table['N1_066_Identification_Code_Qualifier_7'][index],
            "@N1_067_Identification_Code_7":invoice_header_table['N1_067_Identification_Code_7'][index],
            "@N3_166_Address_Info_7":invoice_header_table['N3_166_Address_Info_7'][index],
            "@N4_019_City_Name_7":invoice_header_table['N4_019_City_Name_7'][index],
            "@N4_156_State_Province_Code_7":invoice_header_table['N4_156_State_Province_Code_7'][index],
            "@N4_116_Postal_Code_7":invoice_header_table['N4_116_Postal_Code_7'][index],
            "@G31_382_Number_Of_Units_Shipped_1":invoice_header_table['G31_382_Number_Of_Units_Shipped_1'][index],
            "@G31_355_Unit_Or_Basis_For_Measurement_Code_1_1":invoice_header_table['G31_355_Unit_Or_Basis_For_Measurement_Code_1_1'][index],
            "@G31_081_Weight_1":invoice_header_table['G31_081_Weight_1'][index],
            "@G31_355_Unit_Or_Basis_For_Measurement_Code_2_1":invoice_header_table['G31_355_Unit_Or_Basis_For_Measurement_Code_2_1'][index],
            "@G31_183_Volume_1":invoice_header_table['G31_183_Volume_1'][index],
            "@G31_355_Unit_Or_Basis_For_Measurement_Code_3_1":invoice_header_table['G31_355_Unit_Or_Basis_For_Measurement_Code_3_1'][index],
            "@G33_361_Amount_1":invoice_header_table['G33_361_Amount_1'][index],
            "@G31_382_Number_Of_Units_Shipped_2":invoice_header_table['G31_382_Number_Of_Units_Shipped_2'][index],
            "@G31_355_Unit_Or_Basis_For_Measurement_Code_1_2":invoice_header_table['G31_355_Unit_Or_Basis_For_Measurement_Code_1_2'][index],
            "@G31_081_Weight_2":invoice_header_table['G31_081_Weight_2'][index],
            "@G31_355_Unit_Or_Basis_For_Measurement_Code_2_2":invoice_header_table['G31_355_Unit_Or_Basis_For_Measurement_Code_2_2'][index],
            "@G31_183_Volume_2":invoice_header_table['G31_183_Volume_2'][index],
            "@G31_355_Unit_Or_Basis_For_Measurement_Code_3_2":invoice_header_table['G31_355_Unit_Or_Basis_For_Measurement_Code_3_2'][index],
            "@G33_361_Amount_2":invoice_header_table['G33_361_Amount_2'][index],
            "@G31_382_Number_Of_Units_Shipped_3":invoice_header_table['G31_382_Number_Of_Units_Shipped_3'][index],
            "@G31_355_Unit_Or_Basis_For_Measurement_Code_1_3":invoice_header_table['G31_355_Unit_Or_Basis_For_Measurement_Code_1_3'][index],
            "@G31_081_Weight_3":invoice_header_table['G31_081_Weight_3'][index],
            "@G31_355_Unit_Or_Basis_For_Measurement_Code_2_3":invoice_header_table['G31_355_Unit_Or_Basis_For_Measurement_Code_2_3'][index],
            "@G31_183_Volume_3":invoice_header_table['G31_183_Volume_3'][index],
            "@G31_355_Unit_Or_Basis_For_Measurement_Code_3_3":invoice_header_table['G31_355_Unit_Or_Basis_For_Measurement_Code_3_3'][index],
            "@G33_361_Amount_3":invoice_header_table['G33_361_Amount_3'][index]
                }
        
        cursor_1.execute(
                        """EXEC [barney].[sandbox_mp].[dbo].[ATG_EDI_Invoice_Hdr_Upsert] ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
                        ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
                        ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
                        ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
                        ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
                        ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
                        ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
                        ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?""",
                        [parameters[p] for p in parameters]
                    )
        connection.commit()
        
    except Exception as e:
        fails.append([parameters[p] for p in parameters])

In [ ]:
connection.close()

In [ ]:
# HEADER ALLOWANCE TABLE STORED PROCEDURE
count = 1
fail_count = 0
import pyodbc
for index in invoice_header_allowance_table.index:
    try:
        if(int(len(invoice_header_allowance_table)-index)%1000==0):
            clear_output(wait=True)
            print(f"{len(invoice_header_allowance_table)-index:,}")
            
        parameters = {
            "@ATG_Hdr_Ref":index,
            "@G72_340_Allowance_Or_Charge_Code":invoice_header_allowance_table['G72_340_Allowance_Or_Charge_Code'][index],
            "@G72_331_Allowance_Or_Charge_Method_Of_Handling_Code":invoice_header_allowance_table['G72_331_Allowance_Or_Charge_Method_Of_Handling_Code'][index],
            "@G72_359_Allowance_Or_Charge_Rate":invoice_header_allowance_table['G72_359_Allowance_Or_Charge_Rate'][index],
            "@G72_339_Allowance_Or_Charge_Quantity":invoice_header_allowance_table['G72_339_Allowance_Or_Charge_Quantity'][index],
            "@G72_355_Unit_Or_Basis_For_Measurement_Code":invoice_header_allowance_table['G72_355_Unit_Or_Basis_For_Measurement_Code'][index],
            "@G72_360_Allowance_Or_Charge_Total_Amount":invoice_header_allowance_table['G72_360_Allowance_Or_Charge_Total_Amount'][index],
            "@G73_369_Free_Form_Description":invoice_header_allowance_table['G73_369_Free_Form_Description'][index]
                }
        server = 'barney'
        database = 'sandbox_mp'
        connectionString = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Integrated Security={True};Autocommit={True};Trusted_Connection=yes;'
        connection = pyodbc.connect(connectionString)
        cursor = connection.cursor()
        cursor.execute(
                        "EXEC [barney].[sandbox_mp].[dbo].[ATG_EDI_Invoice_Hdr_Allowances_Upsert] ?,?,?,?,?,?,?,?",
                        [str(parameters[p]) for p in parameters]
                    )
        cursor.commit()
    except Exception as e:
        fail_count += 1
        print(e)
        print(f"Fails: {fail_count:,}")

## Read Extracted Data

In [ ]:
df = pd.read_csv('Header_EDI_Data.csv')

In [ ]:
df.head()

In [ ]:
df = df.drop('Unnamed: 0',axis = 1)

In [ ]:
df = df.fillna("NULL")

In [ ]:
df.head()

In [ ]:
df[~(df['N9_127_Reference_Number_1']=="NULL")].head()

## Other Tests

In [ ]:
full_data_hdr_df.head()

In [ ]:
full_data_hdr_allowance_df.head()

In [ ]:
full_data_dtl_allowance_df.head()

### Get Codes Functions

In [ ]:
def get_allowance_numeric_code(code):
    code = str(code)
    for key,item in g72_allowance_or_charge_codes.items():
        if(item==code):
            return key
    return "None"

In [ ]:
def get_allowance_method_numeric_code(code):
    code = str(code)
    for key,item in g72_allowance_or_charge_method_of_handling_codes.items():
        if(item==code):
            return key
    return "None"

In [ ]:
def get_unit_numeric_code(code):
    code = str(code)
    for key,item in g72_unit_or_basis_for_measurement_codes.items():
        if(item==code):
            return key
    return "None"

In [ ]:
def get_reference_qualifier_code(code):
    code = str(code)
    for key,item in n9_reference_identification_qualifier_codes.items():
        if(item==code):
            return key
    return "None"

In [ ]:
def get_note_reference_code(code):
    code = str(code)
    for key,item in nte_note_reference_codes.items():
        if(item==code):
            return key
    return "None"

In [ ]:
def get_transportation_method_code(code):
    code = str(code)
    for key,item in g27_carrier_detail_transportation_method_type_codes.items():
        if(item==code):
            return key
    return "None"

In [ ]:
def get_terms_type_code(code):
    code = str(code)
    for key,item in g23_terms_type_codes.items():
        if(item==code):
            return key
    return "None"

In [ ]:
def get_terms_basis_date_code(code):
    code = str(code)
    for key,item in g23_terms_basis_date_codes.items():
        if(item==code):
            return key
    return "None"

In [ ]:
def get_unit_basis_meaurement_code(code):
    code = str(code)
    for key,item in g31_unit_or_basis_for_measurement_codes.items():
        if(item==code):
            return key
    return "None"

In [ ]:
def get_FOB_point_code(code):
    code = str(code)
    for key,item in g25_fob_point_codes:
        if(item==code):
            return key
    return "None"

In [ ]:
def get_entity_identification_code(code):
    code = str(code)
    for key,item in n1_entity_identifier_codes.items():
        if(item==code):
            return key
    return "None"

In [ ]:
def get_identification_qualifier_code(code):
    code = str(code)
    for key,item in n1_identifier_code_qualifier_codes.items():
        if(item==code):
            return key
    return "None"

### Extract Codes

In [ ]:
full_data_hdr_allowance_df["G72_340_Allowance_Or_Charge_Numeric_Code"] = full_data_hdr_allowance_df["G72_340_Allowance_Or_Charge_Code"].apply(get_allowance_numeric_code)
full_data_hdr_allowance_df["G72_331_Allowance_Or_Charge_Method_Of_Handling_Numeric_Code"] = full_data_hdr_allowance_df["G72_331_Allowance_Or_Charge_Method_Of_Handling_Code"].apply(get_allowance_method_numeric_code)
full_data_hdr_allowance_df["G72_355_Unit_Or_Basis_For_Measurement_Numeric_Code"] = full_data_hdr_allowance_df["G72_355_Unit_Or_Basis_For_Measurement_Code"].apply(get_unit_numeric_code)

In [ ]:
full_data_dtl_allowance_df["G72_340_Allowance_Or_Charge_Numeric_Code"] = full_data_dtl_allowance_df["G72_340_Allowance_Or_Charge_Code"].apply(get_allowance_numeric_code)
full_data_dtl_allowance_df["G72_331_Allowance_Or_Charge_Method_Of_Handling_Numeric_Code"] = full_data_dtl_allowance_df["G72_331_Allowance_Or_Charge_Method_Of_Handling_Code"].apply(get_allowance_method_numeric_code)
full_data_dtl_allowance_df["G72_355_Unit_Or_Basis_For_Measurement_Numeric_Code"] = full_data_dtl_allowance_df["G72_355_Unit_Basis_For_Measurement"].apply(get_unit_numeric_code)

In [ ]:
from time import time
start = time()
if(True):
    print("0/7")
    print(round(time()-start,2))
    print()
    full_data_hdr_df["N9_128_Reference_Number_Qualifier_Numeric_Code_1"] = full_data_hdr_df["N9_128_Reference_Number_Qualifier_Code_1"].apply(get_reference_qualifier_code)
    full_data_hdr_df["N9_128_Reference_Number_Qualifier_Numeric_Code_2"] = full_data_hdr_df["N9_128_Reference_Number_Qualifier_Code_2"].apply(get_reference_qualifier_code)
    full_data_hdr_df["N9_128_Reference_Number_Qualifier_Numeric_Code_3"] = full_data_hdr_df["N9_128_Reference_Number_Qualifier_Code_3"].apply(get_reference_qualifier_code)
    full_data_hdr_df["N9_128_Reference_Number_Qualifier_Numeric_Code_4"] = full_data_hdr_df["N9_128_Reference_Number_Qualifier_Code_4"].apply(get_reference_qualifier_code)
    full_data_hdr_df["N9_128_Reference_Number_Qualifier_Numeric_Code_5"] = full_data_hdr_df["N9_128_Reference_Number_Qualifier_Code_5"].apply(get_reference_qualifier_code)
    full_data_hdr_df["N9_128_Reference_Number_Qualifier_Numeric_Code_6"] = full_data_hdr_df["N9_128_Reference_Number_Qualifier_Code_6"].apply(get_reference_qualifier_code)
    full_data_hdr_df["N9_128_Reference_Number_Qualifier_Numeric_Code_7"] = full_data_hdr_df["N9_128_Reference_Number_Qualifier_Code_7"].apply(get_reference_qualifier_code)
    full_data_hdr_df["N9_128_Reference_Number_Qualifier_Numeric_Code_8"] = full_data_hdr_df["N9_128_Reference_Number_Qualifier_Code_8"].apply(get_reference_qualifier_code)
    full_data_hdr_df["N9_128_Reference_Number_Qualifier_Numeric_Code_9"] = full_data_hdr_df["N9_128_Reference_Number_Qualifier_Code_9"].apply(get_reference_qualifier_code)
    full_data_hdr_df["N9_128_Reference_Number_Qualifier_Numeric_Code_10"] = full_data_hdr_df["N9_128_Reference_Number_Qualifier_Code_10"].apply(get_reference_qualifier_code)
if(True):
    print("1/7")
    print(round(time()-start,2))
    print()
    full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_1"] = full_data_hdr_df["NTE_363_Note_Reference_Code_1"].apply(get_note_reference_code)
    full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_2"] = full_data_hdr_df["NTE_363_Note_Reference_Code_2"].apply(get_note_reference_code)
    full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_3"] = full_data_hdr_df["NTE_363_Note_Reference_Code_3"].apply(get_note_reference_code)
    full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_4"] = full_data_hdr_df["NTE_363_Note_Reference_Code_4"].apply(get_note_reference_code)
    full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_5"] = full_data_hdr_df["NTE_363_Note_Reference_Code_5"].apply(get_note_reference_code)
    full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_6"] = full_data_hdr_df["NTE_363_Note_Reference_Code_6"].apply(get_note_reference_code)
    full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_7"] = full_data_hdr_df["NTE_363_Note_Reference_Code_7"].apply(get_note_reference_code)
    full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_8"] = full_data_hdr_df["NTE_363_Note_Reference_Code_8"].apply(get_note_reference_code)
    full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_9"] = full_data_hdr_df["NTE_363_Note_Reference_Code_9"].apply(get_note_reference_code)
    full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_10"] = full_data_hdr_df["NTE_363_Note_Reference_Code_10"].apply(get_note_reference_code)
    full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_11"] = full_data_hdr_df["NTE_363_Note_Reference_Code_11"].apply(get_note_reference_code)
    full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_12"] = full_data_hdr_df["NTE_363_Note_Reference_Code_12"].apply(get_note_reference_code)
    full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_13"] = full_data_hdr_df["NTE_363_Note_Reference_Code_13"].apply(get_note_reference_code)
    full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_14"] = full_data_hdr_df["NTE_363_Note_Reference_Code_14"].apply(get_note_reference_code)
    full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_15"] = full_data_hdr_df["NTE_363_Note_Reference_Code_15"].apply(get_note_reference_code)
    full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_16"] = full_data_hdr_df["NTE_363_Note_Reference_Code_16"].apply(get_note_reference_code)
    full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_17"] = full_data_hdr_df["NTE_363_Note_Reference_Code_17"].apply(get_note_reference_code)
    full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_18"] = full_data_hdr_df["NTE_363_Note_Reference_Code_18"].apply(get_note_reference_code)
    full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_19"] = full_data_hdr_df["NTE_363_Note_Reference_Code_19"].apply(get_note_reference_code)
    full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_20"] = full_data_hdr_df["NTE_363_Note_Reference_Code_20"].apply(get_note_reference_code)
if(True):
    print("2/7")
    print(round(time()-start,2))
    print()
    full_data_hdr_df["G27_091_Transportation_Method_Type_Numeric_Code_1"] = full_data_hdr_df["G27_091_Transportation_Method_Type_Code_1"].apply(get_transportation_method_code)
    full_data_hdr_df["G27_091_Transportation_Method_Type_Numeric_Code_2"] = full_data_hdr_df["G27_091_Transportation_Method_Type_Code_2"].apply(get_transportation_method_code)
    full_data_hdr_df["G27_091_Transportation_Method_Type_Numeric_Code_3"] = full_data_hdr_df["G27_091_Transportation_Method_Type_Code_3"].apply(get_transportation_method_code)
    full_data_hdr_df["G27_091_Transportation_Method_Type_Numeric_Code_4"] = full_data_hdr_df["G27_091_Transportation_Method_Type_Code_4"].apply(get_transportation_method_code)
    full_data_hdr_df["G27_091_Transportation_Method_Type_Numeric_Code_5"] = full_data_hdr_df["G27_091_Transportation_Method_Type_Code_5"].apply(get_transportation_method_code)
if(True):
    print("3/7")
    print(round(time()-start,2))
    print()
    full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_1"] = full_data_hdr_df["G23_336_Terms_Type_Code_1"].apply(get_terms_type_code)
    full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_1"] = full_data_hdr_df["G23_333_Terms_Basis_Date_Code_1"].apply(get_terms_basis_date_code)
    full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_2"] = full_data_hdr_df["G23_336_Terms_Type_Code_2"].apply(get_terms_type_code)
    full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_2"] = full_data_hdr_df["G23_333_Terms_Basis_Date_Code_2"].apply(get_terms_basis_date_code)
    full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_3"] = full_data_hdr_df["G23_336_Terms_Type_Code_3"].apply(get_terms_type_code)
    full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_3"] = full_data_hdr_df["G23_333_Terms_Basis_Date_Code_3"].apply(get_terms_basis_date_code)
    full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_4"] = full_data_hdr_df["G23_336_Terms_Type_Code_4"].apply(get_terms_type_code)
    full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_4"] = full_data_hdr_df["G23_333_Terms_Basis_Date_Code_4"].apply(get_terms_basis_date_code)
    full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_5"] = full_data_hdr_df["G23_336_Terms_Type_Code_5"].apply(get_terms_type_code)
    full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_5"] = full_data_hdr_df["G23_333_Terms_Basis_Date_Code_5"].apply(get_terms_basis_date_code)
    full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_6"] = full_data_hdr_df["G23_336_Terms_Type_Code_6"].apply(get_terms_type_code)
    full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_6"] = full_data_hdr_df["G23_333_Terms_Basis_Date_Code_6"].apply(get_terms_basis_date_code)
    full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_7"] = full_data_hdr_df["G23_336_Terms_Type_Code_7"].apply(get_terms_type_code)
    full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_7"] = full_data_hdr_df["G23_333_Terms_Basis_Date_Code_7"].apply(get_terms_basis_date_code)
    full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_8"] = full_data_hdr_df["G23_336_Terms_Type_Code_8"].apply(get_terms_type_code)
    full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_8"] = full_data_hdr_df["G23_333_Terms_Basis_Date_Code_8"].apply(get_terms_basis_date_code)
    full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_9"] = full_data_hdr_df["G23_336_Terms_Type_Code_9"].apply(get_terms_type_code)
    full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_9"] = full_data_hdr_df["G23_333_Terms_Basis_Date_Code_9"].apply(get_terms_basis_date_code)
    full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_10"] = full_data_hdr_df["G23_336_Terms_Type_Code_10"].apply(get_terms_type_code)
    full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_10"] = full_data_hdr_df["G23_333_Terms_Basis_Date_Code_10"].apply(get_terms_basis_date_code)
    full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_11"] = full_data_hdr_df["G23_336_Terms_Type_Code_11"].apply(get_terms_type_code)
    full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_11"] = full_data_hdr_df["G23_333_Terms_Basis_Date_Code_11"].apply(get_terms_basis_date_code)
    full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_12"] = full_data_hdr_df["G23_336_Terms_Type_Code_12"].apply(get_terms_type_code)
    full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_12"] = full_data_hdr_df["G23_333_Terms_Basis_Date_Code_12"].apply(get_terms_basis_date_code)
    full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_13"] = full_data_hdr_df["G23_336_Terms_Type_Code_13"].apply(get_terms_type_code)
    full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_13"] = full_data_hdr_df["G23_333_Terms_Basis_Date_Code_13"].apply(get_terms_basis_date_code)
    full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_14"] = full_data_hdr_df["G23_336_Terms_Type_Code_14"].apply(get_terms_type_code)
    full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_14"] = full_data_hdr_df["G23_333_Terms_Basis_Date_Code_14"].apply(get_terms_basis_date_code)
    full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_15"] = full_data_hdr_df["G23_336_Terms_Type_Code_15"].apply(get_terms_type_code)
    full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_15"] = full_data_hdr_df["G23_333_Terms_Basis_Date_Code_15"].apply(get_terms_basis_date_code)
    full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_16"] = full_data_hdr_df["G23_336_Terms_Type_Code_16"].apply(get_terms_type_code)
    full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_16"] = full_data_hdr_df["G23_333_Terms_Basis_Date_Code_16"].apply(get_terms_basis_date_code)
    full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_17"] = full_data_hdr_df["G23_336_Terms_Type_Code_17"].apply(get_terms_type_code)
    full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_17"] = full_data_hdr_df["G23_333_Terms_Basis_Date_Code_17"].apply(get_terms_basis_date_code)
    full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_18"] = full_data_hdr_df["G23_336_Terms_Type_Code_18"].apply(get_terms_type_code)
    full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_18"] = full_data_hdr_df["G23_333_Terms_Basis_Date_Code_18"].apply(get_terms_basis_date_code)
    full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_19"] = full_data_hdr_df["G23_336_Terms_Type_Code_19"].apply(get_terms_type_code)
    full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_19"] = full_data_hdr_df["G23_333_Terms_Basis_Date_Code_19"].apply(get_terms_basis_date_code)
    full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_20"] = full_data_hdr_df["G23_336_Terms_Type_Code_20"].apply(get_terms_type_code)
    full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_20"] = full_data_hdr_df["G23_333_Terms_Basis_Date_Code_20"].apply(get_terms_basis_date_code)
if(True):
    print("4/7")
    print(round(time()-start,2))
    print()
    full_data_hdr_df["G25_433_FOB_Point_Numeric_Code"] = full_data_hdr_df["G25_433_FOB_Point_Code"].apply(get_FOB_point_code)
if(True):
    print("5/7")
    print(round(time()-start,2))
    print()
    full_data_hdr_df["N1_098_Entity_Identification_Numeric_Code_1"] = full_data_hdr_df["N1_098_Entity_Identification_Code_1"].apply(get_entity_identification_code)
    full_data_hdr_df["N1_066_Identification_Numeric_Code_Qualifier_1"] = full_data_hdr_df["N1_066_Identification_Code_Qualifier_1"].apply(get_identification_qualifier_code)
    full_data_hdr_df["N1_098_Entity_Identification_Numeric_Code_2"] = full_data_hdr_df["N1_098_Entity_Identification_Code_2"].apply(get_entity_identification_code)
    full_data_hdr_df["N1_066_Identification_Numeric_Code_Qualifier_2"] = full_data_hdr_df["N1_066_Identification_Code_Qualifier_2"].apply(get_identification_qualifier_code)
    full_data_hdr_df["N1_098_Entity_Identification_Numeric_Code_3"] = full_data_hdr_df["N1_098_Entity_Identification_Code_3"].apply(get_entity_identification_code)
    full_data_hdr_df["N1_066_Identification_Numeric_Code_Qualifier_3"] = full_data_hdr_df["N1_066_Identification_Code_Qualifier_3"].apply(get_identification_qualifier_code)
    full_data_hdr_df["N1_098_Entity_Identification_Numeric_Code_4"] = full_data_hdr_df["N1_098_Entity_Identification_Code_4"].apply(get_entity_identification_code)
    full_data_hdr_df["N1_066_Identification_Numeric_Code_Qualifier_4"] = full_data_hdr_df["N1_066_Identification_Code_Qualifier_4"].apply(get_identification_qualifier_code)
    full_data_hdr_df["N1_098_Entity_Identification_Numeric_Code_5"] = full_data_hdr_df["N1_098_Entity_Identification_Code_5"].apply(get_entity_identification_code)
    full_data_hdr_df["N1_066_Identification_Numeric_Code_Qualifier_5"] = full_data_hdr_df["N1_066_Identification_Code_Qualifier_5"].apply(get_identification_qualifier_code)
    full_data_hdr_df["N1_098_Entity_Identification_Numeric_Code_6"] = full_data_hdr_df["N1_098_Entity_Identification_Code_6"].apply(get_entity_identification_code)
    full_data_hdr_df["N1_066_Identification_Numeric_Code_Qualifier_6"] = full_data_hdr_df["N1_066_Identification_Code_Qualifier_6"].apply(get_identification_qualifier_code)
    full_data_hdr_df["N1_067_Identification_Numeric_Code_6"] = "None"
    full_data_hdr_df["N1_098_Entity_Identification_Numeric_Code_7"] = full_data_hdr_df["N1_098_Entity_Identification_Code_7"].apply(get_entity_identification_code)
    full_data_hdr_df["N1_066_Identification_Numeric_Code_Qualifier_7"] = full_data_hdr_df["N1_066_Identification_Code_Qualifier_7"].apply(get_identification_qualifier_code)
    full_data_hdr_df["N1_067_Identification_Numeric_Code_7"] = "None"
if(True):
    print("6/7")
    print(round(time()-start,2))
    print()
    full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_1_1"] = full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Code_1_1"].apply(get_unit_basis_meaurement_code)
    full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_2_1"] = full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Code_2_1"].apply(get_unit_basis_meaurement_code)
    full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_3_1"] = full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Code_3_1"].apply(get_unit_basis_meaurement_code)
    full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_1_2"] = full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Code_1_2"].apply(get_unit_basis_meaurement_code)
    full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_2_2"] = full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Code_2_2"].apply(get_unit_basis_meaurement_code)
    full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_3_2"] = full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Code_3_2"].apply(get_unit_basis_meaurement_code)
    full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_1_3"] = full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Code_1_3"].apply(get_unit_basis_meaurement_code)
    full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_2_3"] = full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Code_2_3"].apply(get_unit_basis_meaurement_code)
    full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_3_3"] = full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Code_3_3"].apply(get_unit_basis_meaurement_code)
print("7/7")
print(round(time()-start,2))

### Read New Extractions

In [ ]:
full_data_hdr_df = full_data_hdr_df.fillna("None")

In [ ]:
for col in full_data_hdr_df.columns:
    print(len(full_data_hdr_df[col].notna()))

In [ ]:
full_data_hdr_allowance_df.head()

In [ ]:
print(list(full_data_hdr_allowance_df.columns))

In [ ]:
full_data_dtl_allowance_df.head()

In [ ]:
print(list(full_data_dtl_allowance_df.columns))

### Upload New Extractions

In [ ]:
fail_count = 0
for index in list(full_data_hdr_df.index):
	# try:
		if(int(index)%1000==0):
			clear_output(wait=False)
			print(f"{index:,}\n{len(list(full_data_hdr_df.index))-index:,}")
		parameters = {
		"@G01_245_Invoice_Date": str(full_data_hdr_df["G01_245_Invoice_Date"][index]).replace(" 00:00:00","").replace("NaT",""),
		"@G01_076_Invoice_Number": full_data_hdr_df["G01_076_Invoice_Number"][index],
		"@G01_323_Purchase_Order_Date": str(full_data_hdr_df["G01_323_Purchase_Order_Date"][index]).replace(" 00:00:00","").replace("NaT",""),
		"@G01_324_Purchase_Order_Number": full_data_hdr_df["G01_324_Purchase_Order_Number"][index],
		"@N9_128_Reference_Number_Qualifier_Code_1": full_data_hdr_df["N9_128_Reference_Number_Qualifier_Code_1"][index],
		"@N9_127_Reference_Number_1": full_data_hdr_df["N9_127_Reference_Number_1"][index],
		"@N9_369_Free_Form_Description_1": full_data_hdr_df["N9_369_Free_Form_Description_1"][index],
		"@N9_128_Reference_Number_Qualifier_Code_2": full_data_hdr_df["N9_128_Reference_Number_Qualifier_Code_2"][index],
		"@N9_127_Reference_Number_2": full_data_hdr_df["N9_127_Reference_Number_2"][index],
		"@N9_369_Free_Form_Description_2": full_data_hdr_df["N9_369_Free_Form_Description_2"][index],
		"@N9_128_Reference_Number_Qualifier_Code_3": full_data_hdr_df["N9_128_Reference_Number_Qualifier_Code_3"][index],
		"@N9_127_Reference_Number_3": full_data_hdr_df["N9_127_Reference_Number_3"][index],
		"@N9_369_Free_Form_Description_3": full_data_hdr_df["N9_369_Free_Form_Description_3"][index],
		"@N9_128_Reference_Number_Qualifier_Code_4": full_data_hdr_df["N9_128_Reference_Number_Qualifier_Code_4"][index],
		"@N9_127_Reference_Number_4": full_data_hdr_df["N9_127_Reference_Number_4"][index],
		"@N9_369_Free_Form_Description_4": full_data_hdr_df["N9_369_Free_Form_Description_4"][index],
		"@N9_128_Reference_Number_Qualifier_Code_5": full_data_hdr_df["N9_128_Reference_Number_Qualifier_Code_5"][index],
		"@N9_127_Reference_Number_5": full_data_hdr_df["N9_127_Reference_Number_5"][index],
		"@N9_369_Free_Form_Description_5": full_data_hdr_df["N9_369_Free_Form_Description_5"][index],
		"@N9_128_Reference_Number_Qualifier_Code_6": full_data_hdr_df["N9_128_Reference_Number_Qualifier_Code_6"][index],
		"@N9_127_Reference_Number_6": full_data_hdr_df["N9_127_Reference_Number_6"][index],
		"@N9_369_Free_Form_Description_6": full_data_hdr_df["N9_369_Free_Form_Description_6"][index],
		"@N9_128_Reference_Number_Qualifier_Code_7": full_data_hdr_df["N9_128_Reference_Number_Qualifier_Code_7"][index],
		"@N9_127_Reference_Number_7": full_data_hdr_df["N9_127_Reference_Number_7"][index],
		"@N9_369_Free_Form_Description_7": full_data_hdr_df["N9_369_Free_Form_Description_7"][index],
		"@N9_128_Reference_Number_Qualifier_Code_8": full_data_hdr_df["N9_128_Reference_Number_Qualifier_Code_8"][index],
		"@N9_127_Reference_Number_8": full_data_hdr_df["N9_127_Reference_Number_8"][index],
		"@N9_369_Free_Form_Description_8": full_data_hdr_df["N9_369_Free_Form_Description_8"][index],
		"@N9_128_Reference_Number_Qualifier_Code_9": full_data_hdr_df["N9_128_Reference_Number_Qualifier_Code_9"][index],
		"@N9_127_Reference_Number_9": full_data_hdr_df["N9_127_Reference_Number_9"][index],
		"@N9_369_Free_Form_Description_9": full_data_hdr_df["N9_369_Free_Form_Description_9"][index],
		"@N9_128_Reference_Number_Qualifier_Code_10": full_data_hdr_df["N9_128_Reference_Number_Qualifier_Code_10"][index],
		"@N9_127_Reference_Number_10": full_data_hdr_df["N9_127_Reference_Number_10"][index],
		"@N9_369_Free_Form_Description_10": full_data_hdr_df["N9_369_Free_Form_Description_10"][index],
		"@G62_432_Date_Qualifier_1": full_data_hdr_df["G62_432_Date_Qualifier_1"][index],
		"@G62_373_Date_1": str(full_data_hdr_df["G62_373_Date_1"][index]).replace(" 00:00:00","").replace("NaT",""),
		"@G62_432_Date_Qualifier_2": full_data_hdr_df["G62_432_Date_Qualifier_2"][index],
		"@G62_373_Date_2": str(full_data_hdr_df["G62_373_Date_2"][index]).replace(" 00:00:00","").replace("NaT",""),
		"@G62_432_Date_Qualifier_3": full_data_hdr_df["G62_432_Date_Qualifier_3"][index],
		"@G62_373_Date_3": str(full_data_hdr_df["G62_373_Date_3"][index]).replace(" 00:00:00","").replace("NaT",""),
		"@G62_432_Date_Qualifier_4": full_data_hdr_df["G62_432_Date_Qualifier_4"][index],
		"@G62_373_Date_4": str(full_data_hdr_df["G62_373_Date_4"][index]).replace(" 00:00:00","").replace("NaT",""),
		"@G62_432_Date_Qualifier_5": full_data_hdr_df["G62_432_Date_Qualifier_5"][index],
		"@G62_373_Date_5": str(full_data_hdr_df["G62_373_Date_5"][index]).replace(" 00:00:00","").replace("NaT",""),
		"@NTE_363_Note_Reference_Code_1": full_data_hdr_df["NTE_363_Note_Reference_Code_1"][index],
		"@NTE_003_Free_Form_Message_1": full_data_hdr_df["NTE_003_Free_Form_Message_1"][index],
		"@NTE_363_Note_Reference_Code_2": full_data_hdr_df["NTE_363_Note_Reference_Code_2"][index],
		"@NTE_003_Free_Form_Message_2": full_data_hdr_df["NTE_003_Free_Form_Message_2"][index],
		"@NTE_363_Note_Reference_Code_3": full_data_hdr_df["NTE_363_Note_Reference_Code_3"][index],
		"@NTE_003_Free_Form_Message_3": full_data_hdr_df["NTE_003_Free_Form_Message_3"][index],
		"@NTE_363_Note_Reference_Code_4": full_data_hdr_df["NTE_363_Note_Reference_Code_4"][index],
		"@NTE_003_Free_Form_Message_4": full_data_hdr_df["NTE_003_Free_Form_Message_4"][index],
		"@NTE_363_Note_Reference_Code_5": full_data_hdr_df["NTE_363_Note_Reference_Code_5"][index],
		"@NTE_003_Free_Form_Message_5": full_data_hdr_df["NTE_003_Free_Form_Message_5"][index],
		"@NTE_363_Note_Reference_Code_6": full_data_hdr_df["NTE_363_Note_Reference_Code_6"][index],
		"@NTE_003_Free_Form_Message_6": full_data_hdr_df["NTE_003_Free_Form_Message_6"][index],
		"@NTE_363_Note_Reference_Code_7": full_data_hdr_df["NTE_363_Note_Reference_Code_7"][index],
		"@NTE_003_Free_Form_Message_7": full_data_hdr_df["NTE_003_Free_Form_Message_7"][index],
		"@NTE_363_Note_Reference_Code_8": full_data_hdr_df["NTE_363_Note_Reference_Code_8"][index],
		"@NTE_003_Free_Form_Message_8": full_data_hdr_df["NTE_003_Free_Form_Message_8"][index],
		"@NTE_363_Note_Reference_Code_9": full_data_hdr_df["NTE_363_Note_Reference_Code_9"][index],
		"@NTE_003_Free_Form_Message_9": full_data_hdr_df["NTE_003_Free_Form_Message_9"][index],
		"@NTE_363_Note_Reference_Code_10": full_data_hdr_df["NTE_363_Note_Reference_Code_10"][index],
		"@NTE_003_Free_Form_Message_10": full_data_hdr_df["NTE_003_Free_Form_Message_10"][index],
		"@NTE_363_Note_Reference_Code_11": full_data_hdr_df["NTE_363_Note_Reference_Code_11"][index],
		"@NTE_003_Free_Form_Message_11": full_data_hdr_df["NTE_003_Free_Form_Message_11"][index],
		"@NTE_363_Note_Reference_Code_12": full_data_hdr_df["NTE_363_Note_Reference_Code_12"][index],
		"@NTE_003_Free_Form_Message_12": full_data_hdr_df["NTE_003_Free_Form_Message_12"][index],
		"@NTE_363_Note_Reference_Code_13": full_data_hdr_df["NTE_363_Note_Reference_Code_13"][index],
		"@NTE_003_Free_Form_Message_13": full_data_hdr_df["NTE_003_Free_Form_Message_13"][index],
		"@NTE_363_Note_Reference_Code_14": full_data_hdr_df["NTE_363_Note_Reference_Code_14"][index],
		"@NTE_003_Free_Form_Message_14": full_data_hdr_df["NTE_003_Free_Form_Message_14"][index],
		"@NTE_363_Note_Reference_Code_15": full_data_hdr_df["NTE_363_Note_Reference_Code_15"][index],
		"@NTE_003_Free_Form_Message_15": full_data_hdr_df["NTE_003_Free_Form_Message_15"][index],
		"@NTE_363_Note_Reference_Code_16": full_data_hdr_df["NTE_363_Note_Reference_Code_16"][index],
		"@NTE_003_Free_Form_Message_16": full_data_hdr_df["NTE_003_Free_Form_Message_16"][index],
		"@NTE_363_Note_Reference_Code_17": full_data_hdr_df["NTE_363_Note_Reference_Code_17"][index],
		"@NTE_003_Free_Form_Message_17": full_data_hdr_df["NTE_003_Free_Form_Message_17"][index],
		"@NTE_363_Note_Reference_Code_18": full_data_hdr_df["NTE_363_Note_Reference_Code_18"][index],
		"@NTE_003_Free_Form_Message_18": full_data_hdr_df["NTE_003_Free_Form_Message_18"][index],
		"@NTE_363_Note_Reference_Code_19": full_data_hdr_df["NTE_363_Note_Reference_Code_19"][index],
		"@NTE_003_Free_Form_Message_19": full_data_hdr_df["NTE_003_Free_Form_Message_19"][index],
		"@NTE_363_Note_Reference_Code_20": full_data_hdr_df["NTE_363_Note_Reference_Code_20"][index],
		"@NTE_003_Free_Form_Message_20": full_data_hdr_df["NTE_003_Free_Form_Message_20"][index],
		"@G27_091_Transportation_Method_Type_Code_1": full_data_hdr_df["G27_091_Transportation_Method_Type_Code_1"][index],
		"@G27_207_Equipment_Number_1": full_data_hdr_df["G27_207_Equipment_Number_1"][index],
		"@G27_387_Routing_1": full_data_hdr_df["G27_387_Routing_1"][index],
		"@G27_091_Transportation_Method_Type_Code_2": full_data_hdr_df["G27_091_Transportation_Method_Type_Code_2"][index],
		"@G27_207_Equipment_Number_2": full_data_hdr_df["G27_207_Equipment_Number_2"][index],
		"@G27_387_Routing_2": full_data_hdr_df["G27_387_Routing_2"][index],
		"@G27_091_Transportation_Method_Type_Code_3": full_data_hdr_df["G27_091_Transportation_Method_Type_Code_3"][index],
		"@G27_207_Equipment_Number_3": full_data_hdr_df["G27_207_Equipment_Number_3"][index],
		"@G27_387_Routing_3": full_data_hdr_df["G27_387_Routing_3"][index],
		"@G27_091_Transportation_Method_Type_Code_4": full_data_hdr_df["G27_091_Transportation_Method_Type_Code_4"][index],
		"@G27_207_Equipment_Number_4": full_data_hdr_df["G27_207_Equipment_Number_4"][index],
		"@G27_387_Routing_4": full_data_hdr_df["G27_387_Routing_4"][index],
		"@G27_091_Transportation_Method_Type_Code_5": full_data_hdr_df["G27_091_Transportation_Method_Type_Code_5"][index],
		"@G27_207_Equipment_Number_5": full_data_hdr_df["G27_207_Equipment_Number_5"][index],
		"@G27_387_Routing_5": full_data_hdr_df["G27_387_Routing_5"][index],
		"@G23_336_Terms_Type_Code_1": full_data_hdr_df["G23_336_Terms_Type_Code_1"][index],
		"@G23_333_Terms_Basis_Date_Code_1": full_data_hdr_df["G23_333_Terms_Basis_Date_Code_1"][index],
		"@G23_282_Terms_Start_Date_1": str(full_data_hdr_df["G23_282_Terms_Start_Date_1"][index]).replace(" 00:00:00","").replace("NaT",""),
		"@G23_283_Terms_Due_Date_Qualifier_1": full_data_hdr_df["G23_283_Terms_Due_Date_Qualifier_1"][index],
		"@G23_338_Terms_Discount_Percent_1": full_data_hdr_df["G23_338_Terms_Discount_Percent_1"][index],
		"@G23_370_Terms_Discount_Due_Date_1": str(full_data_hdr_df["G23_370_Terms_Discount_Due_Date_1"][index]).replace(" 00:00:00","").replace("NaT",""),
		"@G23_351_Terms_Discount_Days_Due_1": full_data_hdr_df["G23_351_Terms_Discount_Days_Due_1"][index],
		"@G23_446_Terms_Net_Due_Date_1": str(full_data_hdr_df["G23_446_Terms_Net_Due_Date_1"][index]).replace(" 00:00:00","").replace("NaT",""),
		"@G23_386_Terms_Net_Days_1": full_data_hdr_df["G23_386_Terms_Net_Days_1"][index],
		"@G23_362_Terms_Discount_Amount_1": full_data_hdr_df["G23_362_Terms_Discount_Amount_1"][index],
		"@G23_391_Discounted_Amount_1": full_data_hdr_df["G23_391_Discounted_Amount_1"][index],
		"@G23_390_Amount_Subject_To_Terms_Discount_1": full_data_hdr_df["G23_390_Amount_Subject_To_Terms_Discount_1"][index],
		"@G23_003_Free_Form_Message_1": full_data_hdr_df["G23_003_Free_Form_Message_1"][index],
		"@G23_336_Terms_Type_Code_2": full_data_hdr_df["G23_336_Terms_Type_Code_2"][index],
		"@G23_333_Terms_Basis_Date_Code_2": full_data_hdr_df["G23_333_Terms_Basis_Date_Code_2"][index],
		"@G23_282_Terms_Start_Date_2": str(full_data_hdr_df["G23_282_Terms_Start_Date_2"][index]).replace(" 00:00:00","").replace("NaT",""),
		"@G23_283_Terms_Due_Date_Qualifier_2": full_data_hdr_df["G23_283_Terms_Due_Date_Qualifier_2"][index],
		"@G23_338_Terms_Discount_Percent_2": full_data_hdr_df["G23_338_Terms_Discount_Percent_2"][index],
		"@G23_370_Terms_Discount_Due_Date_2": full_data_hdr_df["G23_370_Terms_Discount_Due_Date_2"][index].replace(" 00:00:00","").replace("NaT",""),
		"@G23_351_Terms_Discount_Days_Due_2": full_data_hdr_df["G23_351_Terms_Discount_Days_Due_2"][index],
		"@G23_446_Terms_Net_Due_Date_2": str(full_data_hdr_df["G23_446_Terms_Net_Due_Date_2"][index]).replace(" 00:00:00","").replace("NaT",""),
		"@G23_386_Terms_Net_Days_2": full_data_hdr_df["G23_386_Terms_Net_Days_2"][index],
		"@G23_362_Terms_Discount_Amount_2": full_data_hdr_df["G23_362_Terms_Discount_Amount_2"][index],
		"@G23_391_Discounted_Amount_2": full_data_hdr_df["G23_391_Discounted_Amount_2"][index],
		"@G23_390_Amount_Subject_To_Terms_Discount_2": full_data_hdr_df["G23_390_Amount_Subject_To_Terms_Discount_2"][index],
		"@G23_003_Free_Form_Message_2": full_data_hdr_df["G23_003_Free_Form_Message_2"][index],
		"@G23_336_Terms_Type_Code_3": full_data_hdr_df["G23_336_Terms_Type_Code_3"][index],
		"@G23_333_Terms_Basis_Date_Code_3": full_data_hdr_df["G23_333_Terms_Basis_Date_Code_3"][index],
		"@G23_282_Terms_Start_Date_3": str(full_data_hdr_df["G23_282_Terms_Start_Date_3"][index]).replace(" 00:00:00","").replace("NaT",""),
		"@G23_283_Terms_Due_Date_Qualifier_3": full_data_hdr_df["G23_283_Terms_Due_Date_Qualifier_3"][index],
		"@G23_338_Terms_Discount_Percent_3": full_data_hdr_df["G23_338_Terms_Discount_Percent_3"][index],
		"@G23_370_Terms_Discount_Due_Date_3": full_data_hdr_df["G23_370_Terms_Discount_Due_Date_3"][index].replace(" 00:00:00","").replace("NaT",""),
		"@G23_351_Terms_Discount_Days_Due_3": full_data_hdr_df["G23_351_Terms_Discount_Days_Due_3"][index],
		"@G23_446_Terms_Net_Due_Date_3": str(full_data_hdr_df["G23_446_Terms_Net_Due_Date_3"][index]).replace(" 00:00:00","").replace("NaT",""),
		"@G23_386_Terms_Net_Days_3": full_data_hdr_df["G23_386_Terms_Net_Days_3"][index],
		"@G23_362_Terms_Discount_Amount_3": full_data_hdr_df["G23_362_Terms_Discount_Amount_3"][index],
		"@G23_391_Discounted_Amount_3": full_data_hdr_df["G23_391_Discounted_Amount_3"][index],
		"@G23_390_Amount_Subject_To_Terms_Discount_3": full_data_hdr_df["G23_390_Amount_Subject_To_Terms_Discount_3"][index],
		"@G23_003_Free_Form_Message_3": full_data_hdr_df["G23_003_Free_Form_Message_3"][index],
		"@G23_336_Terms_Type_Code_4": full_data_hdr_df["G23_336_Terms_Type_Code_4"][index],
		"@G23_333_Terms_Basis_Date_Code_4": full_data_hdr_df["G23_333_Terms_Basis_Date_Code_4"][index],
		"@G23_282_Terms_Start_Date_4": str(full_data_hdr_df["G23_282_Terms_Start_Date_4"][index]).replace(" 00:00:00","").replace("NaT",""),
		"@G23_283_Terms_Due_Date_Qualifier_4": full_data_hdr_df["G23_283_Terms_Due_Date_Qualifier_4"][index],
		"@G23_338_Terms_Discount_Percent_4": full_data_hdr_df["G23_338_Terms_Discount_Percent_4"][index],
		"@G23_370_Terms_Discount_Due_Date_4": str(full_data_hdr_df["G23_370_Terms_Discount_Due_Date_4"][index]).replace(" 00:00:00","").replace("NaT",""),
		"@G23_351_Terms_Discount_Days_Due_4": full_data_hdr_df["G23_351_Terms_Discount_Days_Due_4"][index],
		"@G23_446_Terms_Net_Due_Date_4": str(full_data_hdr_df["G23_446_Terms_Net_Due_Date_4"][index]).replace(" 00:00:00","").replace("NaT",""),
		"@G23_386_Terms_Net_Days_4": full_data_hdr_df["G23_386_Terms_Net_Days_4"][index],
		"@G23_362_Terms_Discount_Amount_4": full_data_hdr_df["G23_362_Terms_Discount_Amount_4"][index],
		"@G23_391_Discounted_Amount_4": full_data_hdr_df["G23_391_Discounted_Amount_4"][index],
		"@G23_390_Amount_Subject_To_Terms_Discount_4": full_data_hdr_df["G23_390_Amount_Subject_To_Terms_Discount_4"][index],
		"@G23_003_Free_Form_Message_4": full_data_hdr_df["G23_003_Free_Form_Message_4"][index],
		"@G23_336_Terms_Type_Code_5": full_data_hdr_df["G23_336_Terms_Type_Code_5"][index],
		"@G23_333_Terms_Basis_Date_Code_5": full_data_hdr_df["G23_333_Terms_Basis_Date_Code_5"][index],
		"@G23_282_Terms_Start_Date_5": str(full_data_hdr_df["G23_282_Terms_Start_Date_5"][index]).replace(" 00:00:00","").replace("NaT",""),
		"@G23_283_Terms_Due_Date_Qualifier_5": full_data_hdr_df["G23_283_Terms_Due_Date_Qualifier_5"][index],
		"@G23_338_Terms_Discount_Percent_5": full_data_hdr_df["G23_338_Terms_Discount_Percent_5"][index],
		"@G23_370_Terms_Discount_Due_Date_5": str(full_data_hdr_df["G23_370_Terms_Discount_Due_Date_5"][index]).replace(" 00:00:00","").replace("NaT",""),
		"@G23_351_Terms_Discount_Days_Due_5": full_data_hdr_df["G23_351_Terms_Discount_Days_Due_5"][index],
		"@G23_446_Terms_Net_Due_Date_5": str(full_data_hdr_df["G23_446_Terms_Net_Due_Date_5"][index]).replace(" 00:00:00","").replace("NaT",""),
		"@G23_386_Terms_Net_Days_5": full_data_hdr_df["G23_386_Terms_Net_Days_5"][index],
		"@G23_362_Terms_Discount_Amount_5": full_data_hdr_df["G23_362_Terms_Discount_Amount_5"][index],
		"@G23_391_Discounted_Amount_5": full_data_hdr_df["G23_391_Discounted_Amount_5"][index],
		"@G23_390_Amount_Subject_To_Terms_Discount_5": full_data_hdr_df["G23_390_Amount_Subject_To_Terms_Discount_5"][index],
		"@G23_003_Free_Form_Message_5": full_data_hdr_df["G23_003_Free_Form_Message_5"][index],
		"@G23_336_Terms_Type_Code_6": full_data_hdr_df["G23_336_Terms_Type_Code_6"][index],
		"@G23_333_Terms_Basis_Date_Code_6": full_data_hdr_df["G23_333_Terms_Basis_Date_Code_6"][index],
		"@G23_282_Terms_Start_Date_6": full_data_hdr_df["G23_282_Terms_Start_Date_6"][index],
		"@G23_283_Terms_Due_Date_Qualifier_6": full_data_hdr_df["G23_283_Terms_Due_Date_Qualifier_6"][index],
		"@G23_338_Terms_Discount_Percent_6": full_data_hdr_df["G23_338_Terms_Discount_Percent_6"][index],
		"@G23_370_Terms_Discount_Due_Date_6": full_data_hdr_df["G23_370_Terms_Discount_Due_Date_6"][index],
		"@G23_351_Terms_Discount_Days_Due_6": full_data_hdr_df["G23_351_Terms_Discount_Days_Due_6"][index],
		"@G23_446_Terms_Net_Due_Date_6": full_data_hdr_df["G23_446_Terms_Net_Due_Date_6"][index],
		"@G23_386_Terms_Net_Days_6": full_data_hdr_df["G23_386_Terms_Net_Days_6"][index],
		"@G23_362_Terms_Discount_Amount_6": full_data_hdr_df["G23_362_Terms_Discount_Amount_6"][index],
		"@G23_391_Discounted_Amount_6": full_data_hdr_df["G23_391_Discounted_Amount_6"][index],
		"@G23_390_Amount_Subject_To_Terms_Discount_6": full_data_hdr_df["G23_390_Amount_Subject_To_Terms_Discount_6"][index],
		"@G23_003_Free_Form_Message_6": full_data_hdr_df["G23_003_Free_Form_Message_6"][index],
		"@G23_336_Terms_Type_Code_7": full_data_hdr_df["G23_336_Terms_Type_Code_7"][index],
		"@G23_333_Terms_Basis_Date_Code_7": full_data_hdr_df["G23_333_Terms_Basis_Date_Code_7"][index],
		"@G23_282_Terms_Start_Date_7": full_data_hdr_df["G23_282_Terms_Start_Date_7"][index],
		"@G23_283_Terms_Due_Date_Qualifier_7": full_data_hdr_df["G23_283_Terms_Due_Date_Qualifier_7"][index],
		"@G23_338_Terms_Discount_Percent_7": full_data_hdr_df["G23_338_Terms_Discount_Percent_7"][index],
		"@G23_370_Terms_Discount_Due_Date_7": full_data_hdr_df["G23_370_Terms_Discount_Due_Date_7"][index],
		"@G23_351_Terms_Discount_Days_Due_7": full_data_hdr_df["G23_351_Terms_Discount_Days_Due_7"][index],
		"@G23_446_Terms_Net_Due_Date_7": full_data_hdr_df["G23_446_Terms_Net_Due_Date_7"][index],
		"@G23_386_Terms_Net_Days_7": full_data_hdr_df["G23_386_Terms_Net_Days_7"][index],
		"@G23_362_Terms_Discount_Amount_7": full_data_hdr_df["G23_362_Terms_Discount_Amount_7"][index],
		"@G23_391_Discounted_Amount_7": full_data_hdr_df["G23_391_Discounted_Amount_7"][index],
		"@G23_390_Amount_Subject_To_Terms_Discount_7": full_data_hdr_df["G23_390_Amount_Subject_To_Terms_Discount_7"][index],
		"@G23_003_Free_Form_Message_7": full_data_hdr_df["G23_003_Free_Form_Message_7"][index],
		"@G23_336_Terms_Type_Code_8": full_data_hdr_df["G23_336_Terms_Type_Code_8"][index],
		"@G23_333_Terms_Basis_Date_Code_8": full_data_hdr_df["G23_333_Terms_Basis_Date_Code_8"][index],
		"@G23_282_Terms_Start_Date_8": full_data_hdr_df["G23_282_Terms_Start_Date_8"][index],
		"@G23_283_Terms_Due_Date_Qualifier_8": full_data_hdr_df["G23_283_Terms_Due_Date_Qualifier_8"][index],
		"@G23_338_Terms_Discount_Percent_8": full_data_hdr_df["G23_338_Terms_Discount_Percent_8"][index],
		"@G23_370_Terms_Discount_Due_Date_8": full_data_hdr_df["G23_370_Terms_Discount_Due_Date_8"][index],
		"@G23_351_Terms_Discount_Days_Due_8": full_data_hdr_df["G23_351_Terms_Discount_Days_Due_8"][index],
		"@G23_446_Terms_Net_Due_Date_8": full_data_hdr_df["G23_446_Terms_Net_Due_Date_8"][index],
		"@G23_386_Terms_Net_Days_8": full_data_hdr_df["G23_386_Terms_Net_Days_8"][index],
		"@G23_362_Terms_Discount_Amount_8": full_data_hdr_df["G23_362_Terms_Discount_Amount_8"][index],
		"@G23_391_Discounted_Amount_8": full_data_hdr_df["G23_391_Discounted_Amount_8"][index],
		"@G23_390_Amount_Subject_To_Terms_Discount_8": full_data_hdr_df["G23_390_Amount_Subject_To_Terms_Discount_8"][index],
		"@G23_003_Free_Form_Message_8": full_data_hdr_df["G23_003_Free_Form_Message_8"][index],
		"@G23_336_Terms_Type_Code_9": full_data_hdr_df["G23_336_Terms_Type_Code_9"][index],
		"@G23_333_Terms_Basis_Date_Code_9": full_data_hdr_df["G23_333_Terms_Basis_Date_Code_9"][index],
		"@G23_282_Terms_Start_Date_9": full_data_hdr_df["G23_282_Terms_Start_Date_9"][index],
		"@G23_283_Terms_Due_Date_Qualifier_9": full_data_hdr_df["G23_283_Terms_Due_Date_Qualifier_9"][index],
		"@G23_338_Terms_Discount_Percent_9": full_data_hdr_df["G23_338_Terms_Discount_Percent_9"][index],
		"@G23_370_Terms_Discount_Due_Date_9": full_data_hdr_df["G23_370_Terms_Discount_Due_Date_9"][index],
		"@G23_351_Terms_Discount_Days_Due_9": full_data_hdr_df["G23_351_Terms_Discount_Days_Due_9"][index],
		"@G23_446_Terms_Net_Due_Date_9": full_data_hdr_df["G23_446_Terms_Net_Due_Date_9"][index],
		"@G23_386_Terms_Net_Days_9": full_data_hdr_df["G23_386_Terms_Net_Days_9"][index],
		"@G23_362_Terms_Discount_Amount_9": full_data_hdr_df["G23_362_Terms_Discount_Amount_9"][index],
		"@G23_391_Discounted_Amount_9": full_data_hdr_df["G23_391_Discounted_Amount_9"][index],
		"@G23_390_Amount_Subject_To_Terms_Discount_9": full_data_hdr_df["G23_390_Amount_Subject_To_Terms_Discount_9"][index],
		"@G23_003_Free_Form_Message_9": full_data_hdr_df["G23_003_Free_Form_Message_9"][index],
		"@G23_336_Terms_Type_Code_10": full_data_hdr_df["G23_336_Terms_Type_Code_10"][index],
		"@G23_333_Terms_Basis_Date_Code_10": full_data_hdr_df["G23_333_Terms_Basis_Date_Code_10"][index],
		"@G23_282_Terms_Start_Date_10": full_data_hdr_df["G23_282_Terms_Start_Date_10"][index],
		"@G23_283_Terms_Due_Date_Qualifier_10": full_data_hdr_df["G23_283_Terms_Due_Date_Qualifier_10"][index],
		"@G23_338_Terms_Discount_Percent_10": full_data_hdr_df["G23_338_Terms_Discount_Percent_10"][index],
		"@G23_370_Terms_Discount_Due_Date_10": full_data_hdr_df["G23_370_Terms_Discount_Due_Date_10"][index],
		"@G23_351_Terms_Discount_Days_Due_10": full_data_hdr_df["G23_351_Terms_Discount_Days_Due_10"][index],
		"@G23_446_Terms_Net_Due_Date_10": full_data_hdr_df["G23_446_Terms_Net_Due_Date_10"][index],
		"@G23_386_Terms_Net_Days_10": full_data_hdr_df["G23_386_Terms_Net_Days_10"][index],
		"@G23_362_Terms_Discount_Amount_10": full_data_hdr_df["G23_362_Terms_Discount_Amount_10"][index],
		"@G23_391_Discounted_Amount_10": full_data_hdr_df["G23_391_Discounted_Amount_10"][index],
		"@G23_390_Amount_Subject_To_Terms_Discount_10": full_data_hdr_df["G23_390_Amount_Subject_To_Terms_Discount_10"][index],
		"@G23_003_Free_Form_Message_10": full_data_hdr_df["G23_003_Free_Form_Message_10"][index],
		"@G23_336_Terms_Type_Code_11": full_data_hdr_df["G23_336_Terms_Type_Code_11"][index],
		"@G23_333_Terms_Basis_Date_Code_11": full_data_hdr_df["G23_333_Terms_Basis_Date_Code_11"][index],
		"@G23_282_Terms_Start_Date_11": full_data_hdr_df["G23_282_Terms_Start_Date_11"][index],
		"@G23_283_Terms_Due_Date_Qualifier_11": full_data_hdr_df["G23_283_Terms_Due_Date_Qualifier_11"][index],
		"@G23_338_Terms_Discount_Percent_11": full_data_hdr_df["G23_338_Terms_Discount_Percent_11"][index],
		"@G23_370_Terms_Discount_Due_Date_11": full_data_hdr_df["G23_370_Terms_Discount_Due_Date_11"][index],
		"@G23_351_Terms_Discount_Days_Due_11": full_data_hdr_df["G23_351_Terms_Discount_Days_Due_11"][index],
		"@G23_446_Terms_Net_Due_Date_11": full_data_hdr_df["G23_446_Terms_Net_Due_Date_11"][index],
		"@G23_386_Terms_Net_Days_11": full_data_hdr_df["G23_386_Terms_Net_Days_11"][index],
		"@G23_362_Terms_Discount_Amount_11": full_data_hdr_df["G23_362_Terms_Discount_Amount_11"][index],
		"@G23_391_Discounted_Amount_11": full_data_hdr_df["G23_391_Discounted_Amount_11"][index],
		"@G23_390_Amount_Subject_To_Terms_Discount_11": full_data_hdr_df["G23_390_Amount_Subject_To_Terms_Discount_11"][index],
		"@G23_003_Free_Form_Message_11": full_data_hdr_df["G23_003_Free_Form_Message_11"][index],
		"@G23_336_Terms_Type_Code_12": full_data_hdr_df["G23_336_Terms_Type_Code_12"][index],
		"@G23_333_Terms_Basis_Date_Code_12": full_data_hdr_df["G23_333_Terms_Basis_Date_Code_12"][index],
		"@G23_282_Terms_Start_Date_12": full_data_hdr_df["G23_282_Terms_Start_Date_12"][index],
		"@G23_283_Terms_Due_Date_Qualifier_12": full_data_hdr_df["G23_283_Terms_Due_Date_Qualifier_12"][index],
		"@G23_338_Terms_Discount_Percent_12": full_data_hdr_df["G23_338_Terms_Discount_Percent_12"][index],
		"@G23_370_Terms_Discount_Due_Date_12": full_data_hdr_df["G23_370_Terms_Discount_Due_Date_12"][index],
		"@G23_351_Terms_Discount_Days_Due_12": full_data_hdr_df["G23_351_Terms_Discount_Days_Due_12"][index],
		"@G23_446_Terms_Net_Due_Date_12": full_data_hdr_df["G23_446_Terms_Net_Due_Date_12"][index],
		"@G23_386_Terms_Net_Days_12": full_data_hdr_df["G23_386_Terms_Net_Days_12"][index],
		"@G23_362_Terms_Discount_Amount_12": full_data_hdr_df["G23_362_Terms_Discount_Amount_12"][index],
		"@G23_391_Discounted_Amount_12": full_data_hdr_df["G23_391_Discounted_Amount_12"][index],
		"@G23_390_Amount_Subject_To_Terms_Discount_12": full_data_hdr_df["G23_390_Amount_Subject_To_Terms_Discount_12"][index],
		"@G23_003_Free_Form_Message_12": full_data_hdr_df["G23_003_Free_Form_Message_12"][index],
		"@G23_336_Terms_Type_Code_13": full_data_hdr_df["G23_336_Terms_Type_Code_13"][index],
		"@G23_333_Terms_Basis_Date_Code_13": full_data_hdr_df["G23_333_Terms_Basis_Date_Code_13"][index],
		"@G23_282_Terms_Start_Date_13": full_data_hdr_df["G23_282_Terms_Start_Date_13"][index],
		"@G23_283_Terms_Due_Date_Qualifier_13": full_data_hdr_df["G23_283_Terms_Due_Date_Qualifier_13"][index],
		"@G23_338_Terms_Discount_Percent_13": full_data_hdr_df["G23_338_Terms_Discount_Percent_13"][index],
		"@G23_370_Terms_Discount_Due_Date_13": full_data_hdr_df["G23_370_Terms_Discount_Due_Date_13"][index],
		"@G23_351_Terms_Discount_Days_Due_13": full_data_hdr_df["G23_351_Terms_Discount_Days_Due_13"][index],
		"@G23_446_Terms_Net_Due_Date_13": full_data_hdr_df["G23_446_Terms_Net_Due_Date_13"][index],
		"@G23_386_Terms_Net_Days_13": full_data_hdr_df["G23_386_Terms_Net_Days_13"][index],
		"@G23_362_Terms_Discount_Amount_13": full_data_hdr_df["G23_362_Terms_Discount_Amount_13"][index],
		"@G23_391_Discounted_Amount_13": full_data_hdr_df["G23_391_Discounted_Amount_13"][index],
		"@G23_390_Amount_Subject_To_Terms_Discount_13": full_data_hdr_df["G23_390_Amount_Subject_To_Terms_Discount_13"][index],
		"@G23_003_Free_Form_Message_13": full_data_hdr_df["G23_003_Free_Form_Message_13"][index],
		"@G23_336_Terms_Type_Code_14": full_data_hdr_df["G23_336_Terms_Type_Code_14"][index],
		"@G23_333_Terms_Basis_Date_Code_14": full_data_hdr_df["G23_333_Terms_Basis_Date_Code_14"][index],
		"@G23_282_Terms_Start_Date_14": full_data_hdr_df["G23_282_Terms_Start_Date_14"][index],
		"@G23_283_Terms_Due_Date_Qualifier_14": full_data_hdr_df["G23_283_Terms_Due_Date_Qualifier_14"][index],
		"@G23_338_Terms_Discount_Percent_14": full_data_hdr_df["G23_338_Terms_Discount_Percent_14"][index],
		"@G23_370_Terms_Discount_Due_Date_14": full_data_hdr_df["G23_370_Terms_Discount_Due_Date_14"][index],
		"@G23_351_Terms_Discount_Days_Due_14": full_data_hdr_df["G23_351_Terms_Discount_Days_Due_14"][index],
		"@G23_446_Terms_Net_Due_Date_14": full_data_hdr_df["G23_446_Terms_Net_Due_Date_14"][index],
		"@G23_386_Terms_Net_Days_14": full_data_hdr_df["G23_386_Terms_Net_Days_14"][index],
		"@G23_362_Terms_Discount_Amount_14": full_data_hdr_df["G23_362_Terms_Discount_Amount_14"][index],
		"@G23_391_Discounted_Amount_14": full_data_hdr_df["G23_391_Discounted_Amount_14"][index],
		"@G23_390_Amount_Subject_To_Terms_Discount_14": full_data_hdr_df["G23_390_Amount_Subject_To_Terms_Discount_14"][index],
		"@G23_003_Free_Form_Message_14": full_data_hdr_df["G23_003_Free_Form_Message_14"][index],
		"@G23_336_Terms_Type_Code_15": full_data_hdr_df["G23_336_Terms_Type_Code_15"][index],
		"@G23_333_Terms_Basis_Date_Code_15": full_data_hdr_df["G23_333_Terms_Basis_Date_Code_15"][index],
		"@G23_282_Terms_Start_Date_15": full_data_hdr_df["G23_282_Terms_Start_Date_15"][index],
		"@G23_283_Terms_Due_Date_Qualifier_15": full_data_hdr_df["G23_283_Terms_Due_Date_Qualifier_15"][index],
		"@G23_338_Terms_Discount_Percent_15": full_data_hdr_df["G23_338_Terms_Discount_Percent_15"][index],
		"@G23_370_Terms_Discount_Due_Date_15": full_data_hdr_df["G23_370_Terms_Discount_Due_Date_15"][index],
		"@G23_351_Terms_Discount_Days_Due_15": full_data_hdr_df["G23_351_Terms_Discount_Days_Due_15"][index],
		"@G23_446_Terms_Net_Due_Date_15": full_data_hdr_df["G23_446_Terms_Net_Due_Date_15"][index],
		"@G23_386_Terms_Net_Days_15": full_data_hdr_df["G23_386_Terms_Net_Days_15"][index],
		"@G23_362_Terms_Discount_Amount_15": full_data_hdr_df["G23_362_Terms_Discount_Amount_15"][index],
		"@G23_391_Discounted_Amount_15": full_data_hdr_df["G23_391_Discounted_Amount_15"][index],
		"@G23_390_Amount_Subject_To_Terms_Discount_15": full_data_hdr_df["G23_390_Amount_Subject_To_Terms_Discount_15"][index],
		"@G23_003_Free_Form_Message_15": full_data_hdr_df["G23_003_Free_Form_Message_15"][index],
		"@G23_336_Terms_Type_Code_16": full_data_hdr_df["G23_336_Terms_Type_Code_16"][index],
		"@G23_333_Terms_Basis_Date_Code_16": full_data_hdr_df["G23_333_Terms_Basis_Date_Code_16"][index],
		"@G23_282_Terms_Start_Date_16": full_data_hdr_df["G23_282_Terms_Start_Date_16"][index],
		"@G23_283_Terms_Due_Date_Qualifier_16": full_data_hdr_df["G23_283_Terms_Due_Date_Qualifier_16"][index],
		"@G23_338_Terms_Discount_Percent_16": full_data_hdr_df["G23_338_Terms_Discount_Percent_16"][index],
		"@G23_370_Terms_Discount_Due_Date_16": full_data_hdr_df["G23_370_Terms_Discount_Due_Date_16"][index],
		"@G23_351_Terms_Discount_Days_Due_16": full_data_hdr_df["G23_351_Terms_Discount_Days_Due_16"][index],
		"@G23_446_Terms_Net_Due_Date_16": full_data_hdr_df["G23_446_Terms_Net_Due_Date_16"][index],
		"@G23_386_Terms_Net_Days_16": full_data_hdr_df["G23_386_Terms_Net_Days_16"][index],
		"@G23_362_Terms_Discount_Amount_16": full_data_hdr_df["G23_362_Terms_Discount_Amount_16"][index],
		"@G23_391_Discounted_Amount_16": full_data_hdr_df["G23_391_Discounted_Amount_16"][index],
		"@G23_390_Amount_Subject_To_Terms_Discount_16": full_data_hdr_df["G23_390_Amount_Subject_To_Terms_Discount_16"][index],
		"@G23_003_Free_Form_Message_16": full_data_hdr_df["G23_003_Free_Form_Message_16"][index],
		"@G23_336_Terms_Type_Code_17": full_data_hdr_df["G23_336_Terms_Type_Code_17"][index],
		"@G23_333_Terms_Basis_Date_Code_17": full_data_hdr_df["G23_333_Terms_Basis_Date_Code_17"][index],
		"@G23_282_Terms_Start_Date_17": full_data_hdr_df["G23_282_Terms_Start_Date_17"][index],
		"@G23_283_Terms_Due_Date_Qualifier_17": full_data_hdr_df["G23_283_Terms_Due_Date_Qualifier_17"][index],
		"@G23_338_Terms_Discount_Percent_17": full_data_hdr_df["G23_338_Terms_Discount_Percent_17"][index],
		"@G23_370_Terms_Discount_Due_Date_17": full_data_hdr_df["G23_370_Terms_Discount_Due_Date_17"][index],
		"@G23_351_Terms_Discount_Days_Due_17": full_data_hdr_df["G23_351_Terms_Discount_Days_Due_17"][index],
		"@G23_446_Terms_Net_Due_Date_17": full_data_hdr_df["G23_446_Terms_Net_Due_Date_17"][index],
		"@G23_386_Terms_Net_Days_17": full_data_hdr_df["G23_386_Terms_Net_Days_17"][index],
		"@G23_362_Terms_Discount_Amount_17": full_data_hdr_df["G23_362_Terms_Discount_Amount_17"][index],
		"@G23_391_Discounted_Amount_17": full_data_hdr_df["G23_391_Discounted_Amount_17"][index],
		"@G23_390_Amount_Subject_To_Terms_Discount_17": full_data_hdr_df["G23_390_Amount_Subject_To_Terms_Discount_17"][index],
		"@G23_003_Free_Form_Message_17": full_data_hdr_df["G23_003_Free_Form_Message_17"][index],
		"@G23_336_Terms_Type_Code_18": full_data_hdr_df["G23_336_Terms_Type_Code_18"][index],
		"@G23_333_Terms_Basis_Date_Code_18": full_data_hdr_df["G23_333_Terms_Basis_Date_Code_18"][index],
		"@G23_282_Terms_Start_Date_18": full_data_hdr_df["G23_282_Terms_Start_Date_18"][index],
		"@G23_283_Terms_Due_Date_Qualifier_18": full_data_hdr_df["G23_283_Terms_Due_Date_Qualifier_18"][index],
		"@G23_338_Terms_Discount_Percent_18": full_data_hdr_df["G23_338_Terms_Discount_Percent_18"][index],
		"@G23_370_Terms_Discount_Due_Date_18": full_data_hdr_df["G23_370_Terms_Discount_Due_Date_18"][index],
		"@G23_351_Terms_Discount_Days_Due_18": full_data_hdr_df["G23_351_Terms_Discount_Days_Due_18"][index],
		"@G23_446_Terms_Net_Due_Date_18": full_data_hdr_df["G23_446_Terms_Net_Due_Date_18"][index],
		"@G23_386_Terms_Net_Days_18": full_data_hdr_df["G23_386_Terms_Net_Days_18"][index],
		"@G23_362_Terms_Discount_Amount_18": full_data_hdr_df["G23_362_Terms_Discount_Amount_18"][index],
		"@G23_391_Discounted_Amount_18": full_data_hdr_df["G23_391_Discounted_Amount_18"][index],
		"@G23_390_Amount_Subject_To_Terms_Discount_18": full_data_hdr_df["G23_390_Amount_Subject_To_Terms_Discount_18"][index],
		"@G23_003_Free_Form_Message_18": full_data_hdr_df["G23_003_Free_Form_Message_18"][index],
		"@G23_336_Terms_Type_Code_19": full_data_hdr_df["G23_336_Terms_Type_Code_19"][index],
		"@G23_333_Terms_Basis_Date_Code_19": full_data_hdr_df["G23_333_Terms_Basis_Date_Code_19"][index],
		"@G23_282_Terms_Start_Date_19": full_data_hdr_df["G23_282_Terms_Start_Date_19"][index],
		"@G23_283_Terms_Due_Date_Qualifier_19": full_data_hdr_df["G23_283_Terms_Due_Date_Qualifier_19"][index],
		"@G23_338_Terms_Discount_Percent_19": full_data_hdr_df["G23_338_Terms_Discount_Percent_19"][index],
		"@G23_370_Terms_Discount_Due_Date_19": full_data_hdr_df["G23_370_Terms_Discount_Due_Date_19"][index],
		"@G23_351_Terms_Discount_Days_Due_19": full_data_hdr_df["G23_351_Terms_Discount_Days_Due_19"][index],
		"@G23_446_Terms_Net_Due_Date_19": full_data_hdr_df["G23_446_Terms_Net_Due_Date_19"][index],
		"@G23_386_Terms_Net_Days_19": full_data_hdr_df["G23_386_Terms_Net_Days_19"][index],
		"@G23_362_Terms_Discount_Amount_19": full_data_hdr_df["G23_362_Terms_Discount_Amount_19"][index],
		"@G23_391_Discounted_Amount_19": full_data_hdr_df["G23_391_Discounted_Amount_19"][index],
		"@G23_390_Amount_Subject_To_Terms_Discount_19": full_data_hdr_df["G23_390_Amount_Subject_To_Terms_Discount_19"][index],
		"@G23_003_Free_Form_Message_19": full_data_hdr_df["G23_003_Free_Form_Message_19"][index],
		"@G23_336_Terms_Type_Code_20": full_data_hdr_df["G23_336_Terms_Type_Code_20"][index],
		"@G23_333_Terms_Basis_Date_Code_20": full_data_hdr_df["G23_333_Terms_Basis_Date_Code_20"][index],
		"@G23_282_Terms_Start_Date_20": full_data_hdr_df["G23_282_Terms_Start_Date_20"][index],
		"@G23_283_Terms_Due_Date_Qualifier_20": full_data_hdr_df["G23_283_Terms_Due_Date_Qualifier_20"][index],
		"@G23_338_Terms_Discount_Percent_20": full_data_hdr_df["G23_338_Terms_Discount_Percent_20"][index],
		"@G23_370_Terms_Discount_Due_Date_20": full_data_hdr_df["G23_370_Terms_Discount_Due_Date_20"][index],
		"@G23_351_Terms_Discount_Days_Due_20": full_data_hdr_df["G23_351_Terms_Discount_Days_Due_20"][index],
		"@G23_446_Terms_Net_Due_Date_20": full_data_hdr_df["G23_446_Terms_Net_Due_Date_20"][index],
		"@G23_386_Terms_Net_Days_20": full_data_hdr_df["G23_386_Terms_Net_Days_20"][index],
		"@G23_362_Terms_Discount_Amount_20": full_data_hdr_df["G23_362_Terms_Discount_Amount_20"][index],
		"@G23_391_Discounted_Amount_20": full_data_hdr_df["G23_391_Discounted_Amount_20"][index],
		"@G23_390_Amount_Subject_To_Terms_Discount_20": full_data_hdr_df["G23_390_Amount_Subject_To_Terms_Discount_20"][index],
		"@G23_003_Free_Form_Message_20": full_data_hdr_df["G23_003_Free_Form_Message_20"][index],
		"@G25_146_Shipment_Method_Of_Payment": full_data_hdr_df["G25_146_Shipment_Method_Of_Payment"][index],
		"@G25_433_FOB_Point_Code": full_data_hdr_df["G25_433_FOB_Point_Code"][index],
		"@G25_434_FOB_Point": full_data_hdr_df["G25_434_FOB_Point"][index],
		"@N1_098_Entity_Identification_Code_1": full_data_hdr_df["N1_098_Entity_Identification_Code_1"][index],
		"@N1_093_Name_1": full_data_hdr_df["N1_093_Name_1"][index],
		"@N1_066_Identification_Code_Qualifier_1": full_data_hdr_df["N1_066_Identification_Code_Qualifier_1"][index],
		"@N1_067_Identification_Code_1": full_data_hdr_df["N1_067_Identification_Code_1"][index],
		"@N3_166_Address_Info_1": full_data_hdr_df["N3_166_Address_Info_1"][index],
		"@N4_019_City_Name_1": full_data_hdr_df["N4_019_City_Name_1"][index],
		"@N4_156_State_Province_Code_1": full_data_hdr_df["N4_156_State_Province_Code_1"][index],
		"@N4_116_Postal_Code_1": full_data_hdr_df["N4_116_Postal_Code_1"][index],
		"@N1_098_Entity_Identification_Code_2": full_data_hdr_df["N1_098_Entity_Identification_Code_2"][index],
		"@N1_093_Name_2": full_data_hdr_df["N1_093_Name_2"][index],
		"@N1_066_Identification_Code_Qualifier_2": full_data_hdr_df["N1_066_Identification_Code_Qualifier_2"][index],
		"@N1_067_Identification_Code_2": full_data_hdr_df["N1_067_Identification_Code_2"][index],
		"@N3_166_Address_Info_2": full_data_hdr_df["N3_166_Address_Info_2"][index],
		"@N4_019_City_Name_2": full_data_hdr_df["N4_019_City_Name_2"][index],
		"@N4_156_State_Province_Code_2": full_data_hdr_df["N4_156_State_Province_Code_2"][index],
		"@N4_116_Postal_Code_2": full_data_hdr_df["N4_116_Postal_Code_2"][index],
		"@N1_098_Entity_Identification_Code_3": full_data_hdr_df["N1_098_Entity_Identification_Code_3"][index],
		"@N1_093_Name_3": full_data_hdr_df["N1_093_Name_3"][index],
		"@N1_066_Identification_Code_Qualifier_3": full_data_hdr_df["N1_066_Identification_Code_Qualifier_3"][index],
		"@N1_067_Identification_Code_3": full_data_hdr_df["N1_067_Identification_Code_3"][index],
		"@N3_166_Address_Info_3": full_data_hdr_df["N3_166_Address_Info_3"][index],
		"@N4_019_City_Name_3": full_data_hdr_df["N4_019_City_Name_3"][index],
		"@N4_156_State_Province_Code_3": full_data_hdr_df["N4_156_State_Province_Code_3"][index],
		"@N4_116_Postal_Code_3": full_data_hdr_df["N4_116_Postal_Code_3"][index],
		"@N1_098_Entity_Identification_Code_4": full_data_hdr_df["N1_098_Entity_Identification_Code_4"][index],
		"@N1_093_Name_4": full_data_hdr_df["N1_093_Name_4"][index],
		"@N1_066_Identification_Code_Qualifier_4": full_data_hdr_df["N1_066_Identification_Code_Qualifier_4"][index],
		"@N1_067_Identification_Code_4": full_data_hdr_df["N1_067_Identification_Code_4"][index],
		"@N3_166_Address_Info_4": full_data_hdr_df["N3_166_Address_Info_4"][index],
		"@N4_019_City_Name_4": full_data_hdr_df["N4_019_City_Name_4"][index],
		"@N4_156_State_Province_Code_4": full_data_hdr_df["N4_156_State_Province_Code_4"][index],
		"@N4_116_Postal_Code_4": full_data_hdr_df["N4_116_Postal_Code_4"][index],
		"@N1_098_Entity_Identification_Code_5": full_data_hdr_df["N1_098_Entity_Identification_Code_5"][index],
		"@N1_093_Name_5": full_data_hdr_df["N1_093_Name_5"][index],
		"@N1_066_Identification_Code_Qualifier_5": full_data_hdr_df["N1_066_Identification_Code_Qualifier_5"][index],
		"@N1_067_Identification_Code_5": full_data_hdr_df["N1_067_Identification_Code_5"][index],
		"@N3_166_Address_Info_5": full_data_hdr_df["N3_166_Address_Info_5"][index],
		"@N4_019_City_Name_5": full_data_hdr_df["N4_019_City_Name_5"][index],
		"@N4_156_State_Province_Code_5": full_data_hdr_df["N4_156_State_Province_Code_5"][index],
		"@N4_116_Postal_Code_5": full_data_hdr_df["N4_116_Postal_Code_5"][index],
		"@N1_098_Entity_Identification_Code_6": full_data_hdr_df["N1_098_Entity_Identification_Code_6"][index],
		"@N1_093_Name_6": full_data_hdr_df["N1_093_Name_6"][index],
		"@N1_066_Identification_Code_Qualifier_6": full_data_hdr_df["N1_066_Identification_Code_Qualifier_6"][index],
		"@N1_067_Identification_Code_6": full_data_hdr_df["N1_067_Identification_Code_6"][index],
		"@N3_166_Address_Info_6": full_data_hdr_df["N3_166_Address_Info_6"][index],
		"@N4_019_City_Name_6": full_data_hdr_df["N4_019_City_Name_6"][index],
		"@N4_156_State_Province_Code_6": full_data_hdr_df["N4_156_State_Province_Code_6"][index],
		"@N4_116_Postal_Code_6": full_data_hdr_df["N4_116_Postal_Code_6"][index],
		"@N1_098_Entity_Identification_Code_7": full_data_hdr_df["N1_098_Entity_Identification_Code_7"][index],
		"@N1_093_Name_7": full_data_hdr_df["N1_093_Name_7"][index],
		"@N1_066_Identification_Code_Qualifier_7": full_data_hdr_df["N1_066_Identification_Code_Qualifier_7"][index],
		"@N1_067_Identification_Code_7": full_data_hdr_df["N1_067_Identification_Code_7"][index],
		"@N3_166_Address_Info_7": full_data_hdr_df["N3_166_Address_Info_7"][index],
		"@N4_019_City_Name_7": full_data_hdr_df["N4_019_City_Name_7"][index],
		"@N4_156_State_Province_Code_7": full_data_hdr_df["N4_156_State_Province_Code_7"][index],
		"@N4_116_Postal_Code_7": full_data_hdr_df["N4_116_Postal_Code_7"][index],
		"@G31_382_Number_Of_Units_Shipped_1": full_data_hdr_df["G31_382_Number_Of_Units_Shipped_1"][index],
		"@G31_355_Unit_Or_Basis_For_Measurement_Code_1_1": full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Code_1_1"][index],
		"@G31_081_Weight_1": full_data_hdr_df["G31_081_Weight_1"][index],
		"@G31_355_Unit_Or_Basis_For_Measurement_Code_2_1": full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Code_2_1"][index],
		"@G31_183_Volume_1": full_data_hdr_df["G31_183_Volume_1"][index],
		"@G31_355_Unit_Or_Basis_For_Measurement_Code_3_1": full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Code_3_1"][index],
		"@G33_361_Amount_1": full_data_hdr_df["G33_361_Amount_1"][index],
		"@G31_382_Number_Of_Units_Shipped_2": full_data_hdr_df["G31_382_Number_Of_Units_Shipped_2"][index],
		"@G31_355_Unit_Or_Basis_For_Measurement_Code_1_2": full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Code_1_2"][index],
		"@G31_081_Weight_2": full_data_hdr_df["G31_081_Weight_2"][index],
		"@G31_355_Unit_Or_Basis_For_Measurement_Code_2_2": full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Code_2_2"][index],
		"@G31_183_Volume_2": full_data_hdr_df["G31_183_Volume_2"][index],
		"@G31_355_Unit_Or_Basis_For_Measurement_Code_3_2": full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Code_3_2"][index],
		"@G33_361_Amount_2": full_data_hdr_df["G33_361_Amount_2"][index],
		"@G31_382_Number_Of_Units_Shipped_3": full_data_hdr_df["G31_382_Number_Of_Units_Shipped_3"][index],
		"@G31_355_Unit_Or_Basis_For_Measurement_Code_1_3": full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Code_1_3"][index],
		"@G31_081_Weight_3": full_data_hdr_df["G31_081_Weight_3"][index],
		"@G31_355_Unit_Or_Basis_For_Measurement_Code_2_3": full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Code_2_3"][index],
		"@G31_183_Volume_3": full_data_hdr_df["G31_183_Volume_3"][index],
		"@G31_355_Unit_Or_Basis_For_Measurement_Code_3_3": full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Code_3_3"][index],
		"@G33_361_Amount_3": full_data_hdr_df["G33_361_Amount_3"][index],
		"@N9_128_Reference_Number_Qualifier_Numeric_Code_1": full_data_hdr_df["N9_128_Reference_Number_Qualifier_Numeric_Code_1"][index],
		"@N9_128_Reference_Number_Qualifier_Numeric_Code_2": full_data_hdr_df["N9_128_Reference_Number_Qualifier_Numeric_Code_2"][index],
		"@N9_128_Reference_Number_Qualifier_Numeric_Code_3": full_data_hdr_df["N9_128_Reference_Number_Qualifier_Numeric_Code_3"][index],
		"@N9_128_Reference_Number_Qualifier_Numeric_Code_4": full_data_hdr_df["N9_128_Reference_Number_Qualifier_Numeric_Code_4"][index],
		"@N9_128_Reference_Number_Qualifier_Numeric_Code_5": full_data_hdr_df["N9_128_Reference_Number_Qualifier_Numeric_Code_5"][index],
		"@N9_128_Reference_Number_Qualifier_Numeric_Code_6": full_data_hdr_df["N9_128_Reference_Number_Qualifier_Numeric_Code_6"][index],
		"@N9_128_Reference_Number_Qualifier_Numeric_Code_7": full_data_hdr_df["N9_128_Reference_Number_Qualifier_Numeric_Code_7"][index],
		"@N9_128_Reference_Number_Qualifier_Numeric_Code_8": full_data_hdr_df["N9_128_Reference_Number_Qualifier_Numeric_Code_8"][index],
		"@N9_128_Reference_Number_Qualifier_Numeric_Code_9": full_data_hdr_df["N9_128_Reference_Number_Qualifier_Numeric_Code_9"][index],
		"@N9_128_Reference_Number_Qualifier_Numeric_Code_10": full_data_hdr_df["N9_128_Reference_Number_Qualifier_Numeric_Code_10"][index],
		"@NTE_363_Note_Reference_Numeric_Code_1": full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_1"][index],
		"@NTE_363_Note_Reference_Numeric_Code_2": full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_2"][index],
		"@NTE_363_Note_Reference_Numeric_Code_3": full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_3"][index],
		"@NTE_363_Note_Reference_Numeric_Code_4": full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_4"][index],
		"@NTE_363_Note_Reference_Numeric_Code_5": full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_5"][index],
		"@NTE_363_Note_Reference_Numeric_Code_6": full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_6"][index],
		"@NTE_363_Note_Reference_Numeric_Code_7": full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_7"][index],
		"@NTE_363_Note_Reference_Numeric_Code_8": full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_8"][index],
		"@NTE_363_Note_Reference_Numeric_Code_9": full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_9"][index],
		"@NTE_363_Note_Reference_Numeric_Code_10": full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_10"][index],
		"@NTE_363_Note_Reference_Numeric_Code_11": full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_11"][index],
		"@NTE_363_Note_Reference_Numeric_Code_12": full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_12"][index],
		"@NTE_363_Note_Reference_Numeric_Code_13": full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_13"][index],
		"@NTE_363_Note_Reference_Numeric_Code_14": full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_14"][index],
		"@NTE_363_Note_Reference_Numeric_Code_15": full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_15"][index],
		"@NTE_363_Note_Reference_Numeric_Code_16": full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_16"][index],
		"@NTE_363_Note_Reference_Numeric_Code_17": full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_17"][index],
		"@NTE_363_Note_Reference_Numeric_Code_18": full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_18"][index],
		"@NTE_363_Note_Reference_Numeric_Code_19": full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_19"][index],
		"@NTE_363_Note_Reference_Numeric_Code_20": full_data_hdr_df["NTE_363_Note_Reference_Numeric_Code_20"][index],
		"@G27_091_Transportation_Method_Type_Numeric_Code_1": full_data_hdr_df["G27_091_Transportation_Method_Type_Numeric_Code_1"][index],
		"@G27_091_Transportation_Method_Type_Numeric_Code_2": full_data_hdr_df["G27_091_Transportation_Method_Type_Numeric_Code_2"][index],
		"@G27_091_Transportation_Method_Type_Numeric_Code_3": full_data_hdr_df["G27_091_Transportation_Method_Type_Numeric_Code_3"][index],
		"@G27_091_Transportation_Method_Type_Numeric_Code_4": full_data_hdr_df["G27_091_Transportation_Method_Type_Numeric_Code_4"][index],
		"@G27_091_Transportation_Method_Type_Numeric_Code_5": full_data_hdr_df["G27_091_Transportation_Method_Type_Numeric_Code_5"][index],
		"@G23_336_Terms_Type_Numeric_Code_1": full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_1"][index],
		"@G23_333_Terms_Basis_Date_Numeric_Code_1": full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_1"][index],
		"@G23_336_Terms_Type_Numeric_Code_2": full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_2"][index],
		"@G23_333_Terms_Basis_Date_Numeric_Code_2": full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_2"][index],
		"@G23_336_Terms_Type_Numeric_Code_3": full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_3"][index],
		"@G23_333_Terms_Basis_Date_Numeric_Code_3": full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_3"][index],
		"@G23_336_Terms_Type_Numeric_Code_4": full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_4"][index],
		"@G23_333_Terms_Basis_Date_Numeric_Code_4": full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_4"][index],
		"@G23_336_Terms_Type_Numeric_Code_5": full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_5"][index],
		"@G23_333_Terms_Basis_Date_Numeric_Code_5": full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_5"][index],
		"@G23_336_Terms_Type_Numeric_Code_6": full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_6"][index],
		"@G23_333_Terms_Basis_Date_Numeric_Code_6": full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_6"][index],
		"@G23_336_Terms_Type_Numeric_Code_7": full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_7"][index],
		"@G23_333_Terms_Basis_Date_Numeric_Code_7": full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_7"][index],
		"@G23_336_Terms_Type_Numeric_Code_8": full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_8"][index],
		"@G23_333_Terms_Basis_Date_Numeric_Code_8": full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_8"][index],
		"@G23_336_Terms_Type_Numeric_Code_9": full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_9"][index],
		"@G23_333_Terms_Basis_Date_Numeric_Code_9": full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_9"][index],
		"@G23_336_Terms_Type_Numeric_Code_10": full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_10"][index],
		"@G23_333_Terms_Basis_Date_Numeric_Code_10": full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_10"][index],
		"@G23_336_Terms_Type_Numeric_Code_11": full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_11"][index],
		"@G23_333_Terms_Basis_Date_Numeric_Code_11": full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_11"][index],
		"@G23_336_Terms_Type_Numeric_Code_12": full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_12"][index],
		"@G23_333_Terms_Basis_Date_Numeric_Code_12": full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_12"][index],
		"@G23_336_Terms_Type_Numeric_Code_13": full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_13"][index],
		"@G23_333_Terms_Basis_Date_Numeric_Code_13": full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_13"][index],
		"@G23_336_Terms_Type_Numeric_Code_14": full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_14"][index],
		"@G23_333_Terms_Basis_Date_Numeric_Code_14": full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_14"][index],
		"@G23_336_Terms_Type_Numeric_Code_15": full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_15"][index],
		"@G23_333_Terms_Basis_Date_Numeric_Code_15": full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_15"][index],
		"@G23_336_Terms_Type_Numeric_Code_16": full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_16"][index],
		"@G23_333_Terms_Basis_Date_Numeric_Code_16": full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_16"][index],
		"@G23_336_Terms_Type_Numeric_Code_17": full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_17"][index],
		"@G23_333_Terms_Basis_Date_Numeric_Code_17": full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_17"][index],
		"@G23_336_Terms_Type_Numeric_Code_18": full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_18"][index],
		"@G23_333_Terms_Basis_Date_Numeric_Code_18": full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_18"][index],
		"@G23_336_Terms_Type_Numeric_Code_19": full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_19"][index],
		"@G23_333_Terms_Basis_Date_Numeric_Code_19": full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_19"][index],
		"@G23_336_Terms_Type_Numeric_Code_20": full_data_hdr_df["G23_336_Terms_Type_Numeric_Code_20"][index],
		"@G23_333_Terms_Basis_Date_Numeric_Code_20": full_data_hdr_df["G23_333_Terms_Basis_Date_Numeric_Code_20"][index],
		"@G25_433_FOB_Point_Numeric_Code": full_data_hdr_df["G25_433_FOB_Point_Numeric_Code"][index],
		"@N1_098_Entity_Identification_Numeric_Code_1": full_data_hdr_df["N1_098_Entity_Identification_Numeric_Code_1"][index],
		"@N1_066_Identification_Numeric_Code_Qualifier_1": full_data_hdr_df["N1_066_Identification_Numeric_Code_Qualifier_1"][index],
		"@N1_098_Entity_Identification_Numeric_Code_2": full_data_hdr_df["N1_098_Entity_Identification_Numeric_Code_2"][index],
		"@N1_066_Identification_Numeric_Code_Qualifier_2": full_data_hdr_df["N1_066_Identification_Numeric_Code_Qualifier_2"][index],
		"@N1_098_Entity_Identification_Numeric_Code_3": full_data_hdr_df["N1_098_Entity_Identification_Numeric_Code_3"][index],
		"@N1_066_Identification_Numeric_Code_Qualifier_3": full_data_hdr_df["N1_066_Identification_Numeric_Code_Qualifier_3"][index],
		"@N1_098_Entity_Identification_Numeric_Code_4": full_data_hdr_df["N1_098_Entity_Identification_Numeric_Code_4"][index],
		"@N1_066_Identification_Numeric_Code_Qualifier_4": full_data_hdr_df["N1_066_Identification_Numeric_Code_Qualifier_4"][index],
		"@N1_098_Entity_Identification_Numeric_Code_5": full_data_hdr_df["N1_098_Entity_Identification_Numeric_Code_5"][index],
		"@N1_066_Identification_Numeric_Code_Qualifier_5": full_data_hdr_df["N1_066_Identification_Numeric_Code_Qualifier_5"][index],
		"@N1_098_Entity_Identification_Numeric_Code_6": full_data_hdr_df["N1_098_Entity_Identification_Numeric_Code_6"][index],
		"@N1_066_Identification_Numeric_Code_Qualifier_6": full_data_hdr_df["N1_066_Identification_Numeric_Code_Qualifier_6"][index],
		"@N1_067_Identification_Numeric_Code_6": full_data_hdr_df["N1_067_Identification_Numeric_Code_6"][index],
		"@N1_098_Entity_Identification_Numeric_Code_7": full_data_hdr_df["N1_098_Entity_Identification_Numeric_Code_7"][index],
		"@N1_066_Identification_Numeric_Code_Qualifier_7": full_data_hdr_df["N1_066_Identification_Numeric_Code_Qualifier_7"][index],
		"@N1_067_Identification_Numeric_Code_7": full_data_hdr_df["N1_067_Identification_Numeric_Code_7"][index],
		"@G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_1_1": full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_1_1"][index],
		"@G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_2_1": full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_2_1"][index],
		"@G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_3_1": full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_3_1"][index],
		"@G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_1_2": full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_1_2"][index],
		"@G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_2_2": full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_2_2"][index],
		"@G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_3_2": full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_3_2"][index],
		"@G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_1_3": full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_1_3"][index],
		"@G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_2_3": full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_2_3"][index],
		"@G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_3_3": full_data_hdr_df["G31_355_Unit_Or_Basis_For_Measurement_Numeric_Code_3_3"][index]
		}
		
		cursor_1.execute(
						"""EXEC [barney].[sandbox_mp].[dbo].[z__ATG_EDI_Invoice_Hdr_Upsert_20240209] ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
						?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
						?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
						?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
						?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
						?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
						?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
						?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
						?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
						?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
						?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?""",
						[str(parameters[p]) for p in parameters]
					)
		cursor_1.commit()
	# except Exception as e:
	# 	fail_count += 1
	# 	print(e)
	# 	print(f"Fails: {fail_count:,}")

In [ ]:
fails = []
for index in full_data_hdr_allowance_df.index:
    #try:
        if(int(index)%1000==0):
            clear_output(wait=False)
            print(f"{index:,}")
        parameters = {
        '@ATG_Hdr_Ref':int(full_data_hdr_allowance_df["ATG_Hdr_Ref"][index]), 
        '@G72_340_Allowance_Or_Charge_Code':full_data_hdr_allowance_df["G72_340_Allowance_Or_Charge_Code"][index], 
        '@G72_331_Allowance_Or_Charge_Method_Of_Handling_Code':full_data_hdr_allowance_df["G72_331_Allowance_Or_Charge_Method_Of_Handling_Code"][index], 
        '@G72_359_Allowance_Or_Charge_Rate':full_data_hdr_allowance_df["G72_359_Allowance_Or_Charge_Rate"][index], 
        '@G72_339_Allowance_Or_Charge_Quantity':full_data_hdr_allowance_df["G72_339_Allowance_Or_Charge_Quantity"][index],
        '@G72_355_Unit_Or_Basis_For_Measurement_Code':full_data_hdr_allowance_df["G72_355_Unit_Or_Basis_For_Measurement_Code"][index], 
        '@G72_360_Allowance_Or_Charge_Total_Amount':full_data_hdr_allowance_df["G72_360_Allowance_Or_Charge_Total_Amount"][index], 
        '@G73_369_Free_Form_Description':full_data_hdr_allowance_df["G73_369_Free_Form_Description"][index], 
        '@G72_340_Allowance_Or_Charge_Numeric_Code':full_data_hdr_allowance_df["G72_340_Allowance_Or_Charge_Numeric_Code"][index], 
        '@G72_331_Allowance_Or_Charge_Method_Of_Handling_Numeric_Code':full_data_hdr_allowance_df["G72_331_Allowance_Or_Charge_Method_Of_Handling_Numeric_Code"][index], 
        '@G72_355_Unit_Or_Basis_For_Measurement_Numeric_Code':full_data_hdr_allowance_df["G72_355_Unit_Or_Basis_For_Measurement_Numeric_Code"][index]
        }
        cursor_1.execute(
                        "EXEC [barney].[sandbox_mp].[dbo].[z__ATG_EDI_Invoice_Hdr_Allowances_Upsert_20240209] ?,?,?,?,?,?,?,?,?,?,?",
                        [str(parameters[p]) for p in parameters]
                    )
        cursor_1.commit()
    # except Exception as e:
    #     fail_count += 1
    #     print(e)
    #     print(f"Fails: {fail_count:,}")

In [ ]:
fails = []
for index in full_data_dtl_allowance_df.index:
    #try:
        parameters = {
        '@ATG_Hdr_Ref':int(full_data_dtl_allowance_df["ATG_Hdr_Ref"][index]), 
        '@ATG_Dtl_Ref':int(full_data_dtl_allowance_df["ATG_Dtl_Ref"][index]),
        '@G72_340_Allowance_Or_Charge_Code':full_data_dtl_allowance_df["G72_340_Allowance_Or_Charge_Code"][index], 
        '@G72_331_Allowance_Or_Charge_Method_Of_Handling_Code':full_data_dtl_allowance_df["G72_331_Allowance_Or_Charge_Method_Of_Handling_Code"][index], 
        '@G72_359_Allowance_Or_Charge_Rate':full_data_dtl_allowance_df["G72_359_Allowance_Or_Charge_Rate"][index], 
        '@G72_339_Allowance_Charge_Quantity':full_data_dtl_allowance_df["G72_339_Allowance_Charge_Quantity"][index],
        '@G72_355_Unit_Basis_For_Measurement':full_data_dtl_allowance_df["G72_355_Unit_Basis_For_Measurement"][index], 
        '@G72_360_Allowance_Or_Charge_Total_Amount':full_data_dtl_allowance_df["G72_360_Allowance_Or_Charge_Total_Amount"][index], 
        '@G73_369_Free_Form_Description':full_data_dtl_allowance_df["G73_369_Free_Form_Description"][index], 
        '@G72_340_Allowance_Or_Charge_Numeric_Code':full_data_dtl_allowance_df["G72_340_Allowance_Or_Charge_Numeric_Code"][index], 
        '@G72_331_Allowance_Or_Charge_Method_Of_Handling_Numeric_Code':full_data_dtl_allowance_df["G72_331_Allowance_Or_Charge_Method_Of_Handling_Numeric_Code"][index], 
        '@G72_355_Unit_Or_Basis_For_Measurement_Numeric_Code':full_data_dtl_allowance_df["G72_355_Unit_Or_Basis_For_Measurement_Numeric_Code"][index]
        }
        cursor_1.execute(
                        "EXEC [barney].[sandbox_mp].[dbo].[z__ATG_EDI_Invoice_Dtl_Allowances_Upsert_20240209] ?,?,?,?,?,?,?,?,?,?,?,?",
                        [str(parameters[p]) for p in parameters]
                    )
        cursor_1.commit()
    # except Exception as e:
    #     fail_count += 1
    #     print(e)
    #     print(f"Fails: {fail_count:,}")